# **Dataset Characterization**

## _ETL Workflow Notebook 2_

## Content:
1. Dataframe general characterization; 
2. Characterizing categorical variables; 
3. Plotting bar charts;
4. Calculating cumulative statistics;
5. Dealing with missing values; 
6. Obtaining the correlation plot; 
7. Obtaining scatter plots and simple linear regressions; 
8. Visualizing time series; 
9. Visualizing histograms; 
10. Testing normality and visualizing the probability plot;
11. Testing and visualizing probability plots for different statistical distributions;
12. Filtering (selecting); or renaming columns; 
13. Dropping specific columns or rows from the dataframe; 
14. Removing duplicate rows from the dataframe.

Marco Cesar Prado Soares, Data Scientist Specialist - Bayer Crop Science LATAM
- marcosoares.feq@gmail.com
- marco.soares@bayer.com

Install statsmodels library

In [ ]:
! pip install statsmodels

Install tensorflow library

In [ ]:
! pip install tensorflow

Install Keras library

In [ ]:
! pip install keras

Install SHAP library

In [ ]:
! pip install shap

In [ ]:
#check the version of the package
! pip show shap

In [ ]:
# Upgrade to the most recent library versions, if a given module is not present and analysis cannot be
# executed.
! pip install pip --upgrade
! pip install tensorflow --upgrade
! pip install keras --upgrade
! pip install shap --upgrade
! pip install sklearn --upgrade
! pip install pandas --upgrade
! pip install numpy --upgrade
! pip install matplotlib --upgrade
! pip install seaborn --upgrade
! pip install scipy --upgrade
! pip install statsmodels --upgrade

## **Load Python Libraries in Global Context**

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **Function for mounting Google Drive or S3 (AWS Simple Storage Service) bucket**

In [23]:
def mount_storage_system (source = 'aws', path_to_store_imported_s3_bucket = '', s3_bucket_name = None, s3_obj_prefix = None):
    
    # source = 'google' for mounting the google drive;
    # source = 'aws' for mounting an AWS S3 bucket.
    
    # THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN source == 'aws'
    
    # path_to_store_imported_s3_bucket: path of the Python environment to which the
    # S3 bucket contents will be imported. If it is None, or if 
    # path_to_store_imported_s3_bucket = '/', bucket will be imported to the root path. 
    # Alternatively, input the path as a string (in quotes). e.g. 
    # path_to_store_imported_s3_bucket = 'copied_s3_bucket'
    
    # s3_bucket_name = None.
    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"
    
    # s3_obj_prefix = None. Keep it None or as an empty string (s3_obj_key_prefix = '')
    # to import the whole bucket content, instead of a single object from it.
    # Alternatively, set it as a string containing the subfolder from the bucket to import:
    # Suppose that your bucket (admin-created) has four objects with the following object 
    # keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
    # s3-dg.pdf. The s3-dg.pdf key does not have a prefix, so its object appears directly 
    # at the root level of the bucket. If you open the Development/ folder, you see 
    # the Projects.xlsx object in it.
    # Check Amazon documentation:
    # https://docs.aws.amazon.com/AmazonS3/latest/userguide/object-keys.html
    
    # In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
    # where 'bucket' is the bucket's name, key_prefix = 'my_path/.../', without the
    # 'file.csv' (file name with extension) last part.
    
    # So, declare the prefix as S3_OBJECT_FOLDER_PREFIX to import only files from
    # a given folder (directory) of the bucket.
    # DO NOT PUT A SLASH before (to the right of) the prefix;
    # DO NOT ADD THE BUCKET'S NAME TO THE right of the prefix:
    # S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/"

    # Alternatively, provide the full path of a given file if you want to import only it:
    # S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/my_file.ext"
    # where my_file is the file's name, and ext is its extension.


    # Attention: after running this function for fetching AWS Simple Storage System (S3), 
    # your 'AWS Access key ID' and your 'Secret access key' will be requested.
    # The 'Secret access key' will be hidden through dots, so it cannot be visualized or copied by
    # other users. On the other hand, the same is not true for 'Access key ID', the bucket's name 
    # and the prefix. All of these are sensitive information from the organization.
    # Therefore, after importing the information, always remember of cleaning the output of this cell
    # and of removing such information from the strings.
    # Remember that these data may contain privilege for accessing the information, so it should not
    # be used for non-authorized people.

    # Also, remember of deleting the imported files from the workspace after finishing the analysis.
    # The costs for storing the files in S3 is quite inferior than those for storing directly in the
    # workspace. Also, files stored in S3 may be accessed for other users than those with access to
    # the notebook's workspace.
    
    
    if (source == 'google'):
        
        from google.colab import drive
        # Google Colab library must be imported only in case it is
        # going to be used, for avoiding AWS compatibility issues.
        
        print("Associate the Python environment to your Google Drive account, and authorize the access in the opened window.")
        
        drive.mount('/content/drive')
        
        print("Now your Python environment is connected to your Google Drive: the root directory of your environment is now the root of your Google Drive.")
        print("In Google Colab, navigate to the folder icon (\'Files\') of the left navigation menu to find a specific folder or file in your Google Drive.")
        print("Click on the folder or file name and select the elipsis (...) icon on the right of the name to reveal the option \'Copy path\', which will give you the path to use as input for loading objects and files on your Python environment.")
        print("Caution: save your files into different directories of the Google Drive. If files are all saved in a same folder or directory, like the root path, they may not be accessible from your Python environment.")
        print("If you still cannot see the file after moving it to a different folder, reload the environment.")
    
    elif (source == 'aws'):
        
        import os
        import boto3
        # boto3 is AWS S3 Python SDK
        # sagemaker and boto3 libraries must be imported only in case 
        # they are going to be used, for avoiding 
        # Google Colab compatibility issues.
        from getpass import getpass

        # Check if path_to_store_imported_s3_bucket is None. If it is, make it the root directory:
        if ((path_to_store_imported_s3_bucket is None)|(str(path_to_store_imported_s3_bucket) == "/")):
            
            # For the S3 buckets, the path should not start with slash. Assign the empty
            # string instead:
            path_to_store_imported_s3_bucket = ""
            print("Bucket\'s content will be copied to the notebook\'s root directory.")
        
        elif (str(path_to_store_imported_s3_bucket) == ""):
            # Guarantee that the path is the empty string.
            # Avoid accessing the else condition, what would raise an error
            # since the empty string has no character of index 0
            path_to_store_imported_s3_bucket = str(path_to_store_imported_s3_bucket)
            print("Bucket\'s content will be copied to the notebook\'s root directory.")
        
        else:
            # Use the str attribute to guarantee that the path was read as a string:
            path_to_store_imported_s3_bucket = str(path_to_store_imported_s3_bucket)
            
            if(path_to_store_imported_s3_bucket[0] == "/"):
                # the first character is the slash. Let's remove it

                # In AWS, neither the prefix nor the path to which the file will be imported
                # (file from S3 to workspace) or from which the file will be exported to S3
                # (the path in the notebook's workspace) may start with slash, or the operation
                # will not be concluded. Then, we have to remove this character if it is present.

                # The slash is character 0. Then, we want all characters from character 1 (the
                # second) to character len(str(path_to_store_imported_s3_bucket)) - 1, the index
                # of the last character. So, we can slice the string from position 1 to position
                # the slicing syntax is: string[1:] - all string characters from character 1
                # string[:10] - all string characters from character 10-1 = 9 (including 9); or
                # string[1:10] - characters from 1 to 9
                # So, slice the whole string, starting from character 1:
                path_to_store_imported_s3_bucket = path_to_store_imported_s3_bucket[1:]
                # attention: even though strings may be seem as list of characters, that can be
                # sliced, we cannot neither simply assign a character to a given position nor delete
                # a character from a position.

        # Ask the user to provide the credentials:
        ACCESS_KEY = input("Enter your AWS Access Key ID here (in the right). It is the value stored in the field \'Access key ID\' from your AWS user credentials CSV file.")
        print("\n") # line break
        SECRET_KEY = getpass("Enter your password (Secret key) here (in the right). It is the value stored in the field \'Secret access key\' from your AWS user credentials CSV file.")
        
        # The use of 'getpass' instead of 'input' hide the password behind dots.
        # So, the password is not visible by other users and cannot be copied.
        
        print("\n")
        print("WARNING: The bucket\'s name, the prefix, the AWS access key ID, and the AWS Secret access key are all sensitive information, which may grant access to protected information from the organization.\n")
        print("After copying data from S3 to your workspace, remember of removing these information from the notebook, specially if it is going to be shared. Also, remember of removing the files from the workspace.\n")
        print("The cost for storing files in Simple Storage Service is quite inferior than the one for storing directly in SageMaker workspace. Also, files stored in S3 may be accessed for other users than those with access the notebook\'s workspace.\n")

        # Check if the user actually provided the mandatory inputs, instead
        # of putting None or empty string:
        if ((ACCESS_KEY is None) | (ACCESS_KEY == '')):
            print("AWS Access Key ID is missing. It is the value stored in the field \'Access key ID\' from your AWS user credentials CSV file.")
            return "error"
        elif ((SECRET_KEY is None) | (SECRET_KEY == '')):
            print("AWS Secret Access Key is missing. It is the value stored in the field \'Secret access key\' from your AWS user credentials CSV file.")
            return "error"
        elif ((s3_bucket_name is None) | (s3_bucket_name == '')):
            print ("Please, enter a valid S3 Bucket\'s name. Do not add sub-directories or folders (prefixes), only the name of the bucket itself.")
            return "error"
        
        else:
            # Use the str attribute to guarantee that all AWS parameters were properly read as strings, and not as
            # other variables (like integers or floats):
            ACCESS_KEY = str(ACCESS_KEY)
            SECRET_KEY = str(SECRET_KEY)
            s3_bucket_name = str(s3_bucket_name)
        
        if(s3_bucket_name[0] == "/"):
                # the first character is the slash. Let's remove it

                # In AWS, neither the prefix nor the path to which the file will be imported
                # (file from S3 to workspace) or from which the file will be exported to S3
                # (the path in the notebook's workspace) may start with slash, or the operation
                # will not be concluded. Then, we have to remove this character if it is present.

                # So, slice the whole string, starting from character 1 (as did for 
                # path_to_store_imported_s3_bucket):
                s3_bucket_name = s3_bucket_name[1:]

        # Remove any possible trailing (white and tab spaces) spaces
        # That may be present in the string. Use the Python string
        # rstrip method, which is the equivalent to the Trim function:
        # When no arguments are provided, the whitespaces and tabulations
        # are the removed characters
        # https://www.w3schools.com/python/ref_string_rstrip.asp?msclkid=ee2d05c3c56811ecb1d2189d9f803f65
        s3_bucket_name = s3_bucket_name.rstrip()
        ACCESS_KEY = ACCESS_KEY.rstrip()
        SECRET_KEY = SECRET_KEY.rstrip()
        # Since the user manually inputs the parameters ACCESS and SECRET_KEY,
        # it is easy to input whitespaces without noticing that.

        # Now process the non-obbligatory parameter.
        # Check if a prefix was passed as input parameter. If so, we must select only the names that start with
        # The prefix.
        # Example: in the bucket 'my_bucket' we have a directory 'dir1'.
        # In the main (root) directory, we have a file 'file1.json' like: '/file1.json'
        # If we pass the prefix 'dir1', we want only the files that start as '/dir1/'
        # such as: 'dir1/file2.json', excluding the file in the main (root) directory and excluding the files in other
        # directories. Also, we want to eliminate the file names with no extensions, like 'dir1/' or 'dir1/dir2',
        # since these object names represent folders or directories, not files.	

        if (s3_obj_prefix is None):
            print ("No prefix, specific object, or subdirectory provided.") 
            print (f"Then, retrieving all content from the bucket \'{s3_bucket_name}\'.\n")
        elif ((s3_obj_prefix == "/") | (s3_obj_prefix == '')):
            # The root directory in the bucket must not be specified starting with the slash
            # If the root "/" or the empty string '' is provided, make
            # it equivalent to None (no directory)
            s3_obj_prefix = None
            print ("No prefix, specific object, or subdirectory provided.") 
            print (f"Then, retrieving all content from the bucket \'{s3_bucket_name}\'.\n")
    
        else:
            # Since there is a prefix, use the str attribute to guarantee that the path was read as a string:
            s3_obj_prefix = str(s3_obj_prefix)
            
            if(s3_obj_prefix[0] == "/"):
                # the first character is the slash. Let's remove it

                # In AWS, neither the prefix nor the path to which the file will be imported
                # (file from S3 to workspace) or from which the file will be exported to S3
                # (the path in the notebook's workspace) may start with slash, or the operation
                # will not be concluded. Then, we have to remove this character if it is present.

                # So, slice the whole string, starting from character 1 (as did for 
                # path_to_store_imported_s3_bucket):
                s3_obj_prefix = s3_obj_prefix[1:]

            # Remove any possible trailing (white and tab spaces) spaces
            # That may be present in the string. Use the Python string
            # rstrip method, which is the equivalent to the Trim function:
            s3_obj_prefix = s3_obj_prefix.rstrip()
            
            # Store the total characters in the prefix string after removing the initial slash
            # and trailing spaces:
            prefix_len = len(s3_obj_prefix)
            
            print("AWS Access Credentials, and bucket\'s prefix, object or subdirectory provided.\n")	

            
        print ("Starting connection with the S3 bucket.\n")
        
        try:
            # Start S3 client as the object 's3_client'
            s3_client = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = SECRET_KEY)
        
            print(f"Credentials accepted by AWS. S3 client successfully started.\n")
            # An object 'data_table.xlsx' in the main (root) directory of the s3_bucket is stored in Python environment as:
            # s3.ObjectSummary(bucket_name='bucket_name', key='data_table.xlsx')
            # The name of each object is stored as the attribute 'key' of the object.
        
        except:
            
            print("Failed to connect to AWS Simple Storage Service (S3). Review if your credentials are correct.")
            print("The variable \'access_key\' must be set as the value (string) stored as \'Access key ID\' in your user security credentials CSV file.")
            print("The variable \'secret_key\' must be set as the value (string) stored as \'Secret access key\' in your user security credentials CSV file.")
        
        try:
            # Connect to the bucket specified as 'bucket_name'.
            # The bucket is started as the object 's3_bucket':
            s3_bucket = s3_client.Bucket(s3_bucket_name)
            print(f"Connection with bucket \'{s3_bucket_name}\' stablished.\n")
            
        except:
            
            print("Failed to connect with the bucket, which usually happens when declaring a wrong bucket\'s name.") 
            print("Check the spelling of your bucket_name string and remember that it must be all in lower-case.\n")
                

        # Then, let's obtain a list of all objects in the bucket (list bucket_objects):
        
        bucket_objects_list = []

        # Loop through all objects of the bucket:
        for stored_obj in s3_bucket.objects.all():
            
            # Loop through all elements 'stored_obj' from s3_bucket.objects.all()
            # Which stores the ObjectSummary for all objects in the bucket s3_bucket:
            # Let's store only the key attribute and use the str function
            # to guarantee that all values were stored as strings.
            bucket_objects_list.append(str(stored_obj.key))
        
        # Now start a support list to store only the elements from
        # bucket_objects_list that are not folders or directories
        # (objects with extensions).
        # If a prefix was provided, only files with that prefix should
        # be added:
        support_list = []
        
        for stored_obj in bucket_objects_list:
            
            # Loop through all elements 'stored_obj' from the list
            # bucket_objects_list

            # Check the file extension.
            file_extension = os.path.splitext(stored_obj)[1][1:]
            
            # The os.path.splitext method splits the string into its FIRST dot (".") to
            # separate the file extension from the full path. Example:
            # "C:/dir1/dir2/data_table.csv" is split into:
            # "C:/dir1/dir2/data_table" (root part) and '.csv' (extension part)
            # https://www.geeksforgeeks.org/python-os-path-splitext-method/?msclkid=2d56198fc5d311ec820530cfa4c6d574

            # os.path.splitext(stored_obj) is a tuple of strings: the first is the complete file
            # root with no extension; the second is the extension starting with a point: '.txt'
            # When we set os.path.splitext(stored_obj)[1], we are selecting the second element of
            # the tuple. By selecting os.path.splitext(stored_obj)[1][1:], we are taking this string
            # from the second character (index 1), eliminating the dot: 'txt'


            # Check if the file extension is not an empty string '' (i.e., that it is different from != the empty
            # string:
            if (file_extension != ''):
                    
                    # The extension is different from the empty string, so it is not neither a folder nor a directory
                    # The object is actually a file and may be copied if it satisfies the prefix condition. If there
                    # is no prefix to check, we may simply copy the object to the list.

                    # If there is a prefix, the first characters of the stored_obj must be the prefix:
                    if not (s3_obj_prefix is None):
                        
                        # Check the characters from the position 0 (1st character) to the position
                        # prefix_len - 1. Since a prefix was declared, we want only the objects that this first portion
                        # corresponds to the prefix. string[i:j] slices the string from index i to index j-1
                        # Then, the 1st portion of the string to check is: string[0:(prefix_len)]

                        # Slice the string stored_obj from position 0 (1st character) to position prefix_len - 1,
                        # The position that the prefix should end.
                        obj_name_first_part = (stored_obj)[0:(prefix_len)]
                        
                        # If this first part is the prefix, then append the object to 
                        # support list:
                        if (obj_name_first_part == (s3_obj_prefix)):

                                support_list.append(stored_obj)

                    else:
                        # There is no prefix, so we can simply append the object to the list:
                        support_list.append(stored_obj)

            
        # Make the objects list the support list itself:
        bucket_objects_list = support_list
            
        # Now, bucket_objects_list contains the names of all objects from the bucket that must be copied.

        print("Finished mapping objects to fetch. Now, all these objects from S3 bucket will be copied to the notebook\'s workspace, in the specified directory.\n")
        print(f"A total of {len(bucket_objects_list)} files were found in the specified bucket\'s prefix (\'{s3_obj_prefix}\').")
        print(f"The first file found is \'{bucket_objects_list[0]}\'; whereas the last file found is \'{bucket_objects_list[len(bucket_objects_list) - 1]}\'.")
            
        # Now, let's try copying the files:
            
        try:
            
            # Loop through all objects in the list bucket_objects and copy them to the workspace:
            for copied_object in bucket_objects_list:

                # Select the object in the bucket previously started as 's3_bucket':
                selected_object = s3_bucket.Object(copied_object)
            
                # Now, copy this object to the workspace:
                # Set the new file_path. Notice that by now, copied_object may be a string like:
                # 'dir1/.../dirN/file_name.ext', where dirN is the n-th directory and ext is the file extension.
                # We want only the file_name to joing with the path to store the imported bucket. So, we can use the
                # str.split method specifying the separator sep = '/' to break the string into a list of substrings.
                # The last element from this list will be 'file_name.ext'
                # https://www.w3schools.com/python/ref_string_split.asp?msclkid=135399b6c63111ecada75d7d91add056

                # 1. Break the copied_object full path into the list object_path_list, using the .split method:
                object_path_list = copied_object.split(sep = "/")

                # 2. Get the last element from this list. Since it has length len(object_path_list) and indexing starts from
                # zero, the index of the last element is (len(object_path_list) - 1):
                fetched_object = object_path_list[(len(object_path_list) - 1)]

                # 3. Finally, join the string fetched_object with the new path (path on the notebook's workspace) to finish
                # The new object's file_path:

                file_path = os.path.join(path_to_store_imported_s3_bucket, fetched_object)

                # Download the selected object to the workspace in the specified file_path
                # The parameter Filename must be input with the path of the copied file, including its name and
                # extension. Example Filename = "/my_table.xlsx" copies a xlsx file named 'my_table' to the notebook's main (root)
                # directory
                selected_object.download_file(Filename = file_path)

                print(f"The file \'{fetched_object}\' was successfully copied to notebook\'s workspace.\n")

                
            print("Finished copying the files from the bucket to the notebook\'s workspace. It may take a couple of minutes untill they be shown in SageMaker environment.\n") 
            print("Do not forget to delete these copies after finishing the analysis. They will remain stored in the bucket.\n")


        except:

            # Run this code for any other exception that may happen (no exception error
            # specified, so any exception runs the following code).
            # Check: https://pythonbasics.org/try-except/?msclkid=4f6b4540c5d011ecb1fe8a4566f632a6
            # for seeing how to handle successive exceptions

            print("Attention! The function raised an exception error, which is probably due to the AWS Simple Storage Service (S3) permissions.")
            print("Before running again this function, check this quick guide for configuring the permission roles in AWS.\n")
            print("It is necessary to create an user with full access permissions to interact with S3 from SageMaker. To configure the User, go to the upper ribbon of AWS, click on Services, and select IAM – Identity and Access Management.")
            print("1. In IAM\'s lateral panel, search for \'Users\' in the group of Access Management.")
            print("2. Click on the \'Add users\' button.")
            print("3. Set an user name in the text box \'User name\'.")
            print("Attention: users and S3 buckets cannot be written in upper case. Also, selecting a name already used by an Amazon user or bucket will raise an error message.\n")
            print("4. In the field \'Select type of Access to AWS\'-\'Select type of AWS credentials\' select the option \'Access key - Programmatic access\'. After that, click on the button \'Next: Permissions\'.")
            print("5. In the field \'Set Permissions\', keep the \'Add user to a group\' button marked.")
            print("6. In the field \'Add user to a group\', click on \'Create group\' (alternatively, you can be added to a group already configured or copy the permissions of another user.")
            print("7. In the text box \'Group\'s name\', set a name for the new group of permissions.")
            print("8. In the search bar below (\'Filter politics\'), search for a politics that fill your needs, and check the option button on the left of this politic. The politics \'AmazonS3FullAccess\' grants full access to the S3 content.")
            print("9. Finally, click on \'Create a group\'.")
            print("10. After the group is created, it will appear with a check box marked, over the previous groups. Keep it marked and click on the button \'Next: Tags\'.")
            print("11. Create and note down the Access key ID and Secret access key. You can also download a comma separated values (CSV) file containing the credentials for future use.")
            print("ATTENTION: These parameters are required for accessing the bucket\'s content from any application, including AWS SageMaker.")
            print("12. Click on \'Next: Review\' and review the user credentials information and permissions.")
            print("13. Click on \'Create user\' and click on the download button to download the CSV file containing the user credentials information.")
            print("The headers of the CSV file (the stored fields) is: \'User name, Password, Access key ID, Secret access key, Console login link\'.")
            print("You need both the values indicated as \'Access key ID\' and as \'Secret access key\' to fetch the S3 bucket.")
            print("\n") # line break
            print("After acquiring the necessary user privileges, use the boto3 library to fetch the bucket from the Python code. boto3 is AWS S3 Python SDK.")
            print("For fetching a specific bucket\'s file use the following code:\n")
            print("1. Set a variable \'access_key\' as the value (string) stored as \'Access key ID\' in your user security credentials CSV file.")
            print("2. Set a variable \'secret_key\' as the value (string) stored as \'Secret access key\' in your user security credentials CSV file.")
            print("3. Set a variable \'bucket_name\' as a string containing only the name of the bucket. Do not add subdirectories, folders (prefixes), or file names.")
            print("Example: if your bucket is named \'my_bucket\' and its main directory contains folders like \'folder1\', \'folder2\', etc, do not declare bucket_name = \'my_bucket/folder1\', even if you only want files from folder1.")
            print("ALWAYS declare only the bucket\'s name: bucket_name = \'my_bucket\'.")
            print("4. Set a variable \'file_path\' containing the path from the bucket\'s subdirectories to the file you want to fetch. Include the file name and its extension.")
            print("If the file is stored in the bucket\'s root (main) directory: file_path = \"my_file.ext\".")
            print("If the path of the file in the bucket is: \'dir1/…/dirN/my_file.ext\', where dirN is the N-th subdirectory, and dir1 is a folder or directory of the main (root) bucket\'s directory: file_path = \"dir1/…/dirN/my_file.ext\".")
            print("Also, we say that \'dir1/…/dirN/\' is the file\'s prefix. Notice that the name of the bucket is never declared here as the path for fetching its content from the Python code.")
            print("5. Set a variable named \'new_path\' to store the path of the file copied to the notebook’s workspace. This path must contain the file name and its extension.")
            print("Example: if you want to copy \'my_file.ext\' to the root directory of the notebook’s workspace, set: new_path = \"/my_file.ext\".")
            print("6. Finally, declare the following code, which refers to the defined variables:\n")

            # Let's use triple quotes to declare a formated string
            example_code = """
                import boto3
                # Start S3 client as the object 's3_client'
                s3_client = boto3.resource('s3', aws_access_key_id = access_key, aws_secret_access_key = secret_key)
                # Connect to the bucket specified as 'bucket_name'.
                # The bucket is started as the object 's3_bucket':
                s3_bucket = s3_client.Bucket(bucket_name)
                # Select the object in the bucket previously started as 's3_bucket':
                selected_object = s3_bucket.Object(file_path)
                # Download the selected object to the workspace in the specified file_path
                # The parameter Filename must be input with the path of the copied file, including its name and
                # extension. Example Filename = "/my_table.xlsx" copies a xlsx file named 'my_table' to the notebook's main (root)
                # directory
                selected_object.download_file(Filename = new_path)
                """

            print(example_code)

            print("An object \'my_file.ext\' in the main (root) directory of the s3_bucket is stored in Python environment as:")
            print("""s3.ObjectSummary(bucket_name='bucket_name', key='my_file.ext'""") 
            # triple quotes to keep the internal quotes without using too much backslashes "\" (the ignore next character)
            print("Then, the name of each object is stored as the attribute \'key\' of the object. To view all objects, we can loop through their \'key\' attributes:\n")
            example_code = """
                # Loop through all objects of the bucket:
                for stored_obj in s3_bucket.objects.all():		
                    # Loop through all elements 'stored_obj' from s3_bucket.objects.all()
                    # Which stores the ObjectSummary for all objects in the bucket s3_bucket:
                    # Print the object’s names:
                    print(stored_obj.key)
                    """

            print(example_code)

                
    else:
        
        print("Select a valid source: \'google\' for mounting Google Drive; or \'aws\' for accessing AWS S3 Bucket.")

# **Function for loading the dataframe**

In [ ]:
def load_pandas_dataframe (file_directory_path, file_name_with_extension, load_txt_file_with_json_format = False, how_missing_values_are_registered = None, has_header = True, decimal_separator = '.', txt_csv_col_sep = "comma", sheet_to_load = None, json_record_path = None, json_field_separator = "_", json_metadata_prefix_list = None):
    
    # Pandas documentation:
    # pd.read_csv: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
    # pd.read_excel: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
    # pd.json_normalize: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html
    # Python JSON documentation:
    # https://docs.python.org/3/library/json.html
    
    import os
    import json
    import numpy as np
    import pandas as pd
    from pandas import json_normalize
    
    ## WARNING: Use this function to load dataframes stored on Excel (xls, xlsx, xlsm, xlsb, odf, ods and odt), 
    ## JSON, txt, or CSV (comma separated values) files.
    
    # file_directory_path - (string, in quotes): input the path of the directory (e.g. folder path) 
    # where the file is stored. e.g. file_directory_path = "/" or file_directory_path = "/folder"
    
    # FILE_NAME_WITH_EXTENSION - (string, in quotes): input the name of the file with the 
    # extension. e.g. FILE_NAME_WITH_EXTENSION = "file.xlsx", or, 
    # FILE_NAME_WITH_EXTENSION = "file.csv", "file.txt", or "file.json"
    # Again, the extensions may be: xls, xlsx, xlsm, xlsb, odf, ods, odt, json, txt or csv.
    
    # load_txt_file_with_json_format = False. Set load_txt_file_with_json_format = True 
    # if you want to read a file with txt extension containing a text formatted as JSON 
    # (but not saved as JSON).
    # WARNING: if load_txt_file_with_json_format = True, all the JSON file parameters of the 
    # function (below) must be set. If not, an error message will be raised.
    
    # HOW_MISSING_VALUES_ARE_REGISTERED = None: keep it None if missing values are registered as None,
    # empty or np.nan. Pandas automatically converts None to NumPy np.nan objects (floats).
    # This parameter manipulates the argument na_values (default: None) from Pandas functions.
    # By default the following values are interpreted as NaN: ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, 
    #‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, 
    # ‘n/a’, ‘nan’, ‘null’.

    # If a different denomination is used, indicate it as a string. e.g.
    # HOW_MISSING_VALUES_ARE_REGISTERED = '.' will convert all strings '.' to missing values;
    # HOW_MISSING_VALUES_ARE_REGISTERED = 0 will convert zeros to missing values.

    # If dict passed, specific per-column NA values. For example, if zero is the missing value
    # only in column 'numeric_col', you can specify the following dictionary:
    # how_missing_values_are_registered = {'numeric-col': 0}
    
    
    # has_header = True if the the imported table has headers (row with columns names).
    # Alternatively, has_header = False if the dataframe does not have header.
    
    # DECIMAL_SEPARATOR = '.' - String. Keep it '.' or None to use the period ('.') as
    # the decimal separator. Alternatively, specify here the separator.
    # e.g. DECIMAL_SEPARATOR = ',' will set the comma as the separator.
    # It manipulates the argument 'decimal' from Pandas functions.
    
    # txt_csv_col_sep = "comma" - This parameter has effect only when the file is a 'txt'
    # or 'csv'. It informs how the different columns are separated.
    # Alternatively, txt_csv_col_sep = "comma", or txt_csv_col_sep = "," 
    # for columns separated by comma;
    # txt_csv_col_sep = "whitespace", or txt_csv_col_sep = " " 
    # for columns separated by simple spaces.
    # You can also set a specific separator as string. For example:
    # txt_csv_col_sep = '\s+'; or txt_csv_col_sep = '\t' (in this last example, the tabulation
    # is used as separator for the columns - '\t' represents the tab character).
    
    # sheet_to_load - This parameter has effect only when for Excel files.
    # keep sheet_to_load = None not to specify a sheet of the file, so that the first sheet
    # will be loaded.
    # sheet_to_load may be an integer or an string (inside quotes). sheet_to_load = 0
    # loads the first sheet (sheet with index 0); sheet_to_load = 1 loads the second sheet
    # of the file (index 1); sheet_to_load = "Sheet1" loads a sheet named as "Sheet1".
    # Declare a number to load the sheet with that index, starting from 0; or declare a
    # name to load the sheet with that name.
    
    ## Parameters for loading JSON files:
    
    # json_record_path (string): manipulate parameter 'record_path' from json_normalize method.
    # Path in each object to list of records. If not passed, data will be assumed to 
    # be an array of records. If a given field from the JSON stores a nested JSON (or a nested
    # dictionary) declare it here to decompose the content of the nested data. e.g. if the field
    # 'books' stores a nested JSON, declare, json_record_path = 'books'
    
    # json_field_separator = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
    # Nested records will generate names separated by sep. 
    # e.g., for json_field_separator = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
    # Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
    # the name of the columns of the dataframe will be formed by concatenating 'main_field', the
    # separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...
    
    # json_metadata_prefix_list: list of strings (in quotes). Manipulates the parameter 
    # 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
    # table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
    # will be repeated in the rows of the dataframe to give the metadata (context) of the rows.
    
    # e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
    # 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
    # Here, there are nested JSONs in the field 'books'. The fields that are not nested
    # are 'name' and 'last'.
    # Then, json_record_path = 'books'
    # json_metadata_prefix_list = ['name', 'last']
    
    
    # Create the complete file path:
    file_path = os.path.join(file_directory_path, file_name_with_extension)
    # Extract the file extension
    file_extension = os.path.splitext(file_path)[1][1:]
    # os.path.splitext(file_path) is a tuple of strings: the first is the complete file
    # root with no extension; the second is the extension starting with a point: '.txt'
    # When we set os.path.splitext(file_path)[1], we are selecting the second element of
    # the tuple. By selecting os.path.splitext(file_path)[1][1:], we are taking this string
    # from the second character (index 1), eliminating the dot: 'txt'
    
    # Check if the decimal separator is None. If it is, set it as '.' (period):
    if (decimal_separator is None):
        decimal_separator = '.'
    
    if ((file_extension == 'txt') | (file_extension == 'csv')): 
        # The operator & is equivalent to 'And' (intersection).
        # The operator | is equivalent to 'Or' (union).
        # pandas.read_csv method must be used.
        if (load_txt_file_with_json_format == True):
            
            print("Reading a txt file containing JSON parsed data. A reading error will be raised if you did not set the JSON parameters.")
            
            with open(file_path, 'r') as opened_file:
                # 'r' stands for read mode; 'w' stands for write mode
                # read the whole file as a string named 'file_full_text'
                file_full_text = opened_file.read()
                # if we used the readlines() method, we would be reading the
                # file by line, not the whole text at once.
                # https://stackoverflow.com/questions/8369219/how-to-read-a-text-file-into-a-string-variable-and-strip-newlines?msclkid=a772c37bbfe811ec9a314e3629df4e1e
                # https://www.tutorialkart.com/python/python-read-file-as-string/#:~:text=example.py%20%E2%80%93%20Python%20Program.%20%23open%20text%20file%20in,and%20prints%20it%20to%20the%20standard%20output.%20Output.?msclkid=a7723a1abfe811ecb68bba01a2b85bd8
                
            #Now, file_full_text is a string containing the full content of the txt file.
            json_file = json.loads(file_full_text)
            # json.load() : This method is used to parse JSON from URL or file.
            # json.loads(): This method is used to parse string with JSON content.
            # e.g. .json.loads() must be used to read a string with JSON and convert it to a flat file
            # like a dataframe.
            # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.
            dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_prefix_list)
        
        else:
            # Not a JSON txt
        
            if (has_header == True):

                if ((txt_csv_col_sep == "comma") | (txt_csv_col_sep == ",")):

                    dataset = pd.read_csv(file_path, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                    # verbose = True for showing number of NA values placed in non-numeric columns.
                    #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                    # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                    # parsing speed by 5-10x.

                elif ((txt_csv_col_sep == "whitespace") | (txt_csv_col_sep == " ")):

                    dataset = pd.read_csv(file_path, delim_whitespace = True, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                    # verbose = True for showing number of NA values placed in non-numeric columns.
                    #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                    # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                    # parsing speed by 5-10x.
                    
                else:
                    
                    try:
                        
                        # Try using the character specified as the argument txt_csv_col_sep:
                        dataset = pd.read_csv(file_path, sep = txt_csv_col_sep, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                        # verbose = True for showing number of NA values placed in non-numeric columns.
                        #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                        # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                        # parsing speed by 5-10x.
                    
                    except:
                        # An error was raised, the separator is not valid
                        print(f"Enter a valid column separator for the {file_extension} file, like: \'comma\' or \'whitespace\'.")


            else:
                # has_header == False

                if ((txt_csv_col_sep == "comma") | (txt_csv_col_sep == ",")):

                    dataset = pd.read_csv(file_path, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                    # verbose = True for showing number of NA values placed in non-numeric columns.
                    #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                    # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                    # parsing speed by 5-10x.
                    
                elif ((txt_csv_col_sep == "whitespace") | (txt_csv_col_sep == " ")):

                    dataset = pd.read_csv(file_path, delim_whitespace = True, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                    # verbose = True for showing number of NA values placed in non-numeric columns.
                    #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                    # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                    # parsing speed by 5-10x.
                    
                else:
                    
                    try:
                        
                        # Try using the character specified as the argument txt_csv_col_sep:
                        dataset = pd.read_csv(file_path, sep = txt_csv_col_sep, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                        # verbose = True for showing number of NA values placed in non-numeric columns.
                        #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                        # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                        # parsing speed by 5-10x.
                    
                    except:
                        # An error was raised, the separator is not valid
                        print(f"Enter a valid column separator for the {file_extension} file, like: \'comma\' or \'whitespace\'.")

    elif (file_extension == 'json'):
        
        with open(file_path, 'r') as opened_file:
            
            json_file = json.load(opened_file)
            # The structure json_file = json.load(open(file_path)) relies on the GC to close the file. That's not a 
            # good idea: If someone doesn't use CPython the garbage collector might not be using refcounting (which 
            # collects unreferenced objects immediately) but e.g. collect garbage only after some time.
            # Since file handles are closed when the associated object is garbage collected or closed 
            # explicitly (.close() or .__exit__() from a context manager) the file will remain open until 
            # the GC kicks in.
            # Using 'with' ensures the file is closed as soon as the block is left - even if an exception 
            # happens inside that block, so it should always be preferred for any real application.
            # source: https://stackoverflow.com/questions/39447362/equivalent-ways-to-json-load-a-file-in-python
            
        # json.load() : This method is used to parse JSON from URL or file.
        # json.loads(): This method is used to parse string with JSON content.
        # Then, json.load for a .json file
        # and json.loads for text file containing json
        # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.   
        dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_prefix_list)
    
    else:
        # If it is not neither a csv nor a txt file, let's assume it is one of different
        # possible Excel files.
        print("Excel file inferred. If an error message is shown, check if a valid file extension was used: \'xlsx\', \'xls\', etc.")
            
        if (sheet_to_load is not None):        
        #Case where the user specifies which sheet of the Excel file should be loaded.
            
            if (has_header == True):
                
                dataset = pd.read_excel(file_path, sheet_name = sheet_to_load, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                # verbose = True for showing number of NA values placed in non-numeric columns.
                #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                # parsing speed by 5-10x.
                
            else:
                #No header
                dataset = pd.read_excel(file_path, sheet_name = sheet_to_load, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                # verbose = True for showing number of NA values placed in non-numeric columns.
                #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                # parsing speed by 5-10x.
        else:
            #No sheet specified
            if (has_header == True):
                
                dataset = pd.read_excel(file_path, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                # verbose = True for showing number of NA values placed in non-numeric columns.
                #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                # parsing speed by 5-10x.
                
            else:
                #No header
                dataset = pd.read_excel(file_path, header = None, na_values = how_missing_values_are_registered, verbose = True, parse_dates = True, infer_datetime_format = True, decimal = decimal_separator)
                # verbose = True for showing number of NA values placed in non-numeric columns.
                #  parse_dates = True: try parsing the index; infer_datetime_format = True : If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in 
                # the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the 
                # parsing speed by 5-10x.
                
    print(f"Dataset extracted from {file_path}. Check the 10 first rows of this dataframe:\n")
    print(dataset.head(10))
    
    return dataset

# **Function for converting JSON object to dataframe**
- Objects may be:
    - String with JSON formatted text;
    - List with nested dictionaries (JSON formatted);
    - Dictionaries, possibly with nested dictionaries (JSON formatted).

In [ ]:
def json_obj_to_dataframe (json_obj_to_convert, json_record_path = None, json_field_separator = "_", json_metadata_prefix_list = None):
    
    import json
    import pandas as pd
    from pandas import json_normalize
    
    # json_obj_to_convert: object containing JSON, or string with JSON content to parse.
    # Objects may be: string with JSON formatted text;
    # list with nested dictionaries (JSON formatted);
    # dictionaries, possibly with nested dictionaries (JSON formatted).
    
    # https://docs.python.org/3/library/json.html
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html#pandas.json_normalize
    
    # json_record_path (string): manipulate parameter 'record_path' from json_normalize method.
    # Path in each object to list of records. If not passed, data will be assumed to 
    # be an array of records. If a given field from the JSON stores a nested JSON (or a nested
    # dictionary) declare it here to decompose the content of the nested data. e.g. if the field
    # 'books' stores a nested JSON, declare, json_record_path = 'books'
    
    # json_field_separator = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
    # Nested records will generate names separated by sep. 
    # e.g., for json_field_separator = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
    # Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
    # the name of the columns of the dataframe will be formed by concatenating 'main_field', the
    # separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...
    
    # json_metadata_prefix_list: list of strings (in quotes). Manipulates the parameter 
    # 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
    # table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
    # will be repeated in the rows of the dataframe to give the metadata (context) of the rows.
    
    # e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
    # 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
    # Here, there are nested JSONs in the field 'books'. The fields that are not nested
    # are 'name' and 'last'.
    # Then, json_record_path = 'books'
    # json_metadata_prefix_list = ['name', 'last']
    
    json_file = json.loads(json_obj_to_convert)
    # json.load() : This method is used to parse JSON from URL or file.
    # json.loads(): This method is used to parse string with JSON content.
    # e.g. .json.loads() must be used to read a string with JSON and convert it to a flat file
    # like a dataframe.
    # check: https://www.pythonpip.com/python-tutorials/how-to-load-json-file-using-python/#:~:text=The%20json.load%20%28%29%20is%20used%20to%20read%20the,and%20alter%20data%20in%20our%20application%20or%20system.
    dataset = json_normalize(json_file, record_path = json_record_path, sep = json_field_separator, meta = json_metadata_prefix_list)
    
    print(f"JSON object {json_obj_to_convert} converted to a flat dataframe object. Check the 10 first rows of this dataframe:\n")
    print(dataset.head(10))
    
    return dataset

# **Function for dataframe general characterization**

In [16]:
def df_gen_charac (df):
    
    import pandas as pd
    
    print("Dataframe 10 first rows:")
    print(df.head(10))
    
    #Line break before next information:
    print("\n")
    df_shape  = df.shape
    print(f"Dataframe shape (rows, columns) = {df_shape}.")
    
    #Line break before next information:
    print("\n")
    df_columns_list = df.columns
    print(f"Dataframe columns list = {df_columns_list}.")
    
    #Line break before next information:
    print("\n")
    df_dtypes = df.dtypes
    print("Dataframe variables types:")
    print(df_dtypes)
    
    #Line break before next information:
    print("\n")
    df_general_statistics = df.describe()
    print("Dataframe general statistics (numerical variables):")
    print(df_general_statistics)
    
    #Line break before next information:
    print("\n")
    df_missing_values = df.isna().sum()
    print("Total of missing values for each feature:")
    print(df_missing_values)
    
    return df_shape, df_columns_list, df_dtypes, df_general_statistics, df_missing_values

# **Function for characterizing categorical variables**

In [17]:
def charac_cat_var (df, categorical_var_name, encode_var = False, new_encoded_col_name = None):
    
    import numpy as np
    import pandas as pd
           
        
            # Encoding syntax:
            # dataset.loc[dataset["CatVar"] == 'Value1', "EncodedColumn"] = 1
            # dataset.loc[boolean_filter, EncodedColumn"] = value,
            # boolean_filter = (dataset["CatVar"] == 'Value1') will be True when the 
            # equality is verified. The .loc method filters the dataframe, accesses the
            # column declared after the comma and inputs the value defined (e.g. value = 1)
            
    
    #WARNING: Use this function to a analyze a single column from a dataframe.
    # The pandas .unique method and the numpy.unique function can handle a single series
    # (column) or 1D arrays.
    
    #WARNING: The first return is the unique values summary/encoding table; 
    # the second one will be the dataframe with the encoded column. This second 
    # dataframe is returned only when encode_var = True
    
    # categorical_var_name: string (inside quotes) containing the name of the column 
    #to be analyzed. e.g. categorical_var_name = "column1"
    
    # encode_var = False - keep it False not to associate an integer value to each 
    # possible category. Alternatively, set encode_var = True to associate an integer
    # starting from zero to each categorical value. This should not be used in case there
    # is no order to be associated. Also, the encoding will be performed in accordance to 
    # the order of occurence in the dataframe df. In case there is no order, the One-Hot
    # Encoding should be used.
    # WARNING: if encoded_var = True, the function will return a new dataframe containing
    # a column correspondent to the column input as 'categorical_var_name' but with the
    # correspondent integer values.
    
    # new_encoded_col_name = None - This parameter shows effects only when encode_var = True.
    # It refers to the column that will be created on the original dataframe containing the
    # integer values associated to the categorical values.
    # Input a string inside quotes to define a name (header) for the column of the summary
    # dataframe to store the integer correspondent to each categorical value.
    # e.g. new_encoded_col_name = "integer_vals". If no name is provided, the standard name
    # formed by the string concatenation categorical_var_name + "_encoded" will be automatically
    # assigned.
    
    # Get unique vals and respective counts.
    # Let's use the more primitive NumPy method, less prone to be updated. It is equivalent
    # to use the pandas .unique method, followed by the pandas .value_counts method.
    
    # Create a copy of the dataframe to eliminate the missing values from it
    df_cleaned = df.dropna(axis = 0)
    
    #Now, check the unique values of the categorical variable:
    unique_vals_list, counts_of_occurences = np.unique(df_cleaned[categorical_var_name], return_counts = True)
    # The functions for counting unique values may raise exception errors when there are
    # missing values. This does not happen when the missing values were incorrectly read as
    # strings.
    
    # Let's normalize the counts_of_occurences array to obtain a correspondent list in percent
    total_sum = np.sum(counts_of_occurences) #np.sum let's sum the whole array
    
    percent_of_occurence = ((counts_of_occurences)/(total_sum)) * 100
    #By dividing the whole array by the total sum, we obtain an array where each element is
    # the fraction of that element in the original array. These values are converted to 
    # percents after being multiplied by 100.
    
    # Let's create a Pandas dataframe summarizing these information:
    # Create the dictionary from the arrays:
    unique_summary = {
                        "Unique_values_of_categorical_variable": unique_vals_list,
                        "Counts_of_occurences": counts_of_occurences,
                        "Percent_of_occurence": percent_of_occurence
                     }
    
    # convert the dictionary to a Pandas dataframe:
    unique_vals_summary = pd.DataFrame(data = unique_summary)
    
    if (encode_var == True):
        
        # Create a column on the unique_vals_summary dataframe to store the integer number,
        # and copy the dataframe index to it. Then, the numbers will go from zero to the
        # index of the last categorical value.
        unique_vals_summary['Value_encoding'] = unique_vals_summary.index
        # pandas .index method creates a range from zero to (len(dataframe) - 1), step = 1.
     
        # Loop through the unique_vals_summary, check the "Unique_values_of_categorical_variable"
        # and the correspondent 'Value_encoding':
        
        for i in range(len(unique_vals_summary) - 1):
            #len(unique_vals_summary) is the total of rows in the dataframe unique_vals_summary.
            # This loop goes from i = 0 (index of the first row), 
            # to i = len(unique_vals_summary) - 1, index of the last row.
            
            # use the .iloc pandas method to subset an element (i, j) from the dataframe.
            # i = row, j = column.
            # j = 0 is the column "Unique_values_of_categorical_variable" (1st column);
            # j = 3 is the column 'Value_encoding' (4th column).
            
            # Select the categorical value (column 0):
            cat_val = unique_vals_summary.iloc[i, 0]
            # Select the encoded value (column 3):
            encoded_val = unique_vals_summary.iloc[i, 3]
            
            # Check if there is no name for the new column
            if (new_encoded_col_name is None):
                
                #if there is not new_encoded_col_name, create one as the default:
                new_encoded_col_name = categorical_var_name + "_encoded"
            
            # Create a boolean filter that checks if the entry on the dataframe corresponds
            # to cat_val:
            boolean_filter = (df[categorical_var_name] == cat_val)
            # boolean_filter = True if the row contains the cat_val in its categorical column.
            
            #Input the encoded value when the filter is True:
            df.loc[boolean_filter, new_encoded_col_name] = encoded_val
            # In each iteration, the boolean_filter and the encoded_val are updated.
            # Then, the command applies the boolean_filter to the dataframe df, and access
            # the column declared after the comma. Then, it sets the values of the column as
            # equals to the value after the equality, encoded_val.
            # It is much more efficient than looping through the each row of the dataset, 
            # check the values on the categorical variable, and then input the integer value. 
            
            # Notice that we used df, not df_cleaned, avoiding the elimination of the
            # rows containing missing values.
        
        #Now that the dataframe was encoded, return the new dataframe
        
        # Print the encoding table
        print("Returning dataset with encoded column and the unique values summary table.")
        print("Unique values summary table:\n")
        print(unique_vals_summary)
        
        return unique_vals_summary, df
        
    else:
        
        #No encoded column will be added to the dataset
        print("Unique values summary table:\n")
        print(unique_vals_summary)
    
        return unique_vals_summary

# **Function for plotting the bar chart**
- Bars may be vertically or horizontally oriented.
- Bar charts are plotted after selecting an aggregation function, and the cumulative percent curve may be obtained and plotted with the bars (in secondary axis).
- To obtain a **Pareto chart**, keep `aggregate_function = 'sum'`, `plot_cumulative_percent = True`, and `orientation = 'vertical'`.

In [5]:
def bar_chart (df, categorical_var_name, response_var_name, aggregate_function = 'sum', add_suffix_to_aggregated_col = True, suffix = None, calculate_and_plot_cumulative_percent = True, orientation = 'vertical', limit_of_plotted_categories = None, horizontal_axis_title = None, vertical_axis_title = None, plot_title = None, x_axis_rotation = 70, y_axis_rotation = 0, grid = True, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):

    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    
    # df: dataframe being analyzed
    
    # categorical_var_name: string (inside quotes) containing the name 
    # of the column to be analyzed. e.g. 
    # categorical_var_name = "column1"
    
    # response_var_name: string (inside quotes) containing the name 
    # of the column that stores the response correspondent to the
    # categories. e.g. response_var_name = "response_feature" 
    
    # aggregate_function = 'sum': String defining the aggregation 
    # method that will be applied. Possible values:
    # 'median', 'mean', 'mode', 'sum', 'min', 'max', 'variance',
    # 'standard_deviation','10_percent_quantile', '20_percent_quantile',
    # '25_percent_quantile', '30_percent_quantile', '40_percent_quantile',
    # '50_percent_quantile', '60_percent_quantile', '70_percent_quantile',
    # '75_percent_quantile', '80_percent_quantile', '90_percent_quantile',
    # and '95_percent_quantile'.
    # To use another aggregate function, the method must be added to the
    # dictionary of methods agg_methods_dict, defined in the function.
    # If None or an invalid function is input, 'sum' will be used.
    
    # add_suffix_to_aggregated_col = True will add a suffix to the
    # aggregated column. e.g. 'responseVar_mean'. If add_suffix_to_aggregated_col 
    # = False, the aggregated column will have the original column name.
    
    # suffix = None. Keep it None if no suffix should be added, or if
    # the name of the aggregate function should be used as suffix, after
    # "_". Alternatively, set it as a string. As recommendation, put the
    # "_" sign in the beginning of this string to separate the suffix from
    # the original column name. e.g. if the response variable is 'Y' and
    # suffix = '_agg', the new aggregated column will be named as 'Y_agg'
    
    # calculate_and_plot_cumulative_percent = True to calculate and plot
    # the line of cumulative percent, or 
    # calculate_and_plot_cumulative_percent = False to omit it.
    # This feature is only shown when aggregate_function = 'sum', 'median',
    # 'mean', or 'mode'. So, it will be automatically set as False if 
    # another aggregate is selected.
    
    # orientation = 'vertical' is the standard, and plots vertical bars
    # (perpendicular to the X axis). In this case, the categories are shown
    # in the X axis, and the correspondent responses are in Y axis.
    # Alternatively, orientation = 'horizontal' results in horizontal bars.
    # In this case, categories are in Y axis, and responses in X axis.
    # If None or invalid values are provided, orientation is set as 'vertical'.
    
    # Note: to obtain a Pareto chart, keep aggregate_function = 'sum',
    # plot_cumulative_percent = True, and orientation = 'vertical'.
    
    # limit_of_plotted_categories: integer value that represents
    # the maximum of categories that will be plot. Keep it None to plot
    # all categories. Alternatively, set an integer value. e.g.: if
    # limit_of_plotted_categories = 4, but there are more categories,
    # the dataset will be sorted in descending order and: 1) The remaining
    # categories will be sum in a new category named 'others' if the
    # aggregate function is 'sum'; 2) Or the other categories will be simply
    # omitted from the plot, for other aggregate functions. Notice that
    # it limits only the variables in the plot: all of them will be
    # returned in the dataframe.
    # Use this parameter to obtain a cleaner plot. Notice that the remaining
    # columns will be aggregated as 'others' even if there is a single column
    # beyond the limit.
    
    
    # Create a local copy of the dataframe to manipulate:
    
    DATASET = df
    
    # Create the dictionary of possible aggregates, to define the
    # aggregation method, according to the set by the user:
    agg_methods_dict = {
        
        'median': DATASET.groupby(categorical_var_name)[response_var_name].median(),
        'mean': DATASET.groupby(categorical_var_name)[response_var_name].mean(),
        'mode': DATASET.groupby(categorical_var_name)[response_var_name].mode(),
        'sum': DATASET.groupby(categorical_var_name)[response_var_name].sum(),
        'min': DATASET.groupby(categorical_var_name)[response_var_name].min(),
        'max': DATASET.groupby(categorical_var_name)[response_var_name].max(),
        'variance': DATASET.groupby(categorical_var_name)[response_var_name].var(),
        'standard_deviation': DATASET.groupby(categorical_var_name)[response_var_name].std(),
        '10_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.10),
        '20_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.20),
        '25_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.25),
        '30_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.30),
        '40_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.40),
        '50_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.50),
        '60_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.60),
        '70_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.70),
        '75_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.75),
        '80_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.80),
        '90_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.90),
        '95_percent_quantile': DATASET.groupby(categorical_var_name)[response_var_name].quantile(0.95)
    }
    
    # check if the function was not set in the dictionary. If not,
    # use 'sum'
    if (aggregate_function not in (agg_methods_dict.keys())):
        
        aggregate_function = 'sum'
        print("Invalid or no aggregation function input, so using the default \'sum\'.")
    
    # Select the method in the dictionary and apply it. To access a value
    # 'val' correspondent to the key 'key' from a dictionary dict, we
    # declare: dict['key'], just as accessing a column from a dataframe.
    
    # The value will be the application of the method itself, i.e., the
    # dataset will be aggregated:
    DATASET = agg_methods_dict[aggregate_function]
    
    # If an aggregate function different from 'sum', 'mean', 'median' or 'mode' 
    # is used with plot_cumulative_percent = True, 
    # set plot_cumulative_percent = False:
    # (check if aggregate function is not in the list of allowed values):
    if ((aggregate_function not in ['sum', 'mean', 'median', 'mode']) & (calculate_and_plot_cumulative_percent == True)):
        
        calculate_and_plot_cumulative_percent = False
        print("The cumulative percent is only calculated when aggregate_function = \'sum\', \'mean\', \'median\', or \'mode\'. So, plot_cumulative_percent was set as False.")
    
    # Guarantee that the columns from the aggregated dataset have the correct
    
    # Let's create a list of the new column names
    # The first element is categorical_var_name, which is not modified:
    list_of_cols = [categorical_var_name]
    
    # Check if add_suffix_to_aggregated_col is False. If it is, simply
    # repeat the original response_var_name:
    if (add_suffix_to_aggregated_col == False):
        
        list_of_cols.append(response_var_name)
    
    else:
        # Let's add a suffix. Check if suffix is None. If it is,
        # set "_" + aggregate_function as suffix:
        
        if (suffix is None):
            suffix = "_" + aggregate_function
        
        # Now, append response_var_name + suffix to the list to
        # create the name of the new aggregated column:
        response_var_name = response_var_name + suffix
        list_of_cols.append(response_var_name)
    
    # Now, rename the columns of the aggregated dataset as the list
    # list_of_cols:
    DATASET.columns = list_of_cols
    
    # Let's sort the dataframe.
    
    # Order the dataframe in descending order by the response.
    # If there are equal responses, order them by category, in
    # ascending order; put the missing values in the first position
    # To pass multiple columns and multiple types of ordering, we use
    # lists. If there was a single column to order by, we would declare
    # it as a string. If only one order of ascending was used, we would
    # declare it as a simple boolean
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html
    
    DATASET = DATASET.sort_values(by = [response_var_name, categorical_var_name], ascending = [False, True], na_position = 'first')
    
    # Now, reset index positions:
    DATASET = DATASET.reset_index(drop = True)
    
    # plot_cumulative_percent = True, create a column to store the
    # cumulative percent:
    if (calculate_and_plot_cumulative_percent): 
        # Run the following code if the boolean value is True (implicity)
        
        # Calculate the total sum of the array correspondent to
        # the column (series) response_var_name
        total_sum = np.sum(np.array(DATASET[response_var_name]))
        
        # Create a column series for the cumulative sum:
        cumsum_col = response_var_name + "_cumsum"
        DATASET[cumsum_col] = DATASET[response_var_name].cumsum()
        
        # Now, create a column for the accumulated percent
        # by dividing cumsum_col by total_sum and multiplying it by
        # 100 (%):
        cum_pct_col = response_var_name + "_cum_pct"
        DATASET[cum_pct_col] = (DATASET[cumsum_col])/(total_sum)*100
        print(f"Successfully calculated cumulative sum and cumulative percent correspondent to the response variable {response_var_name}.")
    
    print("Successfully aggregated and ordered the dataset to plot. Check the 10 first rows of this returned dataset:\n")
    print(DATASET.head(10))
    
    # Check if the total of plotted categories is limited:
    if not (limit_of_plotted_categories is None):
        
        # Since the value is not None, we have to limit it
        # Check if the limit is lower than or equal to the length of the dataframe.
        # If it is, we simply copy the columns to the series (there is no need of
        # a memory-consuming loop or of applying the head method to a local copy
        # of the dataframe):
        df_length = len(DATASET)
            
        if (limit_of_plotted_categories <= df_length):
            # Simply copy the columns to the graphic series:
            categories = DATASET[categorical_var_name]
            responses = DATASET[response_var_name]
            # If there is a cum_pct column, copy it to a series too:
            if (calculate_and_plot_cumulative_percent):
                cum_pct = plotted_df[cum_pct_col]
        
        else:
            # The limit is lower than the total of categories,
            # so we actually have to limit the size of plotted df:
        
            # If aggregate_function is not 'sum', we simply apply
            # the head method to obtain the first rows (number of
            # rows input as parameter; if no parameter is input, the
            # number of 5 rows is used):
            if (aggregate_function != 'sum'):
                # Limit to the number limit_of_plotted_categories:
                # create another local copy of the dataframe not to
                # modify the returned dataframe object:
                plotted_df = DATASET.head(limit_of_plotted_categories)

                # Create the series of elements to plot:
                categories = plotted_df[categorical_var_name]
                responses = plotted_df[response_var_name]
                # If the cumulative percent was obtained, create the series for it:
                if (calculate_and_plot_cumulative_percent):
                    cum_pct = plotted_df[cum_pct_col]

            else:

                # Firstly, copy the elements that will be kept to x, y and (possibly) cum_pct
                # lists.
                # Start the lists:
                categories = []
                responses = []
                if (calculate_and_plot_cumulative_percent):
                    cum_pct = [] # start this list only if its needed to save memory

                for i in range (0, limit_of_plotted_categories):
                    # i goes from 0 (first index) to limit_of_plotted_categories - 1
                    # (index of the last category to be kept):
                    # copy the elements from the DATASET to the list
                    # category is the 1st column (column 0); response is the 2nd (col 1);
                    # and cumulative percent is the 4th (col 3):
                    categories.append(DATASET.iloc[i, 0])
                    responses.append(DATASET.iloc[i, 1])
                    
                    if (calculate_and_plot_cumulative_percent):
                        cum_pct.append(DATASET.iloc[i, 3]) # only if there is something to iloc
                    
                # Now, i = limit_of_plotted_categories - 1
                # Create a variable to store the sum of other responses
                other_responses = 0
                # loop from i = limit_of_plotted_categories to i = df_length-1, index
                # of the last element. Notice that this loop may have a single call, if there
                # is only one element above the limit:
                for i in range (limit_of_plotted_categories, (df_length - 1)):
                    
                    other_responses = other_responses + (DATASET.iloc[i, 1])
                
                # Now, add the last elements to the series:
                # The last category is named 'others':
                categories.append('others')
                # The correspondent aggregated response is the value 
                # stored in other_responses:
                responses.append(other_responses)
                # The cumulative percent is 100%, since this must be the sum of all
                # elements (the previous ones plus the ones aggregated as 'others'
                # must totalize 100%).
                # On the other hand, the cumulative percent is stored only if needed:
                cum_pct.append(100)
    
    else:
        # This is the situation where there is no limit of plotted categories. So, we
        # simply copy the columns to the plotted series (it is equivalent to the 
        # situation where there is a limit, but the limit is equal or inferior to the
        # size of the dataframe):
        categories = DATASET[categorical_var_name]
        responses = DATASET[response_var_name]
        # If there is a cum_pct column, copy it to a series too:
        if (calculate_and_plot_cumulative_percent):
            cum_pct = plotted_df[cum_pct_col]
    
    
    # Now the data is prepared and we only have to plot 
    # categories, responses, and cum_pct:
    
    # Set labels and titles for the case they are None
    if (plot_title is None):
        plot_title = f"Bar_chart_for_{response_var_name}_by_{categorical_var_name}"
    
    if (horizontal_axis_title is None):

        horizontal_axis_title = categorical_var_name

    if (vertical_axis_title is None):
        # Notice that response_var_name already has the suffix indicating the
        # aggregation function
        vertical_axis_title = response_var_name
    
    fig, ax1 = plt.subplots()
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 70 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    plt.title(plot_title)
    ax1.set_xlabel(horizontal_axis_title)
    ax1.set_ylabel(vertical_axis_title, color = 'blue')
    
    if (orientation == 'horizontal'):
        
        # Horizontal bars used - barh method (bar horizontal):
        # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.barh.html
        # Now, the categorical variables stored in series categories must be
        # positioned as the vertical axis Y, whereas the correspondent responses
        # must be in the horizontal axis X.
        ax1.barh(categories, responses, color = 'blue', label = categorical_var_name)
        #.barh(y, x, ...)
        
        if (calculate_and_plot_cumulative_percent):
            # Let's plot the line for the cumulative percent
            # Set the grid for the bar chart as False. If it is True, there will
            # be to grids, one for the bars and other for the percents, making 
            # the image difficult to interpretate:
            ax1.grid(False)
            
            # Create the twin plot for the cumulative percent:
            ax2 = ax1.twinx()
            # Here, the x axis must be the cum_pct value, and the Y
            # axis must be categories (it must be correspondent to the
            # bar chart)
            ax2.plot(cum_pct, categories, '-ro', color = 'red', label = "cumulative\npercent")
            #.plot(x, y, ...)
            ax2.tick_params('x', color = 'red')
            ax2.set_ylabel("Cumulative Percent (\%)", color = 'red')
            ax2.legend()
            ax2.grid(grid) # shown if user set grid = True
            # If user wants to see the grid, it is shown only for the cumulative line.
        
        else:
            # There is no cumulative line, so the parameter grid must control 
            # the bar chart's grid
            ax1.legend()
            ax1.grid(grid)
        
    else: 
        # If None or an invalid orientation was used, set it as vertical
        # Use Matplotlib standard bar method (vertical bar):
        # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html#matplotlib.pyplot.bar
        
        # In this standard case, the categorical variables (categories) are positioned
        # as X, and the responses as Y:
        ax1.bar(categories, responses, color = 'blue', label = categorical_var_name)
        #.bar(x, y, ...)
        
        if (calculate_and_plot_cumulative_percent):
            # Let's plot the line for the cumulative percent
            # Set the grid for the bar chart as False. If it is True, there will
            # be to grids, one for the bars and other for the percents, making 
            # the image difficult to interpretate:
            ax1.grid(False)
            
            # Create the twin plot for the cumulative percent:
            ax2 = ax1.twinx()
            ax2.plot(categories, cum_pct, '-ro', color = 'red', label = "cumulative\npercent")
            #.plot(x, y, ...)
            ax2.tick_params('y', color = 'red')
            ax2.set_ylabel("Cumulative Percent (\%)", color = 'red')
            ax2.legend()
            ax2.grid(grid) # shown if user set grid = True
            # If user wants to see the grid, it is shown only for the cumulative line.
        
        else:
            # There is no cumulative line, so the parameter grid must control 
            # the bar chart's grid
            ax1.legend()
            ax1.grid(grid)
    
    # Notice that the .plot method is used for generating the plot for both orientations.
    # It is different from .bar and .barh, which specify the orientation of a bar; or
    # .hline (creation of an horizontal constant line); or .vline (creation of a vertical
    # constant line).
    
    # Now the parameters specific to the configurations are finished, so we can go back
    # to the general code:
    
    if (export_png == True):
        # Image will be exported
        import os
        
        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = "/"
        
        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "bar_chart"
        
        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 110 dpi
            png_resolution_dpi = 110
        
        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)
        
        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")
    
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    plt.figure(figsize=(12, 8))
    #fig.tight_layout()
    
    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()
    
    return DATASET

# **Function for calculating cumulative statistics**
- Cumulative sum (cumsum); cumulative product (cumprod); cumulative maximum (cummax); cumulative minimum (cummin)

In [3]:
def calculate_cumulative_stats (df, column_to_analyze, cumulative_statistic = 'sum', new_cum_stats_col_name = None):
    
    import numpy as np
    import pandas as pd
    
    # df: the whole dataframe to be processed.
    
    # column_to_analyze: string (inside quotes), 
    # containing the name of the column that will be analyzed. 
    # e.g. column_to_analyze = "column1" will analyze the column named as 'column1'.
    
    # cumulative_statistic: the statistic that will be calculated. The cumulative
    # statistics allowed are: 'sum' (for cumulative sum, cumsum); 'product' 
    # (for cumulative product, cumprod); 'max' (for cumulative maximum, cummax);
    # and 'min' (for cumulative minimum, cummin).
    
    # new_cum_stats_col_name = None or string (inside quotes), 
    # containing the name of the new column created for storing the cumulative statistic
    # calculated. 
    # e.g. new_cum_stats_col_name = "cum_stats" will create a column named as 'cum_stats'.
    # If its None, the new column will be named as column_to_analyze + "_" + [selected
    # cumulative function] ('cumsum', 'cumprod', 'cummax', 'cummin')
     
    
    #WARNING: Use this function to a analyze a single column from a dataframe.
    
    if ((cumulative_statistic not in ['sum', 'product', 'max', 'min']) | (cumulative_statistic is None)):
        
        print("Please, select a valid method for calculating the cumulative statistics: sum, product, max, or min.")
        return "error"
    
    else:
        
        if (new_cum_stats_col_name is None):
            # set the standard name
            # column_to_analyze + "_" + [selected cumulative function] 
            # ('cumsum', 'cumprod', 'cummax', 'cummin')
            # cumulative_statistic variable stores ['sum', 'product', 'max', 'min']
            # we must concatenate "cum" to the left of this string:
            new_cum_stats_col_name = column_to_analyze + "_" + "cum" + cumulative_statistic
        
        # create a local copy of the dataframe to manipulate:
        DATASET = df
        # The series to be analyzed is stored as DATASET[column_to_analyze]
        
        # Now apply the correct method
        # the dictionary dict_of_methods correlates the input cumulative_statistic to the
        # correct Pandas method to be applied to the dataframe column
        dict_of_methods = {
            
            'sum': DATASET[column_to_analyze].cumsum(),
            'product': DATASET[column_to_analyze].cumprod(),
            'max': DATASET[column_to_analyze].cummax(),
            'min': DATASET[column_to_analyze].cummin()
        }
        
        # To access the value (method) correspondent to a given key (input as 
        # cumulative_statistic): dictionary['key'], just as if accessing a column from
        # a dataframe. In this case, the method is accessed as:
        # dict_of_methods[cumulative_statistic], since cumulative_statistic is itself the key
        # of the dictionary of methods.
        
        # store the resultant of the method in a new column of DATASET 
        # named as new_cum_stats_col_name
        DATASET[new_cum_stats_col_name] = dict_of_methods[cumulative_statistic]
        
        print(f"The cumulative {cumulative_statistic} statistic was successfully calculated and added as the column \'{new_cum_stats_col_name}\' of the returned dataframe.")
        print("Check the new dataframe's 10 first rows:\n")
        print(DATASET.head(10))
        
        return DATASET

# **Function for dealing with missing values**

In [18]:
def handle_missing_values (df, subset_columns_list = None, drop_missing_val = True, fill_missing_val = False, eliminate_only_completely_empty_rows = False, minimum_number_of_mis_vals = None, value_to_fill = None, fill_method = "fill_with_zeros"):
    
    import numpy as np
    import pandas as pd
    
    #subset_columns_list = list of columns to look for missing values. Only missing values
    # in these columns will be considered for deciding which columns to remove.
    # Declare it as a list of strings inside quotes containing the columns' names to look at,
    # even if this list contains a single element. e.g. subset_columns_list = ['column1']
    # will check only 'column1'; whereas subset_columns_list = ['col1', 'col2', 'col3'] will
    # chek the columns named as 'col1', 'col2', and 'col3'.
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
    
    #drop_missing_val = True to eliminate the rows containing missing values.
    
    #fill_missing_val = False. Set this to True to activate the mode for filling the missing
    # values.
    
    #eliminate_only_completely_empty_rows = False - This parameter shows effect only when
    # drop_missing_val = True. If you set eliminate_only_completely_empty_rows = True, then
    # only the rows where all the columns are missing will be eliminated.
    # If you define a subset, then only the rows where all the subset columns are missing
    # will be eliminated.
    
    #minimum_number_of_mis_vals = None - This parameter shows effect only when
    # drop_missing_val = True. If you set minimum_number_of_mis_vals equals to an integer value,
    # then only the rows where at least this integer number columns are missing will be 
    # eliminated. e.g. if minimum_number_of_mis_vals = 2, only rows containing missing values
    # for at least 2 columns will be eliminated.
    # If you define a subset, then only the rows where all the subset columns are missing
    # will be eliminated.
    
    #value_to_fill = None - This parameter shows effect only when
    # fill_missing_val = True. Set this parameter as a float value to fill all missing
    # values with this value. e.g. value_to_fill = 0 will fill all missing values with
    # the number 0. You can also pass a function call like 
    # value_to_fill = np.sum(dataset['col1']). In this case, the missing values will be
    # filled with the sum of the series dataset['col1']
    # Alternatively, you can also input a string to fill the missing values. e.g.
    # value_to_fill = 'text' will fill all the missing values with the string "text".
    
    #fill_method = "fill_with_zeros". - This parameter shows effect only 
    # when fill_missing_val = True.
    # Alternatively: fill_method = "fill_with_zeros" - fill all the missing values with 0
    # fill_method = "fill_with_value_to_fill" - fill the missing values with the value
    # defined as the parameter value_to_fill
    # fill_method = "fill_with_avg" - fill the missing values with the average value for 
    # each column.
    # fill_method = "fill_by_interpolating" - fill by interpolating the previous and the 
    # following value. A linear interpolation will be used.
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
    # fill_method = "ffill" - Forward fill: fills missing with previous value.
    
    #WARNING: if the fillna method is selected (fill_missing_val == True), but no filling
    # methodology is selected, the missing values of the dataset will be filled with 0.
    # The same applies when a non-valid fill methodology is selected.
    # Pandas fillna method does not allow us to fill only a selected subset.
    
    #WARNING: if fill_method == "fill_with_value_to_fill" but value_to_fill is None, the 
    # missing values will be filled with the value 0.
    
    boolean_filter1 = (drop_missing_val == True) & (fill_missing_val == False)
    #Notice that the presence of the parameter fill_missing_val will make this filter False,
    # even if drop_missing_val is erroneously set as True.
    boolean_filter2 = (boolean_filter1) & (subset_columns_list is None)
    #These filters are True only if both conditions inside parentheses are True.
    # The operator & is equivalent to 'And' (intersection).
    # The operator | is equivalent to 'Or' (union).
    
    boolean_filter3 = (fill_missing_val == True) & (fill_method is None)
    #boolean_filter3 represents the situation where the fillna method was selected, but
    # no filling method was set.
    
    boolean_filter4 = (value_to_fill is None) & (fill_method == "fill_with_value_to_fill")
    #boolean_filter4 represents the situation where the fillna method will be used and the
    # user selected to fill the missing values with 'value_to_fill', but did not set a value
    # for 'value_to_fill'.
    
    if (boolean_filter1 == True):
        
        if (boolean_filter2 == True):
            
            if (eliminate_only_completely_empty_rows == True):
                #Eliminate only completely empty rows
                cleaned_df = cleaned_df.dropna(axis = 0, how = "all")
                # if axis = 1, dropna will eliminate each column containing missing values.
            
            elif (minimum_number_of_mis_vals is not None):
                #eliminate only rows containing at least 'minimum_number_of_mis_vals' missing vals
                cleaned_df = cleaned_df.dropna(axis = 0, tresh = minimum_number_of_mis_vals)
            
            else:
                #Eliminate all rows containing missing values.
                #The only parameter is drop_missing_val
                cleaned_df = cleaned_df.dropna(axis=0)
        
        else:
            #In this case, there is a subset for applying the Pandas dropna method.
            #Only the coluns in the subset 'subset_columns_list' will be analyzed.
                  
            if (eliminate_only_completely_empty_rows == True):
                #Eliminate only completely empty rows
                cleaned_df = cleaned_df.dropna(subset = subset_columns_list, how = "all")
            
            elif (minimum_number_of_mis_vals is not None):
                #eliminate only rows containing at least 'minimum_number_of_mis_vals' missing vals
                cleaned_df = cleaned_df.dropna(subset = subset_columns_list, tresh = minimum_number_of_mis_vals)
            
            else:
                #Eliminate all rows containing missing values.
                #The only parameter is drop_missing_val
                cleaned_df = cleaned_df.dropna(subset = subset_columns_list)
    
    else:
        
        #In this case, the user set a value for the parameter fill_missing_val, turning
        # the boolean filter 1 to False. That's for avoiding the removal of the missing
        # values, since the user wants to fill the missing data.
        # Pandas fillna method documentation:
        # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html
        
        if (boolean_filter3 == True):
            #fillna method was selected, but no filling method was set.
            # Then, filling with zero.
            cleaned_df = cleaned_df.fillna(0)
        
        elif (boolean_filter4 == True):
            #fill_method == "fill_with_value_to_fill" but value_to_fill is None.
            # Then, filling with zero.
            cleaned_df = cleaned_df.fillna(0)
        
        else:
            #A filling methodology was selected.
            if (fill_method == "fill_with_zeros"):
                cleaned_df = cleaned_df.fillna(0)
            
            elif (fill_method == "fill_with_value_to_fill"):
                cleaned_df = cleaned_df.fillna(value_to_fill)
            
            elif (fill_method == "fill_with_avg"):
                
                #1. Get dataframe's columns list:
                df_cols = df.columns
                #2. Start an empty list that will store all of the average values:
                avg_list = []
                #3. Loop through each column of df_cols.
                # Calculate the average value for the series df[column].
                # Append this value to avg_list.
                
                for column in df_cols:
                    #check all elements of df_cols, named 'column'
                    # Calculate the average of the series df[column]
                    # and append it to the avg_list
                    avg_list.append(np.average(df[column]))
               
                #Now, we have a list containing the average value correspondent
                # to each column of the dataframe, named avg_list.
                # 5. Create a dictionary informing the columns and the values
                # to be input in each column:
                fill_dict = {df_cols: avg_list}
                # This dictionary correlates each column to its average value.
                
                #6. Finally, use this dictionary to fill the missing values of each column
                # with the average value of that column
                cleaned_df = cleaned_df.fillna(value = fill_dict)
                #The method will search the column name in fill_dict (key of the dictionary),
                # and will use the correspondent value (average) to fill the missing values.
            
            elif (fill_method == "fill_by_interpolating"):
                cleaned_df = cleaned_df.interpolate(method='linear')
                #For using a polynomial interpolation:
                # cleaned_df = cleaned_df.interpolate(method='polynomial', order = 2)
                # Modify the order to use other degrees of polynomials (order = 2 for 2nd order).
            
            elif (fill_method == "ffill"):
                cleaned_df = cleaned_df.fillna(method = fill_method)
            
            else:
                print("No valid filling methodology was selected. Then, filling missing values with 0.")
                cleaned_df = cleaned_df.fillna(0)
        
        
    #Reset index before returning the cleaned dataframe:
    cleaned_df = df.reset_index(drop = True)
    
    return cleaned_df

# **Function for obtaining the correlation plot**
- The Pandas method dataset.corr() calculates the Pearson's correlation coefficients R.
- Pearson's correlation coefficients R go from -1 to 1.
- These coefficients are R, not R².

#### To obtain the coefficients R², we raise the results to the 2nd power, i.e., we calculate (dataset.corr())**2
- R² goes from 0 to 1, where 1 represents the perfect correlation.

In [1]:
def correlation_plot (df, show_masked_plot = True, responses_to_return_corr = None, set_returned_limit = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    #show_masked_plot = True - keep as True if you want to see a cleaned version of the plot
    # where a mask is applied.
    
    #responses_to_return_corr - keep as None to return the full correlation tensor.
    # If you want to display the correlations for a particular group of features, input them
    # as a list, even if this list contains a single element. Examples:
    # responses_to_return_corr = ['response1'] for a single response
    # responses_to_return_corr = ['response1', 'response2', 'response3'] for multiple
    # responses. Notice that 'response1',... should be substituted by the name ('string')
    # of a column of the dataset that represents a response variable.
    # WARNING: The returned coefficients will be ordered according to the order of the list
    # of responses. i.e., they will be firstly ordered based on 'response1'
    
    # set_returned_limit = None - This variable will only present effects in case you have
    # provided a response feature to be returned. In this case, keep set_returned_limit = None
    # to return all of the correlation coefficients; or, alternatively, 
    # provide an integer number to limit the total of coefficients returned. 
    # e.g. if set_returned_limit = 10, only the ten highest coefficients will be returned. 

    correlation_matrix = df.corr(method='pearson')
    
    if (show_masked_plot == False):
        #Show standard plot
        
        plt.figure()
        sns.heatmap((correlation_matrix)**2, annot=True, fmt=".2f")
        
        if (export_png == True):
            # Image will be exported
            import os

            #check if the user defined a directory path. If not, set as the default root path:
            if (directory_to_save is None):
                #set as the default
                directory_to_save = "/"

            #check if the user defined a file name. If not, set as the default name for this
            # function.
            if (file_name is None):
                #set as the default
                file_name = "correlation_plot"

            #check if the user defined an image resolution. If not, set as the default 110 dpi
            # resolution.
            if (png_resolution_dpi is None):
                #set as 110 dpi
                png_resolution_dpi = 110

            #Get the new_file_path
            new_file_path = os.path.join(directory_to_save, file_name)

            #Export the file to this new path:
            # The extension will be automatically added by the savefig method:
            plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
            #quality could be set from 1 to 100, where 100 is the best quality
            #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
            #transparent = True or False
            # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
            print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")

        #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
        plt.figure(figsize=(24,8));

    #Oncee the pandas method .corr() calculates R, we raised it to the second power 
    # to obtain R². R² goes from zero to 1, where 1 represents the perfect correlation.
    
    else:
        
        #Show masked (cleaner) plot instead of the standard one
        
        plt.figure()
        # Mask for the upper triangle
        mask = np.zeros_like((correlation_matrix)**2)

        mask[np.triu_indices_from(mask)] = True

        # Generate a custom diverging colormap
        cmap = sns.diverging_palette(220, 10, as_cmap=True)

        # Heatmap with mask and correct aspect ratio
        sns.heatmap(((correlation_matrix)**2), mask=mask, cmap=cmap, vmax=.3, center=0,
                    square=True, linewidths=.5, cbar_kws={"shrink": .5})
        
        if (export_png == True):
            # Image will be exported
            import os

            #check if the user defined a directory path. If not, set as the default root path:
            if (directory_to_save is None):
                #set as the default
                directory_to_save = "/"

            #check if the user defined a file name. If not, set as the default name for this
            # function.
            if (file_name is None):
                #set as the default
                file_name = "correlation_plot"

            #check if the user defined an image resolution. If not, set as the default 110 dpi
            # resolution.
            if (png_resolution_dpi is None):
                #set as 110 dpi
                png_resolution_dpi = 110

            #Get the new_file_path
            new_file_path = os.path.join(directory_to_save, file_name)

            #Export the file to this new path:
            # The extension will be automatically added by the savefig method:
            plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
            #quality could be set from 1 to 100, where 100 is the best quality
            #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
            #transparent = True or False
            # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
            print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")

        #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
        plt.figure(figsize=(24,8));

        #Again, the method dataset.corr() calculates R within the variables of dataset.
        #To calculate R², we simply raise it to the second power: (dataset.corr()**2)
    
    #Sort the values of correlation_matrix in Descending order:
    
    if (responses_to_return_corr is not None):
        
        #Select only the desired responses, by passing the list responses_to_return_corr
        # as parameter for column filtering:
        correlation_matrix = correlation_matrix[responses_to_return_corr]
        
        #Now sort the values according to the responses, by passing the list
        # responses_to_return_corr as the parameter
        correlation_matrix = correlation_matrix.sort_values(by = responses_to_return_corr, ascending = False)
        
        # If a limit of coefficients was determined, apply it:
        if (set_returned_limit is not None):
                
                correlation_matrix = correlation_matrix.head(set_returned_limit)
                #Pandas .head(X) method returns the first X rows of the dataframe.
                # Here, it returns the defined limit of coefficients, set_returned_limit.
                # The default .head() is X = 5.
        
        print(correlation_matrix)
    
    print("ATTENTION: The correlation plots show the linear correlations R², which go from 0 (none correlation) to 1 (perfect correlation). Obviously, the main diagonal always shows R² = 1, since the data is perfectly correlated to itself.")
    print("The returned correlation matrix, on the other hand, presents the linear coefficients of correlation R, not R². R values go from -1 (perfect negative correlation) to 1 (perfect positive correlation).")
    print("None of these coefficients take non-linear relations and the presence of a multiple linear correlation in account. For these cases, it is necessary to calculate R² adjusted, which takes in account the presence of multiple preditors and non-linearities.")
    
    return correlation_matrix

# **Function for obtaining scatter plots and simple linear regressions**
- Here, only a single prediction variable will be analyzed by once.
- The plots will show Y x X, where X is the predict or independent variable.
- The linear regressions will be of the type Y = aX + b, i.e., a single pair (X, Y) analyzed.

        x1, y1, lab1: blue
        x2, y2, lab2: red
        x3, y3, lab3: green
        x4, y4, lab4: black
        x5, y5, lab5: magenta
        x6, y6, lab6: yellow

In [30]:
def scatter_plot_lin_reg (x1 = None, y1 = None, x2 = None, y2 = None, x3 = None, y3 = None, x4 = None, y4 = None, x5 = None, y5 = None, x6 = None, y6 = None, x_axis_rotation = 0, y_axis_rotation = 0, show_linear_reg = True, grid = True, add_splines_lines = False, lab1 = None, lab2 = None, lab3 = None, lab4 = None, lab5 = None, lab6 = None, horizontal_axis_title = None, vertical_axis_title = None, plot_title = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110): 
    
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    from scipy import stats
    
    if (add_splines_lines == True):
        line_value = '-'
    else:
        line_value = ''
    
    if (show_linear_reg == True):
        estatisticas = []
        estatisticas.append("Linear Fitting:")
        estatisticas.append("R² = ")
    
    fig = plt.figure()
    ax = fig.add_subplot()
    
    if not (x1 is None):
        
        if not (lab1 is None):
            label_1 = lab1
        else:
            label_1 = "Y1 x X1"
        
        #falsa negativa: passa se os valores nao forem nulos
        ax.plot(x1, y1, linestyle = line_value, marker = 'o', color='blue', label=label_1)
        
        if (show_linear_reg == True):
            reta1 = []
            #Calculo da regressao linear:
            reg1 = stats.linregress(x1, y1)
            #organizar os X, para que os splines formem a reta correta
            x_reg1 = x1.sort_values()
            x_reg1 = x_reg1.reset_index(drop = True)
            #curva obtida:
            y_reg1 = (reg1).intercept + (reg1).slope*(x_reg1)
            #gerar string da reta
            string1 = "y = %.2f*x + %.2f" %((reg1).slope, (reg1).intercept)
            reta1.append(string1)
            #calcular R2
            r_sq1 = (reg1).rvalue**2
            reta1.append(r_sq1)
            print("\nLinear Fitting 1: " + string1)
            #concatena as strings
            print("\nR² (fitting 1) = %.4f" %(r_sq1))
            string_label1 = 'Linear regression: ' + label_1
            ax.plot(x_reg1, y_reg1,  linestyle='-', marker='', color='blue', label = string_label1)
    
    if not (x2 is None):
        #roda apenas se ambos estiverem presentes
                
        if not (lab2 is None):
            label_2 = lab2
        else:
            label_2 = "Y2"
        
        ax.plot(x2, y2, linestyle = line_value, marker = 'o', color='red', label=label_2)    
        
        if (show_linear_reg == True):
            reta2 = []
            #Calculo da regressao linear:
            reg2 = stats.linregress(x2, y2)
            #organizar os X, para que os splines formem a reta correta
            x_reg2 = x2.sort_values()
            x_reg2 = x_reg2.reset_index(drop = True)
            #curva obtida:
            y_reg2 = (reg2).intercept + (reg2).slope*(x_reg2)
            #gerar string da reta
            string2 = "y = %.2f*x + %.2f" %((reg2).slope, (reg2).intercept)
            reta2.append(string2)
            #calcular R2
            r_sq2 = (reg2).rvalue**2
            reta2.append(r_sq2)
            print("\nLinear Fitting 2: " + string2)
            #concatena as strings
            print("\nR² (fitting 2) = %.4f" %(r_sq2))
            string_label2 = 'Linear regression: ' + label_2
            ax.plot(x_reg2, y_reg2,  linestyle='-', marker='', color='red', label = string_label2)
        
    if not (x3 is None):
                
        if not (lab3 is None):
            label_3 = lab3
        else:
            label_3 = "Y3"
        
        ax.plot(x3, y3, linestyle = line_value, marker = 'o', color='green', label=label_3)
        
        if (show_linear_reg == True):
            reta3 = []
            #Calculo da regressao linear:
            reg3 = stats.linregress(x3, y3)
            #organizar os X, para que os splines formem a reta correta
            x_reg3 = x3.sort_values()
            x_reg3 = x_reg3.reset_index(drop = True)
            #curva obtida:
            y_reg3 = (reg3).intercept + (reg3).slope*(x_reg3)
            #gerar string da reta
            string3 = "y = %.2f*x + %.2f" %((reg3).slope, (reg3).intercept)
            reta3.append(string3)
            #calcular R2
            r_sq3 = (reg3).rvalue**2
            reta3.append(r_sq3)
            print("\nLinear Fitting 3: " + string3)
            #concatena as strings
            print("\nR² (fitting 3) = %.4f" %(r_sq3))
            string_label3 = 'Linear regression: ' + label_3
            ax.plot(x_reg3, y_reg3,  linestyle='-', marker='', color='green', label = string_label3)
        
    if not (x4 is None):
                
        if not (lab4 is None):
            label_4 = lab4
        else:
            label_4 = "Y4"
        
        ax.plot(x4, y4, linestyle = line_value, marker = 'o', color='black', label=label_4)
        
        if (show_linear_reg == True):
            reta4 = []
            #Calculo da regressao linear:
            reg4 = stats.linregress(x4, y4)
            #organizar os X, para que os splines formem a reta correta
            x_reg4 = x4.sort_values()
            x_reg4 = x_reg4.reset_index(drop = True)
            #curva obtida:
            y_reg4 = (reg4).intercept + (reg4).slope*(x_reg4)
            #gerar string da reta
            string4 = "y = %.2f*x + %.2f" %((reg4).slope, (reg4).intercept)
            reta4.append(string4)
            #calcular R2
            r_sq4 = (reg4).rvalue**2
            reta4.append(r_sq4)
            print("\nLinear Fitting 4: " + string4)
            #concatena as strings
            print("\nR² (fitting 4) = %.4f" %(r_sq4))
            string_label4 = 'Linear regression: ' + label_4
            ax.plot(x_reg4, y_reg4,  linestyle='-', marker='', color='black', label = string_label4)
    
    if not (x5 is None):
               
        if not (lab5 is None):
            label_5 = lab5
        else:
            label_5 = "Y5"
        
        ax.plot(x5, y5, linestyle = line_value, marker = 'o', color='magenta', label=label_5)
        
        if (show_linear_reg == True):
            reta5 = []
            #Calculo da regressao linear:
            reg5 = stats.linregress(x5, y5)
            #organizar os X, para que os splines formem a reta correta
            x_reg5 = x5.sort_values()
            x_reg5 = x_reg5.reset_index(drop = True)
            #curva obtida:
            y_reg5 = (reg5).intercept + (reg5).slope*(x_reg5)
            #gerar string da reta
            string5 = "y = %.2f*x + %.2f" %((reg5).slope, (reg5).intercept)
            reta5.append(string5)
            #calcular R2
            r_sq5 = (reg5).rvalue**2
            reta5.append(r_sq5)
            print("\nLinear Fitting 5: " + string5)
            #concatena as strings
            print("\nR² (fitting 5) = %.4f" %(r_sq5))
            string_label5 = 'Linear regression: ' + label_5
            ax.plot(x_reg5, y_reg5,  linestyle='-', marker='', color='magenta', label = string_label5)
   
    if not (x6 is None):
               
        if not (lab6 is None):
            label_6 = lab6
        else:
            label_6 = "Y6"
        
        ax.plot(x6, y6, linestyle = line_value, marker = 'o', color='yellow', label=label_6)
            
        if (show_linear_reg == True):
            reta6 = []
            #Calculo da regressao linear:
            reg6 = stats.linregress(x6, y6)
            #organizar os X, para que os splines formem a reta correta
            x_reg6 = x6.sort_values()
            x_reg6 = x_reg6.reset_index(drop = True)
            #curva obtida:
            y_reg6 = (reg6).intercept + (reg6).slope*(x_reg6)
            #gerar string da reta
            string6 = "y = %.2f*x + %.2f" %((reg6).slope, (reg6).intercept)
            reta6.append(string6)
            #calcular R2
            r_sq6 = (reg6).rvalue**2
            reta6.append(r_sq6)
            print("\nLinear Fitting 6: " + string6)
            #concatena as strings
            print("\nR² (fitting 6) = %.4f" %(r_sq6))
            string_label6 = 'Linear regression: ' + label_6
            ax.plot(x_reg6, y_reg6,  linestyle='-', marker='', color='yellow', label = string_label6)
   
    if not (plot_title is None):
        #titulo do grafico
        ax.set_title(plot_title) 
    
    if not (horizontal_axis_title is None):
        #Titulo do eixo X
        ax.set_xlabel(horizontal_axis_title)
    
    if not (vertical_axis_title is None):
        #Titulo do eixo Y
        ax.set_ylabel(vertical_axis_title)
    
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 0 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    ax.grid(grid)
    ax.legend()
    
    if (export_png == True):
        # Image will be exported
        import os
        
        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = "/"
        
        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "scatter_plot_lin_reg"
        
        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 110 dpi
            png_resolution_dpi = 110
        
        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)
        
        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")
    
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    plt.figure(figsize=(12, 8))
    #fig.tight_layout()
    
    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()
    
    if (show_linear_reg == True):
        
        if not (x2 is None):
            
            if not (x3 is None):
                
                if not (x4 is None):
                    
                    if not (x5 is None):
                        
                        if not (x6 is None):
                            #todos estao presentes
                            d = {'Statistics': estatisticas,
                                 label_1: reta1,
                                 label_2: reta2,
                                 label_3: reta3,
                                 label_4: reta4,
                                 label_5: reta5,
                                 label_6: reta6}
                        
                        else:
                            #apenas 5 estão presentes:
                            d = {'Statistics': estatisticas,
                                 label_1: reta1,
                                 label_2: reta2,
                                 label_3: reta3,
                                 label_4: reta4,
                                 label_5: reta5}
                    
                    else:
                        #apenas 4 estão presentes:
                        d = {'Statistics': estatisticas,
                             label_1: reta1,
                             label_2: reta2,
                             label_3: reta3,
                             label_4: reta4}
                
                else:
                    #apenas 3 estão presentes:
                    d = {'Statistics': estatisticas,
                         label_1: reta1,
                         label_2: reta2,
                         label_3: reta3}
            
            else:
                #apenas 2 estão presentes:
                d = {'Statistics': estatisticas,
                     label_1: reta1,
                     label_2: reta2}
        
        else:
            #apenas 1 esta presente:
            d = {'Statistics': estatisticas,
                 label_1: reta1}
        
        lin_reg_summary = pd.DataFrame(data = d)
        
        return lin_reg_summary     

# **Function for time series visualization**
        x1, y1, lab1: blue
        x2, y2, lab2: red
        x3, y3, lab3: green
        x4, y4, lab4: black
        x5, y5, lab5: magenta
        x6, y6, lab6: yellow

In [25]:
def time_series_vis (x1 = None, y1 = None, x2 = None, y2 = None, x3 = None, y3 = None, x4 = None, y4 = None, x5 = None, y5 = None, x6 = None, y6 = None, x_axis_rotation = 70, y_axis_rotation = 0, grid = True, add_splines_lines = True, add_scatter_dots = False, lab1 = None, lab2 = None, lab3 = None, lab4 = None, lab5 = None, lab6 = None, horizontal_axis_title = None, vertical_axis_title = None, plot_title = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    
    if (add_splines_lines == True):
        line_value = '-'
    else:
        line_value = ''
    
    if (add_scatter_dots == True):
        marker_value = 'o'
    else:
        marker_value = ''
    
    fig = plt.figure()
    ax = fig.add_subplot()
    
    if not (lab1 is None):
        
        label_1 = lab1
    
    else:
        label_1 = "Y1"

    if not (x1 is None):
        ax.plot(x1, y1, linestyle = line_value, marker = marker_value, color='blue', label=label_1)
    
    if not (x2 is None):
        #runs only when both are present
        if not (lab2 is None):
            label_2 = lab2
        else:
            label_2 = "Y2"
        
        ax.plot(x2, y2, linestyle = line_value, marker = marker_value, color='red', label=label_2)
    
    if not (x3 is None):
                
        if not (lab3 is None):
            label_3 = lab3
        else:
            label_3 = "Y3"
        
        ax.plot(x3, y3, linestyle = line_value, marker = marker_value, color='green', label=label_3)
    
    if not (x4 is None):
                
        if not (lab4 is None):
            label_4 = lab4
        else:
            label_4 = "Y4"
        
        ax.plot(x4, y4, linestyle = line_value, marker = marker_value, color='black', label=label_4)
    
    if not (x5 is None):
               
        if not (lab5 is None):
            label_5 = lab5
        else:
            label_5 = "Y5"
        
        ax.plot(x5, y5, linestyle = line_value, marker = marker_value, color='magenta', label=label_5)
   
    if not (x6 is None):
               
        if not (lab6 is None):
            label_6 = lab6
        else:
            label_6 = "Y6"
        
        ax.plot(x6, y6, linestyle = line_value, marker = marker_value, color='yellow', label=label_6)
   
    if not (plot_title is None):
        #graphic's title
        ax.set_title(plot_title) 
    
    if not (horizontal_axis_title is None):
        #X-axis title
        ax.set_xlabel(horizontal_axis_title)
    
    if not (vertical_axis_title is None):
        #Y-axis title
        ax.set_ylabel(vertical_axis_title)
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 70 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    ax.grid(grid)
    ax.legend()
    
    if (export_png == True):
        # Image will be exported
        import os
        
        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = "/"
        
        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "time_series_vis"
        
        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 110 dpi
            png_resolution_dpi = 110
        
        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)
        
        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")
    
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    plt.figure(figsize=(12, 8))
    #fig.tight_layout()
    
    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()

# **Functions for histogram visualization**
- Function `histogram`: ideal bin interval is calculated through Montgomery's method. Histogram is obtained from this calculated bin size.
    - Douglas C. Montgomery (2009). Introduction to Statistical Process Control, Sixth Edition, John Wiley & Sons.
- Function `histogram_alternative`: histogram is obtained by manually defining the total of bins (i.e., into how much intervals the sample space should be divided).

In [27]:
def histogram (y, bar_width, x_axis_rotation = 70, y_axis_rotation = 0, grid = True, normal_curve_overlay = True, data_units_label = None, y_title = None, histogram_title = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
    
    import numpy as np
    import pandas as pd
    import matplotlib
    import matplotlib.pyplot as plt
    
    # ideal bin interval calculated through Montgomery's method. 
    # Histogram is obtained from this calculated bin size.
    # Douglas C. Montgomery (2009). Introduction to Statistical Process Control, 
    # Sixth Edition, John Wiley & Sons.
    
    
    #Calculo do bin size - largura do histograma:
    #1: Encontrar o menor (lowest) e o maior (highest) valor dentro da tabela de dados)
    #2: Calcular rangehist = highest - lowest
    #3: Calcular quantidade de dados (samplesize) de entrada fornecidos
    #4: Calcular a quantidade de celulas da tabela de frequencias (ncells)
    #ncells = numero inteiro mais proximo da (raiz quadrada de samplesize)
    #5: Calcular binsize = rangehist/(ncells)
    #ATENCAO: Nao se esquecer de converter range, ncells, samplesize e binsize para valores absolutos (modulos)
    #isso porque a largura do histograma tem que ser um numero positivo

    y = y.reset_index(drop=True)
    #faz com que os indices desta serie sejam consecutivos e a partir de zero

    #Estatisticas gerais: media (mu) e desvio-padrao (sigma)
    mu = y.mean() 
    sigma = y.std() 

    #Calculo do bin-size
    highest = y.max()
    lowest = y.min()
    rangehist = highest - lowest
    rangehist = abs(rangehist)
    #garante que sera um numero positivo
    samplesize = y.count() #contagem do total de entradas
    ncells = (samplesize)**0.5 #potenciacao: ** - raiz quadrada de samplesize
    #resultado da raiz quadrada e sempre positivo
    ncells = round(ncells) #numero "redondo" mais proximo
    ncells = int(ncells) #parte inteira do numero arredondado
    #ncells = numero de linhas da tabela de frequencias
    binsize = rangehist/ncells
    binsize = round(binsize)
    binsize = int(binsize) #precisa ser inteiro
    
    #Construcao da tabela de frequencias

    j = 0 #indice da tabela de frequencias
    #Este indice e diferente do ordenamento dos valores em ordem crescente
    xhist = []
    #Lista vazia que contera os x do histograma
    yhist = []
    #Listas vazia que conteras o y do histograma
    hist_labels = []
    #Esta lista gravara os limites da barra na forma de strings

    pontomediodabarra = lowest + binsize/2 
    limitedabarra = lowest + binsize
    #ponto medio da barra 
    #limite da primeira barra do histograma
    seriedohist1 = y
    seriedohist1 = seriedohist1.sort_values(ascending=True)
    #serie com os valores em ordem crescente
    seriedohist1 = seriedohist1.reset_index(drop=True)
    #garante que a nova serie tenha indices consecutivos, iniciando em zero
    i = 0 #linha inicial da serie do histograma em ordem crescente
    valcomparado = seriedohist1[i]
    #primeiro valor da serie, o mais baixo

    while (j <= (ncells-1)):
        
        #para quando termina o numero de linhas da tabela
        xhist.append(pontomediodabarra)
        #tempo da tabela de frequencias
        cont = 0
        #variavel de contagem do histograma
        #contagem deve ser reiniciada
       
        if (i < samplesize):
            #2 condicionais para impedir que um termo de indice inexistente
            #seja acessado
            while (valcomparado <= limitedabarra) and (valcomparado < highest):
                #o segundo criterio garante a parada em casos em que os dados sao
                #muito proximos
                    cont = cont + 1 #adiciona contagem a tabela de frequencias
                    i = i + 1
                    
                    if (i < samplesize): 
                        valcomparado = seriedohist1[i]
        
        yhist.append(cont) #valor de ocorrencias contadas
        
        limite_infdabarra = pontomediodabarra - binsize/2
        rotulo = "%.2f - %.2f" %(limite_infdabarra, limitedabarra)
        #intervalo da tabela de frequencias
        #%.2f: 2 casas decimais de aproximação
        hist_labels.append(rotulo)
        
        pontomediodabarra = pontomediodabarra + binsize
        #tanto os pontos medios quanto os limites se deslocam do mesmo intervalo
        
        limitedabarra = limitedabarra + binsize
        #proxima barra
        
        j = j + 1
    
    #Temos que verificar se o valor maximo foi incluido
    #isso porque o processo de aproximacao por numero inteiro pode ter
    #arredondado para baixo e excluido o limite superior
    #Porem, note que na ultima iteracao o limite superior da barra foi 
    #somado de binsize, mas como j ja e maior que ncells-1, o loop parou
    
    #assim, o limitedabarra nesse momento e o limite da barra que seria
    #construida em seguida, nao da ultima barra da tabela de frequencias
    #isso pode fazer com que esta barra ja seja maior que o highest
    
    #note porem que nao aumentamos o valor do limite inferior da barra
    #por isso, basta vermos se ele mais o binsize sao menores que o valor mais alto
        
    while ((limite_infdabarra+binsize) < highest):
        
        #vamos criar novas linhas ate que o ponto mais alto do histograma
        #tenha sido contado
        ncells = ncells + 1 #adiciona uma linha a tabela de frequencias
        xhist.append(pontomediodabarra)
        
        cont = 0 #variavel de contagem do histograma
        
        while (valcomparado <= limitedabarra):
                cont = cont + 1 #adiciona contagem a tabela de frequencias
                i = i + 1
                if (i < samplesize):
                    valcomparado = seriedohist1[i]
                    #apenas se i ainda nao e maior que o total de dados
                
                else: 
                    
                    break
        
        #parar o loop se i atingiu um tamanho maior que a quantidade 
        #de dados.Temos que ter este cuidado porque estamos acrescentando
        #mais linhas a tabela de frequencias para corrigir a aproximacao
        #de ncells por um numero inteiro
        
        yhist.append(cont) #valor de ocorrencias contadas
        
        limite_infdabarra = pontomediodabarra - binsize/2
        rotulo = "%.2f - %.2f" %(limite_infdabarra, limitedabarra)
        #intervalo da tabela de frequencias - 2 casas decimais
        hist_labels.append(rotulo)
        
        pontomediodabarra = pontomediodabarra + binsize
        #tanto os pontos medios quanto os limites se deslocam do mesmo intervalo
        
        limitedabarra = limitedabarra + binsize
        #proxima barra
        
    estatisticas_col1 = []
    #contera as descricoes das colunas da tabela de estatisticas gerais
    estatisticas_col2 = []
    #contera os valores da tabela de estatisticas gerais
    
    estatisticas_col1.append("Count of data evaluated")
    estatisticas_col2.append(samplesize)
    estatisticas_col1.append("Average (mu)")
    estatisticas_col2.append(mu)
    estatisticas_col1.append("Standard deviation (sigma)")
    estatisticas_col2.append(sigma)
    estatisticas_col1.append("Highest value")
    estatisticas_col2.append(highest)
    estatisticas_col1.append("Lowest value")
    estatisticas_col2.append(lowest)
    estatisticas_col1.append("Data range (maximum value - lowest value)")
    estatisticas_col2.append(rangehist)
    estatisticas_col1.append("Bin size (bar width)")
    estatisticas_col2.append(binsize)
    estatisticas_col1.append("Total rows in frequency table")
    estatisticas_col2.append(ncells)
    #como o comando append grava linha a linha em sequencia, garantimos
    #a correspondencia das colunas
    #Assim como em qualquer string, incluindo de rotulos de graficos
    #os \n sao lidos como quebra de linha
    
    d1 = {"General Statistics": estatisticas_col1, "Calculated Value": estatisticas_col2}
    #dicionario das duas series, para criar o dataframe com as descricoes
    estatisticas_gerais = pd.DataFrame(data = d1)
    
    #Casos os títulos estejam presentes (valor nao e None):
    #vamos utiliza-los
    #Caso contrario, vamos criar nomenclaturas genericas para o histograma
    
    eixo_y = "Counting/Frequency"
    
    if not (data_units_label is None):
        xlabel = data_units_label
    
    else:
        xlabel = "Frequency\n table data"
    
    if not (y_title is None):
        eixo_x = y_title
        #lembre-se que no histograma, os dados originais vao pro eixo X
        #O eixo Y vira o eixo da contagem/frequencia daqueles dados
    
    else:
        eixo_x = "X: Mean value of the interval"
    
    if not (histogram_title is None):
        string1 = "- $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        main_label = histogram_title + string1
        #concatena a string do titulo a string com a media e desvio-padrao
        #%.2f: o numero entre %. e f indica a quantidade de casas decimais da 
        #variavel float f. No caso, arredondamos para 2 casas
        #NAO SE ESQUECA DO PONTO: ele que indicara que sera arredondado o 
        #numero de casas
    
    else:
        main_label = "Data Histogram - $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        #os simbolos $\ $ substituem o simbolo pela letra grega
    
    d2 = {"Considered interval": hist_labels, eixo_x: xhist, eixo_y: yhist}
    #dicionario que compoe a tabela de frequencias
    tab_frequencias = pd.DataFrame(data = d2)
    #cria a tabela de frequencias como um dataframe de saida
   
    #parametros da normal ja calculados:
    #mu e sigma
    #numero de bins: ncells
    #limites de especificacao: lsl,usl - target
    
    #valor maximo do histograma
    max_hist = max(yhist)
    #seleciona o valor maximo da serie, para ajustar a curva normal
    #isso porque a normal é criada com valores entre 0 e 1
    #multiplicando ela por max_hist, fazemos ela se adequar a altura do histograma
    
    if (normal_curve_overlay == True):
        
        #construir a normal ajustada/esperada
        #vamos criar pontos ao redor da media mu - 4sigma ate mu + 4sigma, 
        #de modo a garantir a quase totalidade da curva normal. 
        #O incremento será de 0.10 sigma a cada iteracao
        x_inf = mu -(4)*sigma
        x_sup = mu + 4*sigma
        x_inc = (0.10)*sigma
        
        x_normal_adj = []
        y_normal_adj = []
        
        x_adj = x_inf
        y_adj = ((1 / (np.sqrt(2 * np.pi) * sigma)) *np.exp(-0.5 * (1 / sigma * (x_adj - mu))**2))
        x_normal_adj.append(x_adj)
        y_normal_adj.append(y_adj)
        
        while(x_adj < x_sup): 
            
            x_adj = x_adj + x_inc
            y_adj = ((1 / (np.sqrt(2 * np.pi) * sigma)) *np.exp(-0.5 * (1 / sigma * (x_adj - mu))**2))
            x_normal_adj.append(x_adj)
            y_normal_adj.append(y_adj)
        
        #vamos ajustar a altura da curva ao histograma. Para isso, precisamos
        #calcular quantas vezes o ponto mais alto do histograma é maior que o ponto
        #mais alto da normal (chamaremos essa relação de fator). A seguir,
        #multiplicamos cada elemento da normal por este mesmo fator
        max_normal = max(y_normal_adj) 
        #maximo da normal ajustada, numero entre 0 e 1
        
        fator = (max_hist)/(max_normal)
        size_normal = len(y_normal_adj) #quantidade de dados criados
        
        i = 0
        while (i < size_normal):
            y_normal_adj[i] = (y_normal_adj[i])*(fator)
            i = i + 1
    
    #Fazer o grafico
    fig, ax = plt.subplots()
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 70 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    #STANDARD MATPLOTLIB METHOD:
    #bins = number of bins (intervals) of the histogram. Adjust it manually
    #increasing bins will increase the histogram's resolution, but height of bars
    
    #ax.hist(y, bins=20, width = bar_width, label=xlabel, color='blue')
    #IF GRAPHIC IS NOT SHOWN: THAT IS BECAUSE THE DISTANCES BETWEEN VALUES ARE LOW, AND YOU WILL
    #HAVE TO USE THE STANDARD HISTOGRAM METHOD FROM MATPLOTLIB.
    #TO DO THAT, UNMARK LINE ABOVE: ax.hist(y, bins=20, width = bar_width, label=xlabel, color='blue')
    #AND MARK LINE BELOW AS COMMENT: ax.bar(xhist, yhist, width = bar_width, label=xlabel, color='blue')
    
    #IF YOU WANT TO CREATE GRAPHIC AS A BAR CHART BASED ON THE CALCULATED DISTRIBUTION TABLE, 
    #MARK THE LINE ABOVE AS COMMENT AND UNMARK LINE BELOW:
    ax.bar(xhist, yhist, width = bar_width, label=xlabel, color='blue')
    #ajuste manualmente a largura, width, para deixar as barras mais ou menos proximas
    
    if (normal_curve_overlay == True):
    
        #adicionar a normal
        ax.plot(x_normal_adj, y_normal_adj, color = 'black', label = 'Adjusted/expected\n normal curve')
    
    ax.set_xlabel(eixo_x)
    ax.set_ylabel(eixo_y)
    ax.set_title(main_label)
    ax.set_xticks(xhist)
    
    ax.legend()
    ax.grid(grid)
    
    if (export_png == True):
        # Image will be exported
        import os
        
        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = "/"
        
        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "histogram"
        
        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 110 dpi
            png_resolution_dpi = 110
        
        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)
        
        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")
    
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    plt.figure(figsize=(12, 8))
    #fig.tight_layout()
    
    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()
    
    print("General statistics:\n")
    print(estatisticas_gerais)
    print("\n") # line break
    print("Frequency table:\n")
    print(tab_frequencias)

    return estatisticas_gerais, tab_frequencias

In [29]:
def histogram_alternative (y, total_of_bins, bar_width, x_axis_rotation = 70, y_axis_rotation = 0, grid = True, data_units_label = None, y_title = None, histogram_title = None, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
    
    import numpy as np
    import pandas as pd
    import matplotlib
    import matplotlib.pyplot as plt
    
    #Calculo do bin size - largura do histograma:
    #1: Encontrar o menor (lowest) e o maior (highest) valor dentro da tabela de dados)
    #2: Calcular rangehist = highest - lowest
    #3: Calcular quantidade de dados (samplesize) de entrada fornecidos
    #4: Calcular a quantidade de celulas da tabela de frequencias (ncells)
    #ncells = numero inteiro mais proximo da (raiz quadrada de samplesize)
    #5: Calcular binsize = rangehist/(ncells)
    #ATENCAO: Nao se esquecer de converter range, ncells, samplesize e binsize para valores absolutos (modulos)
    #isso porque a largura do histograma tem que ser um numero positivo
    
    #this variable here is to simply guarantee the compatibility of the function,
    # with no extensive code modifications. It has no real effect.
    normal_curve_overlay = True
    

    y = y.reset_index(drop=True)
    #faz com que os indices desta serie sejam consecutivos e a partir de zero

    #Estatisticas gerais: media (mu) e desvio-padrao (sigma)
    mu = y.mean() 
    sigma = y.std() 

    #Calculo do bin-size
    highest = y.max()
    lowest = y.min()
    rangehist = highest - lowest
    rangehist = abs(rangehist)
    #garante que sera um numero positivo
    samplesize = y.count() #contagem do total de entradas
    ncells = (samplesize)**0.5 #potenciacao: ** - raiz quadrada de samplesize
    #resultado da raiz quadrada e sempre positivo
    ncells = round(ncells) #numero "redondo" mais proximo
    ncells = int(ncells) #parte inteira do numero arredondado
    #ncells = numero de linhas da tabela de frequencias
    binsize = rangehist/ncells
    binsize = round(binsize)
    binsize = int(binsize) #precisa ser inteiro
    
    #Construcao da tabela de frequencias

    j = 0 #indice da tabela de frequencias
    #Este indice e diferente do ordenamento dos valores em ordem crescente
    xhist = []
    #Lista vazia que contera os x do histograma
    yhist = []
    #Listas vazia que conteras o y do histograma
    hist_labels = []
    #Esta lista gravara os limites da barra na forma de strings

    pontomediodabarra = lowest + binsize/2 
    limitedabarra = lowest + binsize
    #ponto medio da barra 
    #limite da primeira barra do histograma
    seriedohist1 = y
    seriedohist1 = seriedohist1.sort_values(ascending=True)
    #serie com os valores em ordem crescente
    seriedohist1 = seriedohist1.reset_index(drop=True)
    #garante que a nova serie tenha indices consecutivos, iniciando em zero
    i = 0 #linha inicial da serie do histograma em ordem crescente
    valcomparado = seriedohist1[i]
    #primeiro valor da serie, o mais baixo
        
    estatisticas_col1 = []
    #contera as descricoes das colunas da tabela de estatisticas gerais
    estatisticas_col2 = []
    #contera os valores da tabela de estatisticas gerais
    
    estatisticas_col1.append("Count of data evaluated")
    estatisticas_col2.append(samplesize)
    estatisticas_col1.append("Average (mu)")
    estatisticas_col2.append(mu)
    estatisticas_col1.append("Standard deviation (sigma)")
    estatisticas_col2.append(sigma)
    estatisticas_col1.append("Highest value")
    estatisticas_col2.append(highest)
    estatisticas_col1.append("Lowest value")
    estatisticas_col2.append(lowest)
    estatisticas_col1.append("Data range (maximum value - lowest value)")
    estatisticas_col2.append(rangehist)
    estatisticas_col1.append("Bin size (bar width)")
    estatisticas_col2.append(binsize)
    estatisticas_col1.append("Total rows in frequency table")
    estatisticas_col2.append(ncells)
    #como o comando append grava linha a linha em sequencia, garantimos
    #a correspondencia das colunas
    #Assim como em qualquer string, incluindo de rotulos de graficos
    #os \n sao lidos como quebra de linha
    
    d1 = {"General Statistics": estatisticas_col1, "Calculated Value": estatisticas_col2}
    #dicionario das duas series, para criar o dataframe com as descricoes
    estatisticas_gerais = pd.DataFrame(data = d1)
    
    #Casos os títulos estejam presentes (valor nao e None):
    #vamos utiliza-los
    #Caso contrario, vamos criar nomenclaturas genericas para o histograma
    
    eixo_y = "Counting/Frequency"
    
    if not (data_units_label is None):
        xlabel = data_units_label
    
    else:
        xlabel = "Frequency\n table data"
    
    if not (y_title is None):
        eixo_x = y_title
        #lembre-se que no histograma, os dados originais vao pro eixo X
        #O eixo Y vira o eixo da contagem/frequencia daqueles dados
    
    else:
        eixo_x = "X: Mean value of the interval"
    
    if not (histogram_title is None):
        string1 = "- $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        main_label = histogram_title + string1
        #concatena a string do titulo a string com a media e desvio-padrao
        #%.2f: o numero entre %. e f indica a quantidade de casas decimais da 
        #variavel float f. No caso, arredondamos para 2 casas
        #NAO SE ESQUECA DO PONTO: ele que indicara que sera arredondado o 
        #numero de casas
    
    else:
        main_label = "Data Histogram - $\mu = %.2f$, $\sigma = %.2f$" %(mu, sigma)
        #os simbolos $\ $ substituem o simbolo pela letra grega
    
    d2 = {"Considered interval": hist_labels, eixo_x: xhist, eixo_y: yhist}
    #dicionario que compoe a tabela de frequencias
    tab_frequencias = pd.DataFrame(data = d2)
    #cria a tabela de frequencias como um dataframe de saida
    
    #parametros da normal ja calculados:
    #mu e sigma
    #numero de bins: ncells
    #limites de especificacao: lsl,usl - target
    
    #valor maximo do histograma
    #max_hist = max(yhist)
    #seleciona o valor maximo da serie, para ajustar a curva normal
    #isso porque a normal é criada com valores entre 0 e 1
    #multiplicando ela por max_hist, fazemos ela se adequar a altura do histograma
    
    
    #Fazer o grafico
    fig, ax = plt.subplots()
    
    #ROTATE X AXIS IN XX DEGREES
    plt.xticks(rotation = x_axis_rotation)
    # XX = 70 DEGREES x_axis (Default)
    #ROTATE Y AXIS IN XX DEGREES:
    plt.yticks(rotation = y_axis_rotation)
    # XX = 0 DEGREES y_axis (Default)
    
    #STANDARD MATPLOTLIB METHOD:
    #bins = number of bins (intervals) of the histogram. Adjust it manually
    #increasing bins will increase the histogram's resolution, but height of bars
    
    ax.hist(y, bins = total_of_bins, width = bar_width, label=xlabel, color='blue')
    #IF GRAPHIC IS NOT SHOWN: THAT IS BECAUSE THE DISTANCES BETWEEN VALUES ARE LOW, AND YOU WILL
    #HAVE TO USE THE STANDARD HISTOGRAM METHOD FROM MATPLOTLIB.
    #TO DO THAT, UNMARK LINE ABOVE: ax.hist(y, bins=20, width = bar_width, label=xlabel, color='blue')
    #AND MARK LINE BELOW AS COMMENT: ax.bar(xhist, yhist, width = bar_width, label=xlabel, color='blue')
    
    #IF YOU WANT TO CREATE GRAPHIC AS A BAR CHART BASED ON THE CALCULATED DISTRIBUTION TABLE, 
    #MARK THE LINE ABOVE AS COMMENT AND UNMARK LINE BELOW:
    #ax.bar(xhist, yhist, width = bar_width, label=xlabel, color='blue')
    #ajuste manualmente a largura, width, para deixar as barras mais ou menos proximas
    
    ax.set_xlabel(eixo_x)
    ax.set_ylabel(eixo_y)
    ax.set_title(main_label)
    #ax.set_xticks(xhist)
    
    ax.legend()
    ax.grid(grid)
    
    if (export_png == True):
        # Image will be exported
        import os
        
        #check if the user defined a directory path. If not, set as the default root path:
        if (directory_to_save is None):
            #set as the default
            directory_to_save = "/"
        
        #check if the user defined a file name. If not, set as the default name for this
        # function.
        if (file_name is None):
            #set as the default
            file_name = "histogram_alternative"
        
        #check if the user defined an image resolution. If not, set as the default 110 dpi
        # resolution.
        if (png_resolution_dpi is None):
            #set as 110 dpi
            png_resolution_dpi = 110
        
        #Get the new_file_path
        new_file_path = os.path.join(directory_to_save, file_name)
        
        #Export the file to this new path:
        # The extension will be automatically added by the savefig method:
        plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
        #quality could be set from 1 to 100, where 100 is the best quality
        #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
        #transparent = True or False
        # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
        print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")
    
    #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
    plt.figure(figsize=(12, 8))
    #fig.tight_layout()
    
    ## Show an image read from an image file:
    ## import matplotlib.image as pltimg
    ## img=pltimg.imread('mydecisiontree.png')
    ## imgplot = plt.imshow(img)
    ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
    ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
    ##  '03_05_END.ipynb'
    plt.show()
    
    print("General statistics:\n")
    print(estatisticas_gerais)
    # This function is supposed to be used in cases where the differences between data
    # is very small. In such cases, there will be no trust values calculated for the 
    # frequency table. Therefore, we omit it here, but it can be accessed from the
    # returned dataframe.

    return estatisticas_gerais, tab_frequencias

# **Function for testing data normality and visualizing the probability plot**
- Check the probability that data is actually described by a normal distribution.

In [19]:
def test_data_normality (y, alpha = 0.10, show_probability_plot = True, x_axis_rotation = 0, y_axis_rotation = 0, grid = True, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from statsmodels.stats import diagnostic
    from scipy import stats
    # Check https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html#scipy.stats.probplot
    # Check https://docs.scipy.org/doc/scipy/tutorial/stats.html
    # Check https://docs.scipy.org/doc/scipy-1.8.0/html-scipyorg/reference/generated/scipy.stats.normaltest.html
    
    # WARNING: The statistical tests require at least 20 samples
    
    # Confidence level = 1 - ALPHA. For ALPHA = 0.10, we get a 0.90 = 90% confidence
    # Set ALPHA = 0.05 to get 0.95 = 95% confidence in the analysis.
    # Notice that, when less trust is needed, we can increase ALPHA to get less restrictive
    # results.
    
    # y = series of data that will be tested.
    # y = dataset['Y']
    
    #Alternatively: set SHOW_PROBABILITY_PLOT = True to obtain the probability plot for the
    # variable Y (normal distribution tested). 
    # Set SHOW_PROBABILITY_PLOT = False to omit the probability plot.
    
    lista1 = []
    #esta lista sera a primeira coluna, com as descrições das demais
    lista1.append("p-value: probability that data is described by the normal distribution.")
    lista1.append("Probability of being described by the normal distribution (\%).")
    lista1.append("alpha")
    lista1.append("Criterium: is not described by normal if p < alpha = %.3f." %(alpha))
    #%.3f apresenta f com 3 casas decimais
    #%f se refere a uma variavel float
    #informa ao usuario o valor definido para a rejeição
    lista1.append("Are data described by the normal?")
    #Note que o comando append adiciona os elementos em sequencia, linha a linha
    #nao se especifica indice, pois ja esta subentendido que esta na proxima
    #linha
    
    #Scipy.stats’ normality test
    # It is based on D’Agostino and Pearson’s test that combines 
    # skew and kurtosis to produce an omnibus test of normality.
    _, scipystats_test_pval = stats.normaltest(y)
    # The underscore indicates an output to be ignored, which is s^2 + k^2, 
    # where s is the z-score returned by skewtest and k is the z-score returned by kurtosistest.
    # https://docs.scipy.org/doc/scipy-1.8.0/html-scipyorg/reference/generated/scipy.stats.normaltest.html
    
    #create list with only the p-val
    p_scipy = []
    p_scipy.append(scipystats_test_pval) #p-value
    p_scipy.append(100*scipystats_test_pval) #p in percent
    p_scipy.append(alpha)
    
    if (scipystats_test_pval < alpha):
        p_scipy.append("p = %.3f < %.3f" %(scipystats_test_pval, alpha))
        p_scipy.append("Data not described by normal.")
    else:
        p_scipy.append("p = %.3f >= %.3f" %(scipystats_test_pval, alpha))
        p_scipy.append("Data described by normal.")    
    
    #Lilliefors’ test
    lilliefors_test = diagnostic.kstest_normal(y, dist='norm', pvalmethod='table')
    #Return: linha 1: ksstat: float
    #Kolmogorov-Smirnov test statistic with estimated mean and variance.
    #Linha 2: p-value:float
    #If the pvalue is lower than some threshold, e.g. 0.10, then we can reject the Null hypothesis that the sample comes from a normal distribution.
    
    #criar lista apenas com o p-valor
    p_lillie = []
    p_lillie.append(lilliefors_test[1]) #p-valor
    p_lillie.append(100*lilliefors_test[1]) #p em porcentagem
    p_lillie.append(alpha)
    
    if (lilliefors_test[1] < alpha):
        p_lillie.append("p = %.3f < %.3f" %(lilliefors_test[1], alpha))
        p_lillie.append("Data not described by normal.")
    else:
        p_lillie.append("p = %.3f >= %.3f" %(lilliefors_test[1], alpha))
        p_lillie.append("Data described by normal.")
        
    
    #Anderson-Darling
    ad_test = diagnostic.normal_ad(y, axis=0)
    #Return: Linha 1: ad2: float
    #Anderson Darling test statistic.
    #Linha 2: p-val: float
    #The p-value for hypothesis that the data comes from a normal distribution with unknown mean and variance.
    
    #criar lista apenas com o p-valor
    p_ad = []
    p_ad.append(ad_test[1]) #p-valor
    p_ad.append(100*ad_test[1]) #p em porcentagem
    p_ad.append(alpha)
    
    if (ad_test[1] < alpha):
        p_ad.append("p = %.3f < %.3f" %(ad_test[1], alpha))
        p_ad.append("Data not described by normal.")
    else:
        p_ad.append("p = %.3f >= %.3f" %(ad_test[1], alpha))
        p_ad.append("Data described by normal.")
    
    #NOTA: o comando %f apresenta a variavel float com todas as casas
    #decimais possiveis. Se desejamos um numero certo de casas decimais
    #acrescentamos esse numero a frente. Exemplos: %.1f: 1 casa decimal
    # %.2f: 2 casas; %.3f: 3 casas decimais, %.4f: 4 casas
    
    data_normality_dict = {'Parameters and Interpretation': lista1, 'D’Agostino and Pearson normality test': , 'Lilliefors Test': p_lillie, 'Anderson-Darling Test': p_ad}
    
    #dicionario dos valores obtidos
    data_normality_res = pd.DataFrame(data = data_normality_dict)
    #dataframe de saída
    
    print("Check data normality results:\n")
    print(data_normality_res)
    print("\n") #line break
    
    # Calculate data skewness and kurtosis
    
    # Skewness
    data_skew = stats.skew(y)
    # skewness = 0 : normally distributed.
    # skewness > 0 : more weight in the left tail of the distribution.
    # skewness < 0 : more weight in the right tail of the distribution.
    # https://www.geeksforgeeks.org/scipy-stats-skew-python/
    
    # Kurtosis
    data_kurtosis = stats.kurtosis(y, fisher = True)
    # scipy.stats.kurtosis(array, axis=0, fisher=True, bias=True) function 
    # calculates the kurtosis (Fisher or Pearson) of a data set. It is the the fourth 
    # central moment divided by the square of the variance. 
    # It is a measure of the “tailedness” i.e. descriptor of shape of probability 
    # distribution of a real-valued random variable. 
    # In simple terms, one can say it is a measure of how heavy tail is compared 
    # to a normal distribution.
    # fisher parameter: fisher : Bool; Fisher’s definition is used (normal 0.0) if True; 
    # else Pearson’s definition is used (normal 3.0) if set to False.
    # https://www.geeksforgeeks.org/scipy-stats-kurtosis-function-python/
    print("A normal distribution should present no skewness (distribution distortion); and no kurtosis (long-tail).")
    print(f"For the data analyzed: skewness = {data_skew}; kurtosis = {data_kurtosis}")
    
    if (data_skew < 0):
        
        print(f"Skewness {data_skew} < 0: more weight in the left tail of the distribution.")
    
    elif (data_skew > 0):
        
        print(f"Skewness {data_skew} > 0: more weight in the right tail of the distribution.")
        
    else:
        
        print(f"Skewness {data_skew} = 0: no distortion of the distribution.")
    
    
    print(f"Data kurtosis = {data_kurtosis}")
    
    if (data_kurtosis == 0):
        
        print("Data kurtosis = 0. No long-tail effects detected.")
    
    #Calculate the mode of the distribution:
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html
    data_mode = stats.mode(y, axis = None)[0]
    # returns an array of arrays. The first array is called mode=array and contains the mode.
    # Axis: Default is 0. If None, compute over the whole array.
    # we set axis = None to compute the general mode.
    
    #Create general statistics dictionary:
    general_statistics_dict = {
        
        "Count_of_analyzed_values": len(y)
        "Data_mean": np.mean(y),
        "Data_mean_ignoring_missing_values": np.nanmean(y),
        "Data_variance": np.var(y),
        "Data_variance_ignoring_missing_values": np.nanvar(y),
        "Data_standard_deviation": np.std(y),
        "Data_standard_deviation_ignoring_missing_values": np.nanstd(y),
        "Data_skewness": data_skew,
        "Data_kurtosis": data_kurtosis,
        "Data_mode": data_mode
        
    }
    
    print("Skewness and kurtosis successfully returned in the dictionary general_statistics_dict.\n")
    print(general_statistics_dict)
    print("/n")
    
    if (show_probability_plot == True):
        #Obtain the probability plot  
        fig, ax = plt.subplots()

        ax.set_title("Probability Plot for Normal Distribution")

        #ROTATE X AXIS IN XX DEGREES
        plt.xticks(rotation = x_axis_rotation)
        # XX = 70 DEGREES x_axis (Default)
        #ROTATE Y AXIS IN XX DEGREES:
        plt.yticks(rotation = y_axis_rotation)
        # XX = 0 DEGREES y_axis (Default)   

        res = stats.probplot(y, dist = 'norm', fit = True, plot = ax)
        #This function resturns a tuple, so we must store it into res
        
        #Other distributions to check, see scipy Stats documentation. 
        # you could test dist=stats.loggamma, where stats was imported from scipy
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html#scipy.stats.probplot

        ax.grid(grid)
        ax.legend()

        if (export_png == True):
            # Image will be exported
            import os

            #check if the user defined a directory path. If not, set as the default root path:
            if (directory_to_save is None):
                #set as the default
                directory_to_save = "/"

            #check if the user defined a file name. If not, set as the default name for this
            # function.
            if (file_name is None):
                #set as the default
                file_name = "probability_plot"

            #check if the user defined an image resolution. If not, set as the default 110 dpi
            # resolution.
            if (png_resolution_dpi is None):
                #set as 110 dpi
                png_resolution_dpi = 110

            #Get the new_file_path
            new_file_path = os.path.join(directory_to_save, file_name)

            #Export the file to this new path:
            # The extension will be automatically added by the savefig method:
            plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
            #quality could be set from 1 to 100, where 100 is the best quality
            #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
            #transparent = True or False
            # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
            print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")

        #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
        plt.figure(figsize=(12, 8))
        #fig.tight_layout()

        ## Show an image read from an image file:
        ## import matplotlib.image as pltimg
        ## img=pltimg.imread('mydecisiontree.png')
        ## imgplot = plt.imshow(img)
        ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
        ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
        ##  '03_05_END.ipynb'
        plt.show()
    
    return data_normality_res, general_statistics_dict

# **Function for testing and visualizing probability plots for different statistical distributions**

In [19]:
def test_stat_distribution (y, statistical_distribution = 'lognormal', show_probability_plot = True, x_axis_rotation = 0, y_axis_rotation = 0, grid = True, export_png = False, directory_to_save = None, file_name = None, png_resolution_dpi = 110):
    
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from statsmodels.stats import diagnostic
    from scipy import stats
    # Check https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html#scipy.stats.probplot
    # Check https://docs.scipy.org/doc/scipy/tutorial/stats.html
    # Check https://docs.scipy.org/doc/scipy-1.8.0/html-scipyorg/reference/generated/scipy.stats.normaltest.html
    
    # WARNING: The statistical tests require at least 20 samples
    
    # Attention: if you want to test a normal distribution, use the function 
    # test_data_normality.Function test_data_normality tests normality through 3 methods 
    # and compare them: D’Agostino and Pearson’s; Lilliefors; and Anderson-Darling tests.
    # The calculus of the p-value from the Anderson-Darling statistic is available only 
    # for some distributions. The function specific for the normality calculates these 
    # probabilities of following the normal.
    # Here, the function is destined to test a variety of distributions, and so only the 
    # Anderson-Darling test is performed.
        
    #statistical_distribution: string (inside quotes) containing the tested statistical 
    # distribution.
    # Notice: if data Y follow a 'lognormal', log(Y) follow a normal
    # Poisson is a special case from 'gamma' distribution.
    ## There are 91 accepted statistical distributions:
    # 'alpha', 'anglit', 'arcsine', 'beta', 'beta_prime', 'bradford', 'burr', 'burr12', 
    # 'cauchy', 'skewed_cauchy', 'chi', 'chi-squared', 'cosine', 'double_gamma', 
    # 'double_weibull', 'erlang', 'exponential', 'exponentiated_weibull', 'exponential_power',
    # 'fatigue_life_birnbaum-saunders', 'fisk_log_logistic', 'folded_cauchy', 'folded_normal',
    # 'F', 'gamma', 'generalized_logistic', 'generalized_pareto', 'generalized_exponential', 
    # 'generalized_extreme_value', 'generalized_gamma', 'generalized_half-logistic', 
    # 'generalized_hyperbolic', 'generalized_inverse_gaussian', 'generalized_normal', 
    # 'gilbrat', 'gompertz_truncated_gumbel', 'gumbel', 'gumbel_left-skewed', 'half-cauchy', 
    # 'half-normal', 'half-logistic', 'hyperbolic_secant', 'gauss_hypergeometric', 
    # 'inverted_gamma', 'inverse_normal', 'inverted_weibull', 'johnson_SB', 'johnson_SU', 
    # 'KSone', 'KStwo', 'KStwobign', 'laplace', 'asymmetric_laplace', 'left-skewed_levy', 
    # 'levy', 'logistic', 'log_laplace', 'log_gamma', 'lognormal', 'log-uniform', 'maxwell', 
    # 'mielke_Beta-Kappa', 'nakagami', 'noncentral_chi-squared', 'noncentral_F', 
    # 'noncentral_t', 'normal', 'normal_inverse_gaussian', 'pareto', 'lomax', 
    # 'power_lognormal', 'power_normal', 'power-function', 'R', 'rayleigh', 'rice', 
    # 'reciprocal_inverse_gaussian', 'semicircular', 'studentized_range', 'student-t', 
    # 'trapezoidal', 'triangular', 'truncated_exponential', 'truncated_normal', 'tukey-lambda',
    # 'uniform', 'von_mises', 'wald', 'weibull_maximum_extreme_value', 
    # 'weibull_minimum_extreme_value', 'wrapped_cauchy'
    
    # y = series of data that will be tested.
    # y = dataset['Y']
    
    #Alternatively: set SHOW_PROBABILITY_PLOT = True to obtain the probability plot for the
    # variable Y (normal distribution tested). 
    # Set SHOW_PROBABILITY_PLOT = False to omit the probability plot.
    
    # Calculate data skewness and kurtosis
    
    # Skewness
    data_skew = stats.skew(y)
    # skewness = 0 : normally distributed.
    # skewness > 0 : more weight in the left tail of the distribution.
    # skewness < 0 : more weight in the right tail of the distribution.
    # https://www.geeksforgeeks.org/scipy-stats-skew-python/
    
    # Kurtosis
    data_kurtosis = stats.kurtosis(y, fisher = True)
    # scipy.stats.kurtosis(array, axis=0, fisher=True, bias=True) function 
    # calculates the kurtosis (Fisher or Pearson) of a data set. It is the the fourth 
    # central moment divided by the square of the variance. 
    # It is a measure of the “tailedness” i.e. descriptor of shape of probability 
    # distribution of a real-valued random variable. 
    # In simple terms, one can say it is a measure of how heavy tail is compared 
    # to a normal distribution.
    # fisher parameter: fisher : Bool; Fisher’s definition is used (normal 0.0) if True; 
    # else Pearson’s definition is used (normal 3.0) if set to False.
    # https://www.geeksforgeeks.org/scipy-stats-kurtosis-function-python/
    print("A normal distribution should present no skewness (distribution distortion); and no kurtosis (long-tail).")
    print(f"For the data analyzed: skewness = {data_skew}; kurtosis = {data_kurtosis}")
    
    if (data_skew < 0):
        
        print(f"Skewness {data_skew} < 0: more weight in the left tail of the distribution.")
    
    elif (data_skew > 0):
        
        print(f"Skewness {data_skew} > 0: more weight in the right tail of the distribution.")
        
    else:
        
        print(f"Skewness {data_skew} = 0: no distortion of the distribution.")
    
    
    print(f"Data kurtosis = {data_kurtosis}")
    
    if (data_kurtosis == 0):
        
        print("Data kurtosis = 0. No long-tail effects detected.")
    
    #Calculate the mode of the distribution:
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html
    data_mode = stats.mode(y, axis = None)[0]
    # returns an array of arrays. The first array is called mode=array and contains the mode.
    # Axis: Default is 0. If None, compute over the whole array.
    # we set axis = None to compute the general mode.
    

    # Lets define the statistic distribution dictionary:
    # This are the callable Scipy objects which can be tested through Anderson-Darling test:
    # They are listed and explained in: 
    # https://docs.scipy.org/doc/scipy/tutorial/stats/continuous.html
    
    # This dictionary correlates the input name of the distribution to the correct scipy.stats
    # callable object
    # There are 91 possible statistical distributions:
    
    print("If a compilation error is shown below, please update your Scipy version. Declare and run the following code into a separate cell:")
    print("! pip install scipy --upgrade\n")
    
    callable_statistical_distributions_dict = {
        
        'alpha': stats.alpha, 'anglit': stats.anglit, 'arcsine': stats.arcsine,
        'beta': stats.beta, 'beta_prime': stats.betaprime, 'bradford': stats.bradford,
        'burr': stats.burr, 'burr12': stats.burr12, 'cauchy': stats.cauchy,
        'skewed_cauchy': stats.skewcauchy, 'chi': stats.chi, 'chi-squared': stats.chi2,
        'cosine': stats.cosine, 'double_gamma': stats.dgamma, 'double_weibull': stats.dweibull,
        'erlang': stats.erlang, 'exponential': stats.expon, 'exponentiated_weibull': stats.exponweib,
        'exponential_power': stats.exponpow, 'fatigue_life_birnbaum-saunders': stats.fatiguelife,
        'fisk_log_logistic': stats.fisk, 'folded_cauchy': stats.foldcauchy,
        'folded_normal': stats.foldnorm, 'F': stats.f, 'gamma': stats.gamma,
        'generalized_logistic': stats.genlogistic, 'generalized_pareto': stats.genpareto,
        'generalized_exponential': stats.genexpon, 'generalized_extreme_value': stats.genextreme,
        'generalized_gamma': stats.gengamma, 'generalized_half-logistic': stats.genhalflogistic,
        'generalized_hyperbolic': stats.genhyperbolic, 'generalized_inverse_gaussian': stats.geninvgauss,
        'generalized_normal': stats.gennorm, 'gilbrat': stats.gilbrat,
        'gompertz_truncated_gumbel': stats.gompertz, 'gumbel': stats.gumbel_r,
        'gumbel_left-skewed': stats.gumbel_l, 'half-cauchy': stats.halfcauchy, 'half-normal': stats.halfnorm,
        'half-logistic': stats.halflogistic, 'hyperbolic_secant': stats.hypsecant,
        'gauss_hypergeometric': stats.gausshyper, 'inverted_gamma': stats.invgamma,
        'inverse_normal': stats.invgauss, 'inverted_weibull': stats.invweibull,
        'johnson_SB': stats.johnsonsb, 'johnson_SU': stats.johnsonsu, 'KSone': stats.ksone,
        'KStwo': stats.kstwo, 'KStwobign': stats.kstwobign, 'laplace': stats.laplace,
        'asymmetric_laplace': stats.laplace_asymmetric, 'left-skewed_levy': stats.levy_l,
        'levy': stats.levy, 'logistic': stats.logistic, 'log_laplace': stats.loglaplace,
        'log_gamma': stats.loggamma, 'lognormal': stats.lognorm, 'log-uniform': stats.loguniform,
        'maxwell': stats.maxwell, 'mielke_Beta-Kappa': stats.mielke, 'nakagami': stats.nakagami,
        'noncentral_chi-squared': stats.ncx2, 'noncentral_F': stats.ncf, 'noncentral_t': stats.nct,
        'normal': stats.norm, 'normal_inverse_gaussian': stats.norminvgauss, 'pareto': stats.pareto,
        'lomax': stats.lomax, 'power_lognormal': stats.powerlognorm, 'power_normal': stats.powernorm,
        'power-function': stats.powerlaw, 'R': stats.rdist, 'rayleigh': stats.rayleigh,
        'rice': stats.rayleigh, 'reciprocal_inverse_gaussian': stats.recipinvgauss,
        'semicircular': stats.semicircular, 'studentized_range': stats.studentized_range,
        'student-t': stats.t, 'trapezoidal': stats.trapezoid, 'triangular': stats.triang,
        'truncated_exponential': stats.truncexpon, 'truncated_normal': stats.truncnorm,
        'tukey-lambda': stats.tukeylambda, 'uniform': stats.uniform, 'von_mises': stats.vonmises,
        'wald': stats.wald, 'weibull_maximum_extreme_value': stats.weibull_max,
        'weibull_minimum_extreme_value': stats.weibull_min, 'wrapped_cauchy': stats.wrapcauchy
                
    }
    
    # Get a list of keys from this dictionary, to compare with the selected string:
    list_of_dictionary_keys = callable_statistical_distributions_dict.keys()
    
    #check if an invalid string was provided using the in method:
    # The string must be in the list of dictionary keys
    boolean_filter = statistical_distribution in list_of_dictionary_keys
    # if it is the list, boolean_filter == True. If it is not, boolean_filter == False
    
    if (boolean_filter == False):
        
        print(f"Please, select a valid statistical distribution to test: {list_of_dictionary_keys}")
        return "error"
    
    else:
        
        #Access the object correspondent to the distribution provided. To do so,
        # simply access dict['key1'], where 'key1' is a key from a dictionary dict ={"key1": 'val1'}
        # Access just as accessing a column from a dataframe.
        
        print("ATTENTION: If you want to test a normal distribution, use the function test_data_normality.")
        print("Function test_data_normality tests normality through 3 methods and compare them: D’Agostino and Pearson’s; Lilliefors; and Anderson-Darling tests.")
        print("The calculus of the p-value from the Anderson-Darling statistic is available only for some distributions. The function specific for the normality calculates these probabilities of following the normal.")
        print("Here, the function is destined to test a variety of distributions, and so only the Anderson-Darling test is performed.\n")
        
        DISTRIBUTION = callable_statistical_distributions_dict[statistical_distribution]
        
        anderson_darling_statistic = diagnostic.anderson_statistic(y, dist = DISTRIBUTION, fit = True, axis=0)
        print(f"Anderson-Darling statistic for the distribution {statistical_distribution} = {anderson_darling_statistic}")
        print("The Anderson–Darling test assesses whether a sample comes from a specified distribution. It makes use of the fact that, when given a hypothesized underlying distribution and assuming the data does arise from this distribution, the cumulative distribution function (CDF) of the data can be assumed to follow a uniform distribution. The data can be then tested for uniformity with a distance test (Shapiro 1980).")
        # source: https://en.wikipedia.org/wiki/Anderson%E2%80%93Darling_test
        
        #Fit the distribution and get its parameters
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.fit.html
        
        distribution_parameters = DISTRIBUTION.fit(y, method = "MLE")
        # With method="MLE" (default), the fit is computed by minimizing the negative 
        # log-likelihood function. A large, finite penalty (rather than infinite negative 
        # log-likelihood) is applied for observations beyond the support of the distribution. 
        # With method="MM", the fit is computed by minimizing the L2 norm of the relative errors 
        # between the first k raw (about zero) data moments and the corresponding distribution 
        # moments, where k is the number of non-fixed parameters. 
        
        # distribution_parameters: Estimates for any shape parameters (if applicable), 
        # followed by those for location and scale.
        print(f"Distribution shape parameters calculated for {statistical_distribution}: {distribution_parameters}")
        
         #Create general statistics dictionary:
        general_statistics_dict = {

            "Count_of_analyzed_values": len(y)
            "Data_mean": np.mean(y),
            "Data_mean_ignoring_missing_values": np.nanmean(y),
            "Data_variance": np.var(y),
            "Data_variance_ignoring_missing_values": np.nanvar(y),
            "Data_standard_deviation": np.std(y),
            "Data_standard_deviation_ignoring_missing_values": np.nanstd(y),
            "Data_skewness": data_skew,
            "Data_kurtosis": data_kurtosis,
            "Data_mode": data_mode,
            "Anderson-Darling_statistic_A": anderson_darling_statistic,
            "Distribution_parameters": distribution_parameters

        }

        print("General statistics successfully returned as the dictionary general_statistics_dict.\n")
        print(general_statistics_dict)
        print("/n")
        
        # The critical values for the Anderson-Darling test are dependent on the 
        # specific distribution that is being tested. Tabulated values and formulas have been 
        # published (Stephens, 1974, 1976, 1977, 1979) for a few specific distributions (normal, 
        # lognormal, exponential, Weibull, logistic, extreme value type 1). The test is a 
        # one-sided test and the hypothesis that the distribution is of a specific form is 
        # rejected if the test statistic, A, is greater than the critical value. Note 
        # that for a given distribution, the Anderson-Darling statistic may be 
        # multiplied by a constant (which usually depends on the sample size, n). 
        # These constants are given in the various papers by Stephens. In the sample 
        # output below, this is the "adjusted Anderson-Darling" statistic. This is what 
        # should be compared against the critical values. Also, be aware that different 
        # constants (and therefore critical values) have been published. You just 
        # need to be aware of what constant was used for a given set of critical values 
        # (the needed constant is typically given with the critical values).
        # https://itl.nist.gov/div898/handbook/eda/section3/eda3e.htm

        if (show_probability_plot == True):
            #Obtain the probability plot  
            fig, ax = plt.subplots()

            ax.set_title(f"Probability Plot for {statistical_distribution} Distribution")

            #ROTATE X AXIS IN XX DEGREES
            plt.xticks(rotation = x_axis_rotation)
            # XX = 70 DEGREES x_axis (Default)
            #ROTATE Y AXIS IN XX DEGREES:
            plt.yticks(rotation = y_axis_rotation)
            # XX = 0 DEGREES y_axis (Default)   

            res = stats.probplot(y, dist = DISTRIBUTION, fit = True, plot = ax)
            #This function resturns a tuple, so we must store it into res
                   
            #Other distributions to check, see scipy Stats documentation. 
            # you could test dist=stats.loggamma, where stats was imported from scipy
            # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html#scipy.stats.probplot

            ax.grid(grid)
            ax.legend()

            if (export_png == True):
                # Image will be exported
                import os

                #check if the user defined a directory path. If not, set as the default root path:
                if (directory_to_save is None):
                    #set as the default
                    directory_to_save = "/"

                #check if the user defined a file name. If not, set as the default name for this
                # function.
                if (file_name is None):
                    #set as the default
                    file_name = "probability_plot"

                #check if the user defined an image resolution. If not, set as the default 110 dpi
                # resolution.
                if (png_resolution_dpi is None):
                    #set as 110 dpi
                    png_resolution_dpi = 110

                #Get the new_file_path
                new_file_path = os.path.join(directory_to_save, file_name)

                #Export the file to this new path:
                # The extension will be automatically added by the savefig method:
                plt.savefig(new_file_path, dpi = png_resolution_dpi, quality = 100, format = 'png', transparent = False) 
                #quality could be set from 1 to 100, where 100 is the best quality
                #format (str, supported formats) = 'png', 'pdf', 'ps', 'eps' or 'svg'
                #transparent = True or False
                # For other parameters of .savefig method, check https://indianaiproduction.com/matplotlib-savefig/
                print (f"Figure exported as \'{new_file_path}.png\'. Any previous file in this root path was overwritten.")

            #Set image size (x-pixels, y-pixels) for printing in the notebook's cell:
            plt.figure(figsize=(12, 8))
            #fig.tight_layout()

            ## Show an image read from an image file:
            ## import matplotlib.image as pltimg
            ## img=pltimg.imread('mydecisiontree.png')
            ## imgplot = plt.imshow(img)
            ## See linkedIn Learning course: "Supervised machine learning and the technology boom",
            ##  Ex_Files_Supervised_Learning, Exercise Files, lesson '03. Decision Trees', '03_05', 
            ##  '03_05_END.ipynb'
            plt.show()

        return general_statistics_dict

# **Function for column filtering (selecting); or column renaming**

In [ ]:
def col_filter_rename (df, cols_list, mode = 'filter'):
    
    import pandas as pd
    
    #mode = 'filter' for filtering only the list of columns passed as cols_list;
    #mode = 'rename' for renaming the columns with the names passed as cols_list.
    
    #cols_list = list of strings containing the names (headers) of the columns to select
    # (filter); or to be set as the new columns' names, according to the selected mode.
    # For instance: cols_list = ['col1', 'col2', 'col3'] will 
    # select columns 'col1', 'col2', and 'col3' (or rename the columns with these names). 
    # Declare the names inside quotes.
    
    print(f"Original columns in the dataframe:\n{df.columns}")
    
    if (mode == 'filter'):
        
        #filter the dataframe so that it will contain only the cols_list.
        df = df[cols_list]
        print("Dataframe filtered according to the list provided.")
        
    elif (mode == 'rename'):
        
        #Check if the number of columns of the dataset is equal to the number of elements
        # of the new list. It will avoid raising an exception error.
        boolean_filter = (len(cols_list) == len(df.columns))
        
        if (boolean_filter == False):
            #Impossible to rename, number of elements are different.
            print("The number of columns of the dataframe is different from the number of elements of the list. Please, provide a list with number of elements equals to the number of columns.")
        
        else:
            #Same number of elements, so that we can update the columns' names.
            df.columns = cols_list
            print("Dataframe columns renamed according to the list provided.")
            print("Warning: the substitution is element-wise: the first element of the list is now the name of the first column, and so on, ..., so that the last element is the name of the last column.")
        
        
    else:
        print("Enter a valid mode: \'filter\' or \'rename\'.")
    
    return df

# **Function for dropping specific columns or rows from the dataframe**

In [3]:
def drop_columns (df, what_to_drop = 'columns', cols_list = None, row_index_list = None, reset_index_after_drop = True):
    
    import pandas as pd
    
    # check https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html?highlight=drop
    
    # what_to_drop = 'columns' for removing the columns specified by their names (headers)
    # in cols_list (a list of strings).
    # what_to_drop = 'rows' for removing the rows specified by their indices in
    # row_index_list (a list of integers). Remember that the indexing starts from zero, i.e.,
    # the first row is row number zero.
    
    # cols_list = list of strings containing the names (headers) of the columns to be removed
    # For instance: cols_list = ['col1', 'col2', 'col3'] will 
    # remove columns 'col1', 'col2', and 'col3' from the dataframe.
    # If a single column will be dropped, you can declare it as a string (outside a list)
    # e.g. cols_list = 'col1'; or cols_list = ['col1']
    
    # row_index_list = a list of integers containing the indices of the rows that will be dropped.
    # e.g. row_index_list = [0, 1, 2] will drop the rows with indices 0 (1st row), 1 (2nd row), and
    # 2 (third row). Again, if a single row will be dropped, you can declare it as an integer (outside
    # a list).
    # e.g. row_index_list = 20 or row_index_list = [20] to drop the row with index 20 (21st row).
    
    # reset_index_after_drop = True. keep it True to restarting the indexing numeration after dropping.
    # Alternatively, set reset_index_after_drop = False to keep the original numeration (the removed indices
    # will be missing).
    
    #Create a local copy of the dataframe to manipulate:
    DATASET = df
    
    if (what_to_drop == 'columns'):
        
        if (cols_list is None):
            #check if a list was not input:
            print("Input a list of columns cols_list to be dropped.")
            return "error"
        
        else:
            #Drop the columns in cols_list:
            DATASET.drop(columns = cols_list)
            print(f"The columns in {cols_list} headers list were successfully removed.\n")
    
    elif (what_to_drop == 'rows'):
        
        if (row_index_list is None):
            #check if a list was not input:
            print("Input a list of rows indices row_index_list to be dropped.")
            return "error"
        
        else:
            #Drop the rows in row_index_list:
            DATASET.drop(row_index_list)
            print(f"The rows in {row_index_list} indices list were successfully removed.\n")
    
    else:
        print("Input a valid string as what_to_drop, rows or columns.")
        return "error"
    
    if (reset_index_after_drop == True):
        
        #restart the indexing
        DATASET = DATASET.reset_index(drop = True)
        print("The indices of the dataset were successfully restarted.\n")
    
    print("Check the 10 first rows from the returned dataset:\n")
    print(DATASET.head(10))
    
    return DATASET

# **Function for removing duplicate rows from the dataframe**

In [2]:
def remove_duplicate_rows (df, list_of_columns_to_analyze = None, which_row_to_keep = 'first', reset_index_after_drop = True):
    
    import pandas as pd
    # check https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
    
    # if list_of_columns_to_analyze = None, the whole dataset will be analyzed, i.e., rows
    # will be removed only if they have same values for all columns from the dataset.
    # Alternatively, pass a list of columns names (strings), if you want to remove rows with
    # same values for that combination of columns. Pass it as a list, even if there is a single column
    # being declared.
    # e.g. list_of_columns_to_analyze = ['column1'] will check only 'column1'. Entries with same value
    # on 'column1' will be considered duplicates and will be removed.
    # list_of_columns_to_analyze = ['col1', 'col2',  'col3'] will analyze the combination of 3 columns:
    # 'col1', 'col2', and 'col3'. Only rows with same value for these 3 columns will be considered
    # duplicates and will be removed.
    
    # which_row_to_keep = 'first' will keep the first detected row and remove all other duplicates. If
    # None or an invalid string is input, this method will be selected.
    # which_row_to_keep = 'last' will keep only the last detected duplicate row, and remove all the others.
    
    # reset_index_after_drop = True. keep it True to restarting the indexing numeration after dropping.
    # Alternatively, set reset_index_after_drop = False to keep the original numeration (the removed indices
    # will be missing).
    
    #Create a local copy of the dataframe to manipulate:
    DATASET = df
    
    if (which_row_to_keep == 'last'):
        
        #keep only the last duplicate.
        if (list_of_columns_to_analyze is None):
            # use the whole dataset
            DATASET = DATASET.drop_duplicates(keep = 'last')
            print(f"The rows with duplicate entries were successfully removed.")
            print("Only the last one of the duplicate entries was kept in the dataset.\n")
        
        else:
            #use the subset of columns
            if (list_of_columns_to_analyze is None):
                #check if a list was not input:
                print("Input a list of columns list_of_columns_to_analyze to be analyzed.")
                return "error"
        
            else:
                #Drop the columns in cols_list:
                DATASET = DATASET.drop_duplicates(subset = list_of_columns_to_analyze, keep = 'last')
                print(f"The rows with duplicate values for the columns in {list_of_columns_to_analyze} headers list were successfully removed.")
                print("Only the last one of the duplicate entries was kept in the dataset.\n")
    
    else:
        
        #keep only the first duplicate.
        if (list_of_columns_to_analyze is None):
            # use the whole dataset
            DATASET = DATASET.drop_duplicates()
            print(f"The rows with duplicate entries were successfully removed.")
            print("Only the first one of the duplicate entries was kept in the dataset.\n")
        
        else:
            #use the subset of columns
            if (list_of_columns_to_analyze is None):
                #check if a list was not input:
                print("Input a list of columns list_of_columns_to_analyze to be analyzed.")
                return "error"
        
            else:
                #Drop the columns in cols_list:
                DATASET = DATASET.drop_duplicates(subset = list_of_columns_to_analyze)
                print(f"The rows with duplicate values for the columns in {list_of_columns_to_analyze} headers list were successfully removed.")
                print("Only the first one of the duplicate entries was kept in the dataset.\n")
    
    if (reset_index_after_drop == True):
        
        #restart the indexing
        DATASET = DATASET.reset_index(drop = True)
        print("The indices of the dataset were successfully restarted.\n")
    
    print("Check the 10 first rows from the returned dataset:\n")
    print(DATASET.head(10))
    
    return DATASET

# **Function for exporting the dataframe as CSV File (to notebook's workspace)**

In [15]:
def export_dataframe (dataframe_to_be_exported, new_file_name, file_directory_path = None):
    
    import os
    import pandas as pd
    
    ## WARNING: all files exported from this function are .csv (comma separated values)
    
    # dataframe_to_be_exported: dataframe object that is going to be exported from the
    # function. Since it is an object (not a string), it should not be declared in quotes.
    # example: dataframe_to_be_exported = dataset will export the dataset object.
    # ATTENTION: The dataframe object must be a Pandas dataframe.
    
    # FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
    # (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
    # or FILE_DIRECTORY_PATH = "/folder"
    # If you want to export the file to AWS S3, this parameter will have no effect.
    # In this case, you can set FILE_DIRECTORY_PATH = None

    # NEW_FILE_NAME - (string, in quotes): input the name of the 
    # file without the extension. e.g. NEW_FILE_NAME = "my_file" will export a file
    # 'my_file.csv' to notebook's workspace.
    
    # Create the complete file path:
    file_path = os.path.join(file_directory_path, new_file_name)
    # Concatenate the extension ".csv":
    file_path = file_path + ".csv"

    dataframe_to_be_exported.to_csv(file_path, index = False)

    print(f"Dataframe {new_file_name_with_csv_extension} exported as CSV file to notebook\'s workspace as \'{file_path}\'.")
    print("Warning: if there was a file in this file path, it was replaced by the exported dataframe.")

# **Function for downloading a file from Google Colab to the local machine; or uploading a file from the machine to Colab's instant memory**

In [16]:
def upload_to_or_download_file_from_colab (action = 'download', file_to_download_from_colab = None):
    
    # action = 'download' to download the file to the local machine
    # action = 'upload' to upload a file from local machine to
    # Google Colab's instant memory
    
    # file_to_download_from_colab = None. This parameter is obbligatory when
    # action = 'download'. 
    # Declare as file_to_download_from_colab the file that you want to download, with
    # the correspondent extension.
    # It should not be declared in quotes.
    # e.g. to download a dictionary named dict, object_to_download_from_colab = 'dict.pkl'
    # To download a dataframe named df, declare object_to_download_from_colab = 'df.csv'
    # To export a model named keras_model, declare object_to_download_from_colab = 'keras_model.h5'
 
    from google.colab import files
    # google.colab library must be imported only in case 
    # it is going to be used, for avoiding 
    # AWS compatibility issues.
        
    if (action == 'upload'):
            
        print("Click on the button for file selection and select the files from your machine that will be uploaded in the Colab environment.")
        print("Warning: the files will be removed from Colab memory after the Kernel dies or after the notebook is closed.")
        # this functionality requires the previous declaration:
        ## from google.colab import files
            
        colab_files_dict = files.upload()
            
        # The files are stored into a dictionary called colab_files_dict where the keys
        # are the names of the files and the values are the files themselves.
        ## e.g. if you upload a single file named "dictionary.pkl", the dictionary will be
        ## colab_files_dict = {'dictionary.pkl': file}, where file is actually a big string
        ## representing the contents of the file. The length of this value is the size of the
        ## uploaded file, in bytes.
        ## To access the file is like accessing a value from a dictionary: 
        ## d = {'key1': 'val1'}, d['key1'] == 'val1'
        ## we simply declare the key inside brackets and quotes, the same way we would do for
        ## accessing the column of a dataframe.
        ## In this example, colab_files_dict['dictionary.pkl'] access the content of the 
        ## .pkl file, and len(colab_files_dict['dictionary.pkl']) is the size of the .pkl
        ## file in bytes.
        ## To check the dictionary keys, apply the method .keys() to the dictionary (with empty
        ## parentheses): colab_files_dict.keys()
            
        for key in colab_files_dict.keys():
            #loop through each element of the list of keys of the dictionary
            # (list colab_files_dict.keys()). Each element is named 'key'
            print(f"User uploaded file {key} with length {len(colab_files_dict[key])} bytes.")
            # The key is the name of the file, and the length of the value
            ## correspondent to the key is the file's size in bytes.
            ## Notice that the content of the uploaded object must be passed 
            ## as argument for a proper function to be interpreted. 
            ## For instance, the content of a xlsx file should be passed as
            ## argument for Pandas .read_excel function; the pkl file must be passed as
            ## argument for pickle.
            ## e.g., if you uploaded 'table.xlsx' and stored it into colab_files_dict you should
            ## declare df = pd.read_excel(colab_files_dict['table.xlsx']) to obtain a dataframe
            ## df from the uploaded table. Notice that is the value, not the key, that is the
            ## argument.
                
            print("The uploaded files are stored into a dictionary object named as colab_files_dict.")
            print("Each key from this dictionary is the name of an uploaded file. The value correspondent to that key is the file itself.")
            print("The structure of a general Python dictionary is dict = {\'key1\': value1}. To access value1, declare file = dict[\'key1\'], as if you were accessing a column from a dataframe.")
            print("Then, if you uploaded a file named \'table.xlsx\', you can access this file as:")
            print("uploaded_file = colab_files_dict[\'table.xlsx\']")
            print("Notice, though, that the object uploaded_file is the whole file content, not a Python object already converted. To convert to a Python object, pass this element as argument for a proper function or method.")
            print("In this example, to convert the object uploaded_file to a dataframe, Pandas pd.read_excel function could be used. In the following line, a df dataframe object is obtained from the uploaded file:")
            print("df = pd.read_excel(uploaded_file)")
            print("Also, the uploaded file itself will be available in the Colaboratory Notebook\'s workspace.")
            
            return colab_files_dict
        
    elif (action == 'download'):
            
        if (file_to_download_from_colab is None):
                
            #No object was declared
            print("Please, inform a file to download from the notebook\'s workspace. It should be declared in quotes and with the extension: e.g. \'table.csv\'.")
            
        else:
                
            print("The file will be downloaded to your computer.")

            files.download(file_to_download_from_colab)

            print(f"File {file_to_download_from_colab} successfully downloaded from Colab environment.")

    else:
            
            print("Please, select a valid action, \'download\' or \'upload\'.")

# **Function for exporting a list of files from notebook's workspace to AWS Simple Storage Service (S3)**

In [8]:
def export_files_to_s3 (list_of_file_names_with_extensions, directory_of_notebook_workspace_storing_files_to_export = None, s3_bucket_name = None, s3_obj_prefix = None):
    
    import os
    import boto3
    # boto3 is AWS S3 Python SDK
    # sagemaker and boto3 libraries must be imported only in case 
    # they are going to be used, for avoiding 
    # Google Colab compatibility issues.
    from getpass import getpass
    
    # list_of_file_names_with_extensions: list containing all the files to export to S3.
    # Declare it as a list even if only a single file will be exported.
    # It must be a list of strings containing the file names followed by the extensions.
    # Example, to a export a single file my_file.ext, where my_file is the name and ext is the
    # extension:
    # list_of_file_names_with_extensions = ['my_file.ext']
    # To export 3 files, file1.ext1, file2.ext2, and file3.ext3:
    # list_of_file_names_with_extensions = ['file1.ext1', 'file2.ext2', 'file3.ext3']
    # Other examples:
    # list_of_file_names_with_extensions = ['Screen_Shot.png', 'dataset.csv']
    # list_of_file_names_with_extensions = ["dictionary.pkl", "model.h5"]
    # list_of_file_names_with_extensions = ['doc.pdf', 'model.dill']
    
    # directory_of_notebook_workspace_storing_files_to_export: directory from notebook's workspace
    # from which the files will be exported to S3. Keep it None, or
    # directory_of_notebook_workspace_storing_files_to_export = "/"; or
    # directory_of_notebook_workspace_storing_files_to_export = '' (empty string) to export from
    # the root (main) directory.
    # Alternatively, set as a string containing only the directories and folders, not the file names.
    # Examples: directory_of_notebook_workspace_storing_files_to_export = 'folder1';
    # directory_of_notebook_workspace_storing_files_to_export = 'folder1/folder2/'
    
    # For this function, all exported files must be located in the same directory.
    
    
    # s3_bucket_name = None.
    ## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
    # with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
    # "aws-bucket-1"
    
    # s3_obj_prefix = None. Keep it None or as an empty string (s3_obj_key_prefix = '')
    # to import the whole bucket content, instead of a single object from it.
    # Alternatively, set it as a string containing the subfolder from the bucket to import:
    # Suppose that your bucket (admin-created) has four objects with the following object 
    # keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
    # s3-dg.pdf. The s3-dg.pdf key does not have a prefix, so its object appears directly 
    # at the root level of the bucket. If you open the Development/ folder, you see 
    # the Projects.xlsx object in it.
    # Check Amazon documentation:
    # https://docs.aws.amazon.com/AmazonS3/latest/userguide/object-keys.html
    
    # In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
    # where 'bucket' is the bucket's name, key_prefix = 'my_path/.../', without the
    # 'file.csv' (file name with extension) last part.
    
    # So, declare the prefix as S3_OBJECT_FOLDER_PREFIX to import only files from
    # a given folder (directory) of the bucket.
    # DO NOT PUT A SLASH before (to the right of) the prefix;
    # DO NOT ADD THE BUCKET'S NAME TO THE right of the prefix:
    # S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/"

    # Alternatively, provide the full path of a given file if you want to import only it:
    # S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/my_file.ext"
    # where my_file is the file's name, and ext is its extension.


    # Attention: after running this function for connecting with AWS Simple Storage System (S3), 
    # your 'AWS Access key ID' and your 'Secret access key' will be requested.
    # The 'Secret access key' will be hidden through dots, so it cannot be visualized or copied by
    # other users. On the other hand, the same is not true for 'Access key ID', the bucket's name 
    # and the prefix. All of these are sensitive information from the organization.
    # Therefore, after importing the information, always remember of cleaning the output of this cell
    # and of removing such information from the strings.
    # Remember that these data may contain privilege for accessing the information, so it should not
    # be used for non-authorized people.

    # Also, remember of deleting the exported from the workspace after finishing the analysis.
    # The costs for storing the files in S3 is quite inferior than those for storing directly in the
    # workspace. Also, files stored in S3 may be accessed for other users than those with access to
    # the notebook's workspace.
    
    
    # Check if directory_of_notebook_workspace_storing_files_to_export is None. 
    # If it is, make it the root directory:
    if ((directory_of_notebook_workspace_storing_files_to_export is None)|(str(directory_of_notebook_workspace_storing_files_to_export) == "/")):
            
            # For the S3 buckets, the path should not start with slash. Assign the empty
            # string instead:
            directory_of_notebook_workspace_storing_files_to_export = ""
            print("The files will be exported from the notebook\'s root directory to S3.")
    
    elif (str(directory_of_notebook_workspace_storing_files_to_export) == ""):
        
            # Guarantee that the path is the empty string.
            # Avoid accessing the else condition, what would raise an error
            # since the empty string has no character of index 0
            directory_of_notebook_workspace_storing_files_to_export = str(directory_of_notebook_workspace_storing_files_to_export)
            print("The files will be exported from the notebook\'s root directory to S3.")
          
    else:
        # Use the str attribute to guarantee that the path was read as a string:
        directory_of_notebook_workspace_storing_files_to_export = str(directory_of_notebook_workspace_storing_files_to_export)
            
        if(directory_of_notebook_workspace_storing_files_to_export[0] == "/"):
            # the first character is the slash. Let's remove it

            # In AWS, neither the prefix nor the path to which the file will be imported
            # (file from S3 to workspace) or from which the file will be exported to S3
            # (the path in the notebook's workspace) may start with slash, or the operation
            # will not be concluded. Then, we have to remove this character if it is present.

            # The slash is character 0. Then, we want all characters from character 1 (the
            # second) to character len(str(path_to_store_imported_s3_bucket)) - 1, the index
            # of the last character. So, we can slice the string from position 1 to position
            # the slicing syntax is: string[1:] - all string characters from character 1
            # string[:10] - all string characters from character 10-1 = 9 (including 9); or
            # string[1:10] - characters from 1 to 9
            # So, slice the whole string, starting from character 1:
            directory_of_notebook_workspace_storing_files_to_export = directory_of_notebook_workspace_storing_files_to_export[1:]
            # attention: even though strings may be seem as list of characters, that can be
            # sliced, we cannot neither simply assign a character to a given position nor delete
            # a character from a position.

    # Ask the user to provide the credentials:
    ACCESS_KEY = input("Enter your AWS Access Key ID here (in the right). It is the value stored in the field \'Access key ID\' from your AWS user credentials CSV file.")
    print("\n") # line break
    SECRET_KEY = getpass("Enter your password (Secret key) here (in the right). It is the value stored in the field \'Secret access key\' from your AWS user credentials CSV file.")
        
    # The use of 'getpass' instead of 'input' hide the password behind dots.
    # So, the password is not visible by other users and cannot be copied.
        
    print("\n")
    print("WARNING: The bucket\'s name, the prefix, the AWS access key ID, and the AWS Secret access key are all sensitive information, which may grant access to protected information from the organization.\n")
    print("After finish exporting data to S3, remember of removing these information from the notebook, specially if it is going to be shared. Also, remember of removing the files from the workspace.\n")
    print("The cost for storing files in Simple Storage Service is quite inferior than the one for storing directly in SageMaker workspace. Also, files stored in S3 may be accessed for other users than those with access the notebook\'s workspace.\n")

    # Check if the user actually provided the mandatory inputs, instead
    # of putting None or empty string:
    if ((ACCESS_KEY is None) | (ACCESS_KEY == '')):
        print("AWS Access Key ID is missing. It is the value stored in the field \'Access key ID\' from your AWS user credentials CSV file.")
        return "error"
    elif ((SECRET_KEY is None) | (SECRET_KEY == '')):
        print("AWS Secret Access Key is missing. It is the value stored in the field \'Secret access key\' from your AWS user credentials CSV file.")
        return "error"
    elif ((s3_bucket_name is None) | (s3_bucket_name == '')):
        print ("Please, enter a valid S3 Bucket\'s name. Do not add sub-directories or folders (prefixes), only the name of the bucket itself.")
        return "error"
    
    else:
        # Use the str attribute to guarantee that all AWS parameters were properly read as strings, and not as
        # other variables (like integers or floats):
        ACCESS_KEY = str(ACCESS_KEY)
        SECRET_KEY = str(SECRET_KEY)
        s3_bucket_name = str(s3_bucket_name)

    if(s3_bucket_name[0] == "/"):
        # the first character is the slash. Let's remove it

        # In AWS, neither the prefix nor the path to which the file will be imported
        # (file from S3 to workspace) or from which the file will be exported to S3
        # (the path in the notebook's workspace) may start with slash, or the operation
        # will not be concluded. Then, we have to remove this character if it is present.

        # So, slice the whole string, starting from character 1 (as did for 
        # path_to_store_imported_s3_bucket):
        s3_bucket_name = s3_bucket_name[1:]

    # Remove any possible trailing (white and tab spaces) spaces
    # That may be present in the string. Use the Python string
    # rstrip method, which is the equivalent to the Trim function:
    # When no arguments are provided, the whitespaces and tabulations
    # are the removed characters
    # https://www.w3schools.com/python/ref_string_rstrip.asp?msclkid=ee2d05c3c56811ecb1d2189d9f803f65
    s3_bucket_name = s3_bucket_name.rstrip()
    ACCESS_KEY = ACCESS_KEY.rstrip()
    SECRET_KEY = SECRET_KEY.rstrip()
    # Since the user manually inputs the parameters ACCESS and SECRET_KEY,
    # it is easy to input whitespaces without noticing that.

    # Now process the non-obbligatory parameter.
    # Check if a prefix was passed as input parameter. If so, we must select only the names that start with
    # The prefix.
    # Example: in the bucket 'my_bucket' we have a directory 'dir1'.
    # In the main (root) directory, we have a file 'file1.json' like: '/file1.json'
    # If we pass the prefix 'dir1', we want only the files that start as '/dir1/'
    # such as: 'dir1/file2.json', excluding the file in the main (root) directory and excluding the files in other
    # directories. Also, we want to eliminate the file names with no extensions, like 'dir1/' or 'dir1/dir2',
    # since these object names represent folders or directories, not files.	

    if (s3_obj_prefix is None):
        print ("No prefix, specific object, or subdirectory provided.") 
        print (f"Then, exporting to \'{s3_bucket_name}\' root (main) directory.\n")
        # s3_path: path that the file should have in S3:
        s3_path = "" # empty string for the root directory
    elif ((s3_obj_prefix == "/") | (s3_obj_prefix == '')):
        # The root directory in the bucket must not be specified starting with the slash
        # If the root "/" or the empty string '' is provided, make
        # it equivalent to None (no directory)
        print ("No prefix, specific object, or subdirectory provided.") 
        print (f"Then, exporting to \'{s3_bucket_name}\' root (main) directory.\n")
        # s3_path: path that the file should have in S3:
        s3_path = "" # empty string for the root directory
    
    else:
        # Since there is a prefix, use the str attribute to guarantee that the path was read as a string:
        s3_obj_prefix = str(s3_obj_prefix)
            
        if(s3_obj_prefix[0] == "/"):
            # the first character is the slash. Let's remove it

            # In AWS, neither the prefix nor the path to which the file will be imported
            # (file from S3 to workspace) or from which the file will be exported to S3
            # (the path in the notebook's workspace) may start with slash, or the operation
            # will not be concluded. Then, we have to remove this character if it is present.

            # So, slice the whole string, starting from character 1 (as did for 
            # path_to_store_imported_s3_bucket):
            s3_obj_prefix = s3_obj_prefix[1:]

        # Remove any possible trailing (white and tab spaces) spaces
        # That may be present in the string. Use the Python string
        # rstrip method, which is the equivalent to the Trim function:
        s3_obj_prefix = s3_obj_prefix.rstrip()
            
        # s3_path: path that the file should have in S3:
        # Make the path the prefix itself, since there is a prefix:
        s3_path = s3_obj_prefix
            
        print("AWS Access Credentials, and bucket\'s prefix, object or subdirectory provided.\n")	

            
        print ("Starting connection with the S3 bucket.\n")
        
        try:
            # Start S3 client as the object 's3_client'
            s3_client = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key = SECRET_KEY)
        
            print(f"Credentials accepted by AWS. S3 client successfully started.\n")
            # An object 'data_table.xlsx' in the main (root) directory of the s3_bucket is stored in Python environment as:
            # s3.ObjectSummary(bucket_name='bucket_name', key='data_table.xlsx')
            # The name of each object is stored as the attribute 'key' of the object.
        
        except:
            
            print("Failed to connect to AWS Simple Storage Service (S3). Review if your credentials are correct.")
            print("The variable \'access_key\' must be set as the value (string) stored as \'Access key ID\' in your user security credentials CSV file.")
            print("The variable \'secret_key\' must be set as the value (string) stored as \'Secret access key\' in your user security credentials CSV file.")
        
        
        try:
            # Connect to the bucket specified as 'bucket_name'.
            # The bucket is started as the object 's3_bucket':
            s3_bucket = s3_client.Bucket(s3_bucket_name)
            print(f"Connection with bucket \'{s3_bucket_name}\' stablished.\n")
            
        except:
            
            print("Failed to connect with the bucket, which usually happens when declaring a wrong bucket\'s name.") 
            print("Check the spelling of your bucket_name string and remember that it must be all in lower-case.\n")
                
        # Now, let's obtain the lists of all file paths in the notebook's workspace and
        # of the paths that the files should have in S3, after being exported.
        
        try:
            
            # start the lists:
            workspace_full_paths = []
            s3_full_paths = []
            
            # Get the total of files in list_of_file_names_with_extensions:
            total_of_files = len(list_of_file_names_with_extensions)
            
            # And Loop through all elements, named 'my_file' from the list
            for my_file in list_of_file_names_with_extensions:
                
                # Get the full path in the notebook's workspace:
                workspace_file_full_path = os.path.join(directory_of_notebook_workspace_storing_files_to_export, my_file)
                # Get the full path that the file will have in S3:
                s3_file_full_path = os.path.join(s3_path, my_file)
                
                # Append these paths to the correspondent lists:
                workspace_full_paths.append(workspace_file_full_path)
                s3_full_paths.append(s3_file_full_path)
                
            # Now, both lists have the same number of elements. For an element (file) i,
            # workspace_full_paths has the full file path in notebook's workspace, and
            # s3_full_paths has the path that the new file should have in S3 bucket.
        
        except:
            
            print("The function returned an error when trying to access the list of files. Declare it as a list of strings, even if there is a single element in the list.")
            print("Example: list_of_file_names_with_extensions = [\'my_file.ext\']\n")
            return "error"
        
        
        # Now, loop through all elements i from the lists.
        # The first elements of the lists have index 0; the last elements have index
        # total_of_files - 1, since there are 'total_of_files' elements:
        
        # Then, export the correspondent element to S3:
        
        try:
            
            for i in range(total_of_files):
                # goes from i = 0 to i = total_of_files - 1

                # get the element from list workspace_file_full_path 
                # (original path of file i, from which it will be exported):
                PATH_IN_WORKSPACE = workspace_full_paths[i]

                # get the correspondent element of list s3_full_paths
                # (path that the file i should have in S3, after being exported):
                S3_FILE_PATH = s3_full_paths[i]

                # Start the new object in the bucket previously started as 's3_bucket'.
                # Start it with the specified prefix, in S3_FILE_PATH:
                new_s3_object = s3_bucket.Object(S3_FILE_PATH)
                
                # Finally, upload the file in PATH_IN_WORKSPACE.
                # Make new_s3_object the exported file:
            
                # Upload the selected object from the workspace path PATH_IN_WORKSPACE
                # to the S3 path specified as S3_FILE_PATH.
                # The parameter Filename must be input with the path of the copied file, including its name and
                # extension. Example Filename = "/my_table.xlsx" exports a xlsx file named 'my_table' to the notebook's main (root)
                # directory
                new_s3_object.upload_file(Filename = PATH_IN_WORKSPACE)

                print(f"The file \'{list_of_file_names_with_extensions[i]}\' was successfully exported from notebook\'s workspace to AWS Simple Storage Service (S3).\n")

                
            print("Finished exporting the files from the the notebook\'s workspace to S3 bucket. It may take a couple of minutes untill they be shown in S3 environment.\n") 
            print("Do not forget to delete these copies after finishing the analysis. They will remain stored in the bucket.\n")


        except:

            # Run this code for any other exception that may happen (no exception error
            # specified, so any exception runs the following code).
            # Check: https://pythonbasics.org/try-except/?msclkid=4f6b4540c5d011ecb1fe8a4566f632a6
            # for seeing how to handle successive exceptions

            print("Attention! The function raised an exception error, which is probably due to the AWS Simple Storage Service (S3) permissions.")
            print("Before running again this function, check this quick guide for configuring the permission roles in AWS.\n")
            print("It is necessary to create an user with full access permissions to interact with S3 from SageMaker. To configure the User, go to the upper ribbon of AWS, click on Services, and select IAM – Identity and Access Management.")
            print("1. In IAM\'s lateral panel, search for \'Users\' in the group of Access Management.")
            print("2. Click on the \'Add users\' button.")
            print("3. Set an user name in the text box \'User name\'.")
            print("Attention: users and S3 buckets cannot be written in upper case. Also, selecting a name already used by an Amazon user or bucket will raise an error message.\n")
            print("4. In the field \'Select type of Access to AWS\'-\'Select type of AWS credentials\' select the option \'Access key - Programmatic access\'. After that, click on the button \'Next: Permissions\'.")
            print("5. In the field \'Set Permissions\', keep the \'Add user to a group\' button marked.")
            print("6. In the field \'Add user to a group\', click on \'Create group\' (alternatively, you can be added to a group already configured or copy the permissions of another user.")
            print("7. In the text box \'Group\'s name\', set a name for the new group of permissions.")
            print("8. In the search bar below (\'Filter politics\'), search for a politics that fill your needs, and check the option button on the left of this politic. The politics \'AmazonS3FullAccess\' grants full access to the S3 content.")
            print("9. Finally, click on \'Create a group\'.")
            print("10. After the group is created, it will appear with a check box marked, over the previous groups. Keep it marked and click on the button \'Next: Tags\'.")
            print("11. Create and note down the Access key ID and Secret access key. You can also download a comma separated values (CSV) file containing the credentials for future use.")
            print("ATTENTION: These parameters are required for accessing the bucket\'s content from any application, including AWS SageMaker.")
            print("12. Click on \'Next: Review\' and review the user credentials information and permissions.")
            print("13. Click on \'Create user\' and click on the download button to download the CSV file containing the user credentials information.")
            print("The headers of the CSV file (the stored fields) is: \'User name, Password, Access key ID, Secret access key, Console login link\'.")
            print("You need both the values indicated as \'Access key ID\' and as \'Secret access key\' to fetch the S3 bucket.")
            print("\n") # line break
            print("After acquiring the necessary user privileges, use the boto3 library to export the file from the notebook’s workspace to the bucket (i.e., to upload a file to the bucket).")
            print("For exporting the file as a new bucket\'s file use the following code:\n")
            print("1. Set a variable \'access_key\' as the value (string) stored as \'Access key ID\' in your user security credentials CSV file.")
            print("2. Set a variable \'secret_key\' as the value (string) stored as \'Secret access key\' in your user security credentials CSV file.")
            print("3. Set a variable \'bucket_name\' as a string containing only the name of the bucket. Do not add subdirectories, folders (prefixes), or file names.")
            print("Example: if your bucket is named \'my_bucket\' and its main directory contains folders like \'folder1\', \'folder2\', etc, do not declare bucket_name = \'my_bucket/folder1\', even if you only want files from folder1.")
            print("ALWAYS declare only the bucket\'s name: bucket_name = \'my_bucket\'.")
            print("4. Set a variable \'file_path_in_workspace\' containing the path of the file in notebook’s workspace. The file will be exported from “file_path_in_workspace” to the S3 bucket.")
            print("If the file is stored in the notebook\'s root (main) directory: file_path = \"my_file.ext\".")
            print("If the path of the file in the notebook workspace is: \'dir1/…/dirN/my_file.ext\', where dirN is the N-th subdirectory, and dir1 is a folder or directory of the main (root) bucket\'s directory: file_path = \"dir1/…/dirN/my_file.ext\".")
            print("5. Set a variable named \'file_path_in_s3\' containing the path from the bucket’s subdirectories to the file you want to fetch. Include the file name and its extension.")
            print("6. Finally, declare the following code, which refers to the defined variables:\n")

            # Let's use triple quotes to declare a formated string
            example_code = """
                import boto3
                # Start S3 client as the object 's3_client'
                s3_client = boto3.resource('s3', aws_access_key_id = access_key, aws_secret_access_key = secret_key)
                # Connect to the bucket specified as 'bucket_name'.
                # The bucket is started as the object 's3_bucket':
                s3_bucket = s3_client.Bucket(bucket_name)
                # Start the new object in the bucket previously started as 's3_bucket'.
                # Start it with the specified prefix, in file_path_in_s3:
                new_s3_object = s3_bucket.Object(file_path_in_s3)
                # Finally, upload the file in file_path_in_workspace.
                # Make new_s3_object the exported file:
                # Upload the selected object from the workspace path file_path_in_workspace
                # to the S3 path specified as file_path_in_s3.
                # The parameter Filename must be input with the path of the copied file, including its name and
                # extension. Example Filename = "/my_table.xlsx" exports a xlsx file named 'my_table' to 
                # the notebook's main (root) directory.
                new_s3_object.upload_file(Filename = file_path_in_workspace)
                """

            print(example_code)

            print("An object \'my_file.ext\' in the main (root) directory of the s3_bucket is stored in Python environment as:")
            print("""s3.ObjectSummary(bucket_name='bucket_name', key='my_file.ext'""") 
            # triple quotes to keep the internal quotes without using too much backslashes "\" (the ignore next character)
            print("Then, the name of each object is stored as the attribute \'key\' of the object. To view all objects, we can loop through their \'key\' attributes:\n")
            example_code = """
                # Loop through all objects of the bucket:
                for stored_obj in s3_bucket.objects.all():		
                    # Loop through all elements 'stored_obj' from s3_bucket.objects.all()
                    # Which stores the ObjectSummary for all objects in the bucket s3_bucket:
                    # Print the object’s names:
                    print(stored_obj.key)
                    """

            print(example_code)

## **Call the functions**

### **Mounting Google Drive or S3 (AWS Simple Storage Service) bucket**

In [ ]:
SOURCE = 'aws'
# SOURCE = 'google' for mounting the google drive;
# SOURCE = 'aws' for accessing an AWS S3 bucket

## THE FOLLOWING PARAMETERS HAVE EFFECT ONLY WHEN SOURCE == 'aws':

PATH_TO_STORE_IMPORTED_S3_BUCKET = ''
# PATH_TO_STORE_IMPORTED_S3_BUCKET: path of the Python environment to which the
# S3 bucket contents will be imported. If it is None; or if it is an empty string; or if 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = '/', bucket will be imported to the root path. 
# Alternatively, input the path as a string (in quotes). e.g. 
# PATH_TO_STORE_IMPORTED_S3_BUCKET = 'copied_s3_bucket'

S3_BUCKET_NAME = 'my_bucket'
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"

S3_OBJECT_FOLDER_PREFIX = ""
# S3_OBJECT_FOLDER_PREFIX = None. Keep it None; or as an empty string 
# (S3_OBJECT_FOLDER_PREFIX = ''); or as the root "/" to import the 
# whole bucket content, instead of a single object from it.
# Alternatively, set it as a string containing the subfolder from the bucket to import:
# Suppose that your bucket (admin-created) has four objects with the following object 
# keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
# s3-dg.pdf. 
# The s3-dg.pdf key does not have a prefix, so its object appears directly 
# at the root level of the bucket. If you open the Development/ folder, you see 
# the Projects.xlsx object in it.
# In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
# where 'bucket' is the bucket's name, prefix = 'my_path/.../', without the
# 'file.csv' (file name with extension) last part.

# So, declare the prefix as S3_OBJECT_FOLDER_PREFIX to import only files from
# a given folder (directory) of the bucket.
# DO NOT PUT A SLASH before (to the right of) the prefix;
# DO NOT ADD THE BUCKET'S NAME TO THE right of the prefix:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/"

# Alternatively, provide the full path of a given file if you want to import only it:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/my_file.ext"
# where my_file is the file's name, and ext is its extension.


# Attention: after running this function for fetching AWS Simple Storage System (S3), 
# your 'AWS Access key ID' and your 'Secret access key' will be requested.
# The 'Secret access key' will be hidden through dots, so it cannot be visualized or copied by
# other users. On the other hand, the same is not true for 'Access key ID', the bucket's name 
# and the prefix. All of these are sensitive information from the organization.
# Therefore, after importing the information, always remember of cleaning the output of this cell
# and of removing such information from the strings.
# Remember that these data may contain privilege for accessing protected information, 
# so it should not be used for non-authorized people.

# Also, remember of deleting the imported files from the workspace after finishing the analysis.
# The costs for storing the files in S3 is quite inferior than those for storing directly in the
# workspace. Also, files stored in S3 may be accessed for other users than those with access to
# the notebook's workspace.
mount_storage_system (source = SOURCE, path_to_store_imported_s3_bucket = PATH_TO_STORE_IMPORTED_S3_BUCKET, s3_bucket_name = S3_BUCKET_NAME, s3_obj_prefix = S3_OBJECT_FOLDER_PREFIX)

### **Importing the dataset**

In [ ]:
## WARNING: Use this function to load dataframes stored on Excel (xls, xlsx, xlsm, xlsb, odf, ods and odt), 
## JSON, txt, or CSV (comma separated values) files.

FILE_DIRECTORY_PATH = "/"
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
# or FILE_DIRECTORY_PATH = "/folder"

FILE_NAME_WITH_EXTENSION = "dataset.csv"
# FILE_NAME_WITH_EXTENSION - (string, in quotes): input the name of the file with the 
# extension. e.g. FILE_NAME_WITH_EXTENSION = "file.xlsx", or, 
# FILE_NAME_WITH_EXTENSION = "file.csv", "file.txt", or "file.json"
# Again, the extensions may be: xls, xlsx, xlsm, xlsb, odf, ods, odt, json, txt or csv.

LOAD_TXT_FILE_WITH_JSON_FORMAT = False
# LOAD_TXT_FILE_WITH_JSON_FORMAT = False. Set LOAD_TXT_FILE_WITH_JSON_FORMAT = True 
# if you want to read a file with txt extension containing a text formatted as JSON 
# (but not saved as JSON).
# WARNING: if LOAD_TXT_FILE_WITH_JSON_FORMAT = True, all the JSON file parameters of the 
# function (below) must be set. If not, an error message will be raised.

HOW_MISSING_VALUES_ARE_REGISTERED = None
# HOW_MISSING_VALUES_ARE_REGISTERED = None: keep it None if missing values are registered as None,
# empty or np.nan. Pandas automatically converts None to NumPy np.nan objects (floats).
# This parameter manipulates the argument na_values (default: None) from Pandas functions.
# By default the following values are interpreted as NaN: ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, 
#‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, 
# ‘n/a’, ‘nan’, ‘null’.

# If a different denomination is used, indicate it as a string. e.g.
# HOW_MISSING_VALUES_ARE_REGISTERED = '.' will convert all strings '.' to missing values;
# HOW_MISSING_VALUES_ARE_REGISTERED = 0 will convert zeros to missing values.

# If dict passed, specific per-column NA values. For example, if zero is the missing value
# only in column 'numeric_col', you can specify the following dictionary:
# how_missing_values_are_registered = {'numeric-col': 0}

    
HAS_HEADER = True
# HAS_HEADER = True if the the imported table has headers (row with columns names).
# Alternatively, HAS_HEADER = False if the dataframe does not have header.

DECIMAL_SEPARATOR = '.'
# DECIMAL_SEPARATOR = '.' - String. Keep it '.' or None to use the period ('.') as
# the decimal separator. Alternatively, specify here the separator.
# e.g. DECIMAL_SEPARATOR = ',' will set the comma as the separator.
# It manipulates the argument 'decimal' from Pandas functions.

TXT_CSV_COL_SEP = "comma"
# txt_csv_col_sep = "comma" - This parameter has effect only when the file is a 'txt'
# or 'csv'. It informs how the different columns are separated.
# Alternatively, txt_csv_col_sep = "comma", or txt_csv_col_sep = "," 
# for columns separated by comma;
# txt_csv_col_sep = "whitespace", or txt_csv_col_sep = " " 
# for columns separated by simple spaces.
# You can also set a specific separator as string. For example:
# txt_csv_col_sep = '\s+'; or txt_csv_col_sep = '\t' (in this last example, the tabulation
# is used as separator for the columns - '\t' represents the tab character).

SHEET_TO_LOAD = None
# SHEET_TO_LOAD - This parameter has effect only when for Excel files.
# keep SHEET_TO_LOAD = None not to specify a sheet of the file, so that the first sheet
# will be loaded.
# SHEET_TO_LOAD may be an integer or an string (inside quotes). SHEET_TO_LOAD = 0
# loads the first sheet (sheet with index 0); SHEET_TO_LOAD = 1 loads the second sheet
# of the file (index 1); SHEET_TO_LOAD = "Sheet1" loads a sheet named as "Sheet1".
# Declare a number to load the sheet with that index, starting from 0; or declare a
# name to load the sheet with that name.

## Parameters for loading JSON files:

JSON_RECORD_PATH = None
# JSON_RECORD_PATH (string): manipulate parameter 'record_path' from json_normalize method.
# Path in each object to list of records. If not passed, data will be assumed to 
# be an array of records. If a given field from the JSON stores a nested JSON (or a nested
# dictionary) declare it here to decompose the content of the nested data. e.g. if the field
# 'books' stores a nested JSON, declare, JSON_RECORD_PATH = 'books'

JSON_FIELD_SEPARATOR = "_"
# JSON_FIELD_SEPARATOR = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
# Nested records will generate names separated by sep. 
# e.g., for JSON_FIELD_SEPARATOR = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
# Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
# the name of the columns of the dataframe will be formed by concatenating 'main_field', the
# separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...

JSON_METADATA_PREFIX_LIST = None
# JSON_METADATA_PREFIX_LIST: list of strings (in quotes). Manipulates the parameter 
# 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
# table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
# will be repeated in the rows of the dataframe to give the metadata (context) of the rows.

# e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
# 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
# Here, there are nested JSONs in the field 'books'. The fields that are not nested
# are 'name' and 'last'.
# Then, JSON_RECORD_PATH = 'books'
# JSON_METADATA_PREFIX_LIST = ['name', 'last']


# The dataframe will be stored in the object named 'dataset':
# Simply modify this object on the left of equality:
dataset = load_pandas_dataframe (file_directory_path = FILE_DIRECTORY_PATH, file_name_with_extension = FILE_NAME_WITH_EXTENSION, load_txt_file_with_json_format = LOAD_TXT_FILE_WITH_JSON_FORMAT, how_missing_values_are_registered = HOW_MISSING_VALUES_ARE_REGISTERED, has_header = HAS_HEADER, decimal_separator = DECIMAL_SEPARATOR, txt_csv_col_sep = TXT_CSV_COL_SEP, sheet_to_load = SHEET_TO_LOAD, json_record_path = JSON_RECORD_PATH, json_field_separator = JSON_FIELD_SEPARATOR, json_metadata_prefix_list = JSON_METADATA_PREFIX_LIST)

### **Converting JSON object to dataframe**

In [3]:
JSON_OBJ_TO_CONVERT = json_object #Alternatively: object containing the JSON to be converted

# JSON_OBJ_TO_CONVERT: object containing JSON, or string with JSON content to parse.
# Objects may be: string with JSON formatted text;
# list with nested dictionaries (JSON formatted);
# dictionaries, possibly with nested dictionaries (JSON formatted).

## Parameters for loading JSON files:

JSON_RECORD_PATH = None
# JSON_RECORD_PATH (string): manipulate parameter 'record_path' from json_normalize method.
# Path in each object to list of records. If not passed, data will be assumed to 
# be an array of records. If a given field from the JSON stores a nested JSON (or a nested
# dictionary) declare it here to decompose the content of the nested data. e.g. if the field
# 'books' stores a nested JSON, declare, JSON_RECORD_PATH = 'books'

JSON_FIELD_SEPARATOR = "_"
# JSON_FIELD_SEPARATOR = "_" (string). Manipulates the parameter 'sep' from json_normalize method.
# Nested records will generate names separated by sep. 
# e.g., for JSON_FIELD_SEPARATOR = ".", {‘foo’: {‘bar’: 0}} -> foo.bar.
# Then, if a given field 'main_field' stores a nested JSON with fields 'field1', 'field2', ...
# the name of the columns of the dataframe will be formed by concatenating 'main_field', the
# separator, and the names of the nested fields: 'main_field_field1', 'main_field_field2',...

JSON_METADATA_PREFIX_LIST = None
# JSON_METADATA_PREFIX_LIST: list of strings (in quotes). Manipulates the parameter 
# 'meta' from json_normalize method. Fields to use as metadata for each record in resulting 
# table. Declare here the non-nested fields, i.e., the fields in the principal JSON. They
# will be repeated in the rows of the dataframe to give the metadata (context) of the rows.

# e.g. Suppose a JSON with the following structure: {'name': 'Mary', 'last': 'Shelley',
# 'books': [{'title': 'Frankestein', 'year': 1818}, {'title': 'Mathilda ', 'year': 1819},{'title': 'The Last Man', 'year': 1826}]},
# Here, there are nested JSONs in the field 'books'. The fields that are not nested
# are 'name' and 'last'.
# Then, JSON_RECORD_PATH = 'books'
# JSON_METADATA_PREFIX_LIST = ['name', 'last']

#The dataframe will be stored in the object named 'dataset':
# Simply modify this object on the left of equality:
dataset = json_obj_to_dataframe (json_obj_to_convert = JSON_OBJ_TO_CONVERT, json_record_path = JSON_RECORD_PATH, json_field_separator = JSON_FIELD_SEPARATOR, json_metadata_prefix_list = JSON_METADATA_PREFIX_LIST)

### **Characterizing the dataframe**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

#New dataframes saved as df_shape, df_columns_list, df_dtypes, df_general_statistics, df_missing_values.
# Simply modify this object on the left of equality:
df_shape, df_columns_list, df_dtypes, df_general_statistics, df_missing_values = df_gen_charac (df = DATASET)

### **Characterizing the categorical variables**

#### Case 1: do not return a new column with encoded values

In [ ]:
#WARNING: Use this function to a analyze a single column from a dataframe.
# The pandas .unique method and the numpy.unique function can handle a single series
# (column) or 1D arrays.
    
#WARNING: The first return is the unique values summary/encoding table; 
# the second one will be the dataframe with the encoded column. This second 
# dataframe is returned only when encode_var = True

DATASET = dataset #Alternatively: object containing the dataset to be analyzed

CATEGORICAL_VAR_NAME = "categorical_column_name"
# CATEGORICAL_VAR_NAME: string (inside quotes) containing the name of the column 
#to be analyzed. e.g. CATEGORICAL_VAR_NAME = "column1"
# Alternatively: substitute the string by the name of the column where the categorical
# variable is. Keep it inside quotes. e.g. if the categorical values are in the column
# named 'col1', set CATEGORICAL_VAR_NAME = 'col1'

ENCODE_VAR = False
# ENCODE_VAR = False - keep it False not to associate an integer value to each 
# possible category. Alternatively, set ENCODE_VAR = True to associate an integer
# starting from zero to each categorical value. This should not be used in case there
# is no order to be associated. Also, the encoding will be performed in accordance to 
# the order of occurence in the dataframe df. In case there is no order, the One-Hot
# Encoding should be used.
# WARNING: if ENCODE_VAR = True, the function will return a new dataframe containing
# a column correspondent to the column input as 'CATEGORICAL_VAR_NAME' but with the
# correspondent integer values.

NEW_ENCODED_COL_NAME = None
# NEW_ENCODED_COL_NAME = None - This parameter shows effects only when ENCODED_VAR = True.
# It refers to the column that will be created on the original dataframe containing the
# integer values associated to the categorical values.
# Input a string inside quotes to define a name (header) for the column of the summary
# dataframe to store the integer correspondent to each categorical value.
# e.g. NEW_ENCODED_COL_NAME = "integer_vals". If no name is provided, the standard name
# formed by the string concatenation categorical_var_name + "_encoded" will be automatically
# assigned.

#New dataframe saved as unique_vals_summary. Simply modify this object on the left of equality:
unique_vals_summary = charac_cat_var (df = DATASET, categorical_var_name = CATEGORICAL_VAR_NAME, encode_var = ENCODE_VAR, new_encoded_col_name = NEW_ENCODED_COL_NAME)

#### Case 2: return a new column with encoded values

In [ ]:
#WARNING: Use this function to a analyze a single column from a dataframe.
# The pandas .unique method and the numpy.unique function can handle a single series
# (column) or 1D arrays.
    
#WARNING: The first return is the unique values summary/encoding table; 
# the second one will be the dataframe with the encoded column. This second 
# dataframe is returned only when encode_var = True

DATASET = dataset #Alternatively: object containing the dataset to be analyzed

CATEGORICAL_VAR_NAME = "categorical_column_name"
# CATEGORICAL_VAR_NAME: string (inside quotes) containing the name of the column 
#to be analyzed. e.g. CATEGORICAL_VAR_NAME = "column1"
# Alternatively: substitute the string by the name of the column where the categorical
# variable is. Keep it inside quotes. e.g. if the categorical values are in the column
# named 'col1', set CATEGORICAL_VAR_NAME = 'col1'

ENCODE_VAR = True
# ENCODE_VAR = False - keep it False not to associate an integer value to each 
# possible category. Alternatively, set ENCODE_VAR = True to associate an integer
# starting from zero to each categorical value. This should not be used in case there
# is no order to be associated. Also, the encoding will be performed in accordance to 
# the order of occurence in the dataframe df. In case there is no order, the One-Hot
# Encoding should be used.
# WARNING: if ENCODE_VAR = True, the function will return a new dataframe containing
# a column correspondent to the column input as 'CATEGORICAL_VAR_NAME' but with the
# correspondent integer values.

NEW_ENCODED_COL_NAME = None
# NEW_ENCODED_COL_NAME = None - This parameter shows effects only when ENCODED_VAR = True.
# It refers to the column that will be created on the original dataframe containing the
# integer values associated to the categorical values.
# Input a string inside quotes to define a name (header) for the column of the summary
# dataframe to store the integer correspondent to each categorical value.
# e.g. NEW_ENCODED_COL_NAME = "integer_vals". If no name is provided, the standard name
# formed by the string concatenation categorical_var_name + "_encoded" will be automatically
# assigned.

# New dataframes saved as unique_vals_summary and encoded_df.
# Simply modify these objects on the left of equality:
unique_vals_summary, encoded_df = charac_cat_var (df = DATASET, categorical_var_name = CATEGORICAL_VAR_NAME, encode_var = ENCODE_VAR, new_encoded_col_name = NEW_ENCODED_COL_NAME)

### **Plotting a bar chart**
- Bars may be vertically or horizontally oriented.
- Bar charts are plotted after selecting an aggregation function, and the cumulative percent curve may be obtained and plotted with the bars (in secondary axis).
- To obtain a Pareto chart, keep aggregate_function = 'sum', plot_cumulative_percent = True, and orientation = 'vertical'.

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

CATEGORICAL_VAR_NAME = 'categorical_column_name'
# CATEGORICAL_VAR_NAME: string (inside quotes) containing the name 
# of the column to be analyzed. e.g. 
# CATEGORICAL_VAR_NAME = "column1"

RESPONSE_VAR_NAME = "response_column_name"
# RESPONSE_VAR_NAME: string (inside quotes) containing the name 
# of the column that stores the response correspondent to the
# categories. e.g. RESPONSE_VAR_NAME = "response_feature"

AGGREGATE_FUNCTION = 'sum'
# AGGREGATE_FUNCTION = 'sum': String defining the aggregation 
# method that will be applied. Possible values:
# 'median', 'mean', 'mode', 'sum', 'min', 'max', 'variance',
# 'standard_deviation','10_percent_quantile', '20_percent_quantile',
# '25_percent_quantile', '30_percent_quantile', '40_percent_quantile',
# '50_percent_quantile', '60_percent_quantile', '70_percent_quantile',
# '75_percent_quantile', '80_percent_quantile', '90_percent_quantile',
# and '95_percent_quantile'.
# To use another aggregate function, the method must be added to the
# dictionary of methods agg_methods_dict, defined in the function.
# If None or an invalid function is input, 'sum' will be used.

ADD_SUFFIX_TO_AGGREGATED_COL = True
# ADD_SUFFIX_TO_AGGREGATED_COL = True will add a suffix to the
# aggregated column. e.g. 'responseVar_mean'. If ADD_SUFFIX_TO_AGGREGATED_COL
# = False, the aggregated column will have the original column name.
SUFFIX = None
# suffix = None. Keep it None if no suffix should be added, or if
# the name of the aggregate function should be used as suffix, after
# "_". Alternatively, set it as a string. As recommendation, put the
# "_" sign in the beginning of this string to separate the suffix from
# the original column name. e.g. if the response variable is 'Y' and
# suffix = '_agg', the new aggregated column will be named as 'Y_agg'
CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = True
# CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = True to calculate and plot
# the line of cumulative percent, or 
# CALCULATE_AND_PLOT_CUMULATIVE_PERCENT = False to omit it.
# This feature is only shown when AGGREGATE_FUNCTION = 'sum', 'median',
# 'mean', or 'mode'. So, it will be automatically set as False if 
# another aggregate is selected.
ORIENTATION = 'vertical'
# ORIENTATION = 'vertical' is the standard, and plots vertical bars
# (perpendicular to the X axis). In this case, the categories are shown
# in the X axis, and the correspondent responses are in Y axis.
# Alternatively, ORIENTATION = 'horizontal' results in horizontal bars.
# In this case, categories are in Y axis, and responses in X axis.
# If None or invalid values are provided, orientation is set as 'vertical'.
LIMIT_OF_PLOTTED_CATEGORIES = None
# LIMIT_OF_PLOTTED_CATEGORIES: integer value that represents
# the maximum of categories that will be plot. Keep it None to plot
# all categories. Alternatively, set an integer value. e.g.: if
# LIMIT_OF_PLOTTED_CATEGORIES = 4, but there are more categories,
# the dataset will be sorted in descending order and: 1) The remaining
# categories will be sum in a new category named 'others' if the
# aggregate function is 'sum'; 2) Or the other categories will be simply
# omitted from the plot, for other aggregate functions. Notice that
# it limits only the variables in the plot: all of them will be
# returned in the dataframe.
# Use this parameter to obtain a cleaner plot. Notice that the remaining
# columns will be aggregated as 'others' even if there is a single column
# beyond the limit.

X_AXIS_ROTATION = 70
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.

HORIZONTAL_AXIS_TITLE = None #Alternatively: string inside quotes for horizontal title
VERTICAL_AXIS_TITLE = None #Alternatively: string inside quotes for vertical title
PLOT_TITLE = None #Alternatively: string inside quotes for graphic title
# e.g. HORIZONTAL_AXIS_TITLE = "X", VERTICAL_AXIS_TITLE = "Y", PLOT_TITLE = "YxX"

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'scatter_plot_lin_reg.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

# New dataframe saved as aggregated_sorted_df. 
# Simply modify this object on the left of equality:
aggregated_sorted_df = bar_chart (df = DATASET, categorical_var_name = CATEGORICAL_VAR_NAME, response_var_name = RESPONSE_VAR_NAME, aggregate_function = AGGREGATE_FUNCTION, add_suffix_to_aggregated_col = ADD_SUFFIX_TO_AGGREGATED_COL, suffix = SUFFIX, calculate_and_plot_cumulative_percent = CALCULATE_AND_PLOT_CUMULATIVE_PERCENT, orientation = ORIENTATION, limit_of_plotted_categories = LIMIT_OF_PLOTTED_CATEGORIES, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Calculating cumulative statistics**
- Cumulative sum (cumsum); cumulative product (cumprod); cumulative maximum (cummax); cumulative minimum (cummin)

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

COLUMN_TO_ANALYZE = 'column_to_analyze'
# COLUMN_TO_ANALYZE = string (inside quotes) containing the name of the column that will be analyzed. 
# e.g. column_to_analyze = "column1" will analyze the column named as 'column1'.

CUMULATIVE_STATISTIC = 'sum'
# CUMULATIVE_STATISTIC: the statistic that will be calculated. The cumulative
# statistics allowed are: 'sum' (for cumulative sum, cumsum); 'product' 
# (for cumulative product, cumprod); 'max' (for cumulative maximum, cummax);
# and 'min' (for cumulative minimum, cummin).

NEW_CUM_STATS_COL_NAME = None
# NEW_CUM_STATS_COL_NAME = None or string (inside quotes), 
# containing the name of the new column created for storing the cumulative statistic
# calculated. 
# e.g. NEW_CUM_STATS_COL_NAME = "cum_stats" will create a column named as 'cum_stats'.
# If its None, the new column will be named as column_to_analyze + "_" + [selected
# cumulative function] ('cumsum', 'cumprod', 'cummax', 'cummin')

# New dataframe saved as new_df
# Simply modify this object on the left of equality:
new_df = calculate_cumulative_stats (df = DATASET, column_to_analyze = COLUMN_TO_ANALYZE, cumulative_statistic = CUMULATIVE_STATISTIC, new_cum_stats_col_name = NEW_CUM_STATS_COL_NAME)

### **Handling Missing Values**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

SUBSET_COLUMNS_LIST = None
# SUBSET_COLUMNS_LIST = list of columns to look for missing values. Only missing values
# in these columns will be considered for deciding which columns to remove.
# Declare it as a list of strings inside quotes containing the columns' names to look at,
# even if this list contains a single element. e.g. subset_columns_list = ['column1']
# will check only 'column1'; whereas subset_columns_list = ['col1', 'col2', 'col3'] will
# chek the columns named as 'col1', 'col2', and 'col3'.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
    
DROP_MISSING_VAL = True
# DROP_MISSING_VAL = True to eliminate the rows containing missing values.
# Alternatively: DROP_MISSING_VAL = False to use the filling method.

FILL_MISSING_VAL = False
# FILL_MISSING_VAL = False. Set this to True to activate the mode for filling the missing
# values.

ELIMINATE_ONLY_COMPLETELY_EMPTY_ROWS = False
# ELIMINATE_ONLY_COMPLETELY_EMPTY_ROWS = False - This parameter shows effect only when
# DROP_MISSING_VAL = True. If you set ELIMINATE_ONLY_COMPLETELY_EMPTY_ROWS = True, then
# only the rows where all the columns are missing will be eliminated.
# If you define a subset, then only the rows where all the subset columns are missing
# will be eliminated.

MINIMUM_NUMBER_OF_MISSING_VALS = None
# MINIMUM_NUMBER_OF_MISSING_VALS = None - This parameter shows effect only when
# DROP_MISSING_VAL = True. If you set MINIMUM_NUMBER_OF_MISSING_VALS equals to an integer 
# value, then only the rows where at least this integer number columns are missing will be 
# eliminated. e.g. if MINIMUM_NUMBER_OF_MISSING_VALS = 2, only rows containing missing 
# values for at least 2 columns will be eliminated.
# If you define a subset, then only the rows where all the subset columns are missing
# will be eliminated.

VALUE_TO_FILL = None
# VALUE_TO_FILL = None - This parameter shows effect only when
# FILL_MISSING_VAL = True. Set this parameter as a float value to fill all missing
# values with this value. e.g. VALUE_TO_FILL = 0 will fill all missing values with
# the number 0. You can also pass a function call like 
# VALUE_TO_FILL = np.sum(dataset['col1']). In this case, the missing values will be
# filled with the sum of the series dataset['col1']
# Alternatively, you can also input a string to fill the missing values. e.g.
# VALUE_TO_FILL = 'text' will fill all the missing values with the string "text".

FILL_METHOD = "fill_with_zeros"
# FILL_METHOD = "fill_with_zeros". - This parameter shows effect only 
# when FILL_MISSING_VAL = True.
# Alternatively: FILL_METHOD = "fill_with_zeros" - fill all the missing values with 0
# FILL_METHOD = "fill_with_value_to_fill" - fill the missing values with the value
# defined as the parameter VALUE_TO_FILL
# FILL_METHOD = "fill_with_avg" - fill the missing values with the average value for 
# each column.
# FILL_METHOD = "fill_by_interpolating" - fill by interpolating the previous and the 
# following value. A linear interpolation will be used.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
# FILL_METHOD = "ffill" - Forward fill: fills missing with previous value.
    
#WARNING: if the fillna method is selected (FILL_MISSING_VAL == True), but no filling
# methodology is selected, the missing values of the dataset will be filled with 0.
# The same applies when a non-valid fill methodology is selected.
# Pandas fillna method does not allow us to fill only a selected subset.
    
#WARNING: if FILL_MISSING_VAL == "fill_with_value_to_fill" but VALUE_TO_FILL is None, the 
# missing values will be filled with the value 0.    

#New dataframe saved as cleaned_df
# Simply modify this object on the left of equality:
cleaned_df = handle_missing_values (df = DATASET, subset_columns_list = SUBSET_COLUMNS_LIST, drop_missing_val = DROP_MISSING_VAL, fill_missing_val = FILL_MISSING_VAL, eliminate_only_completely_empty_rows = ELIMINATE_ONLY_COMPLETELY_EMPTY_ROWS, minimum_number_of_mis_vals = MINIMUM_NUMBER_OF_MISSING_VALS, value_to_fill = VALUE_TO_FILL, fill_method = FILL_METHOD)

### **Obtaining correlation plots**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

SHOW_MASKED_PLOT = True
#SHOW_MASKED_PLOT = True - keep as True if you want to see a cleaned version of the plot
# where a mask is applied. Alternatively, SHOW_MASKED_PLOT = True, or 
# SHOW_MASKED_PLOT = False

RESPONSES_TO_RETURN_CORR = None
#RESPONSES_TO_RETURN_CORR - keep as None to return the full correlation tensor.
# If you want to display the correlations for a particular group of features, input them
# as a list, even if this list contains a single element. Examples:
# responses_to_return_corr = ['response1'] for a single response
# responses_to_return_corr = ['response1', 'response2', 'response3'] for multiple
# responses. Notice that 'response1',... should be substituted by the name ('string')
# of a column of the dataset that represents a response variable.
# WARNING: The returned coefficients will be ordered according to the order of the list
# of responses. i.e., they will be firstly ordered based on 'response1'
# Alternatively: a list containing strings (inside quotes) with the names of the response
# columns that you want to see the correlations. Declare as a list even if it contains a
# single element.

SET_RETURNED_LIMIT = None
# SET_RETURNED_LIMIT = None - This variable will only present effects in case you have
# provided a response feature to be returned. In this case, keep set_returned_limit = None
# to return all of the correlation coefficients; or, alternatively, 
# provide an integer number to limit the total of coefficients returned. 
# e.g. if set_returned_limit = 10, only the ten highest coefficients will be returned. 

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'scatter_plot_lin_reg.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

#New dataframe saved as correlation_matrix. Simply modify this object on the left of equality:
correlation_matrix = correlation_plot (df = DATASET, show_masked_plot = SHOW_MASKED_PLOT, responses_to_return_corr = RESPONSES_TO_RETURN_CORR, set_returned_limit = SET_RETURNED_LIMIT, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Obtaining scatter plots and simple linear regressions**
        x1, y1, lab1: blue
        x2, y2, lab2: red
        x3, y3, lab3: green
        x4, y4, lab4: black
        x5, y5, lab5: magenta
        x6, y6, lab6: yellow

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

X1 = DATASET['X1']
#Alternatively: None; or other column in quotes, substituting 'X1'
# e.g. X1 = DATASET['Time'] for a X variable named 'Time', if 'Time' is a float, not a
# a datetime64. If 'Time' should be interpreted as a timestamp, then, we would declare as:

# X1 = (DATASET['Time']).astype('datetime64[D]')

# In summary: apply the method .astype('datetime64[D]') if you want the value to be
# interpreted (correctly) as a timestamp.

Y1 = DATASET['Y1'] 
#Alternatively: None; or other column in quotes, substituting 'Y1'
# e.g. Y1 = DATASET['Speed'] for a Y variable named 'Speed'

X2 = None #Alternatively: series for X2 (analogous to X1)
Y2 = None #Alternatively: series for Y2 (analogous to Y1)
X3 = None #Alternatively: series for X3 (analogous to X1)
Y3 = None #Alternatively: series for Y3 (analogous to Y1)
X4 = None #Alternatively: series for X4 (analogous to X1)
Y4 = None #Alternatively: series for Y4 (analogous to Y1)
X5 = None #Alternatively: series for X5 (analogous to X1)
Y5 = None #Alternatively: series for Y5 (analogous to Y1)
X6 = None #Alternatively: series for X6 (analogous to X1)
Y6 = None #Alternatively: series for Y6 (analogous to Y1)
# Warning: if X2, X3, X4, X5, and X6 were timestamps, do not forget to use the method
# .astype('datetime64[D]'). e.g.: X2 = (DATASET['DATE']).astype('datetime64[D]')
# If all X axis are the same, you can also declare: X2 = X1, X3 = X1, X4 = X1, X5 = X1
# and X6 = X1.

X_AXIS_ROTATION = 0
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

SHOW_LINEAR_REG = True
#Alternatively: set SHOW_LINEAR_REG = True to plot the linear regressions graphics and show 
# the linear regressions calculated for each pair Y x X (i.e., each correlation 
# Y = aX + b, as well as the R² coefficient calculated). 
# Set SHOW_LINEAR_REG = False to omit both the linear regressions plots on the graphic, and
# the correlations and R² coefficients obtained.

GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.
ADD_SPLINE_LINES = False #Alternatively: True or False
# If ADD_SPLINE_LINES = False, no lines connecting the successive values are shown.
# Since we are obtaining a scatter plot, there is no meaning in omitting the dots,
# as we can do for the time series visualization function.

LAB1 = None #Alternatively: string inside quotes containing the label for series 1
LAB2 = None #Alternatively: string inside quotes containing the label for series 2
LAB3 = None #Alternatively: string inside quotes containing the label for series 3
LAB4 = None #Alternatively: string inside quotes containing the label for series 4
LAB5 = None #Alternatively: string inside quotes containing the label for series 5
LAB6 = None #Alternatively: string inside quotes containing the label for series 6
#e.g. LAB1 = "Y1_values"

HORIZONTAL_AXIS_TITLE = None #Alternatively: string inside quotes for horizontal title
VERTICAL_AXIS_TITLE = None #Alternatively: string inside quotes for vertical title
PLOT_TITLE = None #Alternatively: string inside quotes for graphic title
# e.g. HORIZONTAL_AXIS_TITLE = "X", VERTICAL_AXIS_TITLE = "Y", PLOT_TITLE = "YxX"

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'scatter_plot_lin_reg.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

#New dataframe saved as lin_reg_summary. Simply modify this object on the left of equality:
lin_reg_summary = scatter_plot_lin_reg (x1 = X1, y1 = Y1, x2 = X2, y2 = Y2, x3 = X3, y3 = Y3, x4 = X4, y4 = Y4, x5 = X5, y5 = Y5, x6 = X6, y6 = Y6, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, show_linear_reg = SHOW_LINEAR_REG, grid = GRID, add_splines_lines = ADD_SPLINE_LINES, lab1 = LAB1, lab2 = LAB2, lab3 = LAB3, lab4 = LAB4, lab5 = LAB5, lab6 = LAB6, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Visualizing time series**
        x1, y1, lab1: blue
        x2, y2, lab2: red
        x3, y3, lab3: green
        x4, y4, lab4: black
        x5, y5, lab5: magenta
        x6, y6, lab6: yellow

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

#X1 = dataset.index to use the index as the axis itself
X1 = (DATASET['DATE']).astype('datetime64[D]') 
#Alternatively: None; or other column in quotes, substituting 'DATE'
# WARNING: Modify only the object in the first parenthesis: DATASET['DATE']
# Do not modify the method .astype('datetime64[D]')
#Remove .astype('datetime64[D]') if it is not a datetime.
# e.g. X1 = DATASET['Time'] for a X variable named 'Time', if 'Time' is a float, not a
# a datetime64. If 'Time' should be interpreted as a timestamp, then, we would declare as:

# X1 = (DATASET['Time']).astype('datetime64[D]')

# In summary: apply the method .astype('datetime64[D]') if you want the value to be
# interpreted (correctly) as a timestamp.

#Notice that there is a data transforming step to guarantee that the 'DATE' was interpreted as a timestamp, not as object or string.
#The astype method defines the type of variable as 'datetime64[D]'. If we wanted the timestamps to be resolved in seconds, we should use
# 'datetime64[ns]'.
Y1 = DATASET['Y1'] 
#Alternatively: None; or other column in quotes, substituting 'Y1'
# e.g. Y1 = DATASET['Speed'] for a Y variable named 'Speed'

X2 = None #Alternatively: series for X2 (analogous to X1)
Y2 = None #Alternatively: series for Y2 (analogous to Y1)
X3 = None #Alternatively: series for X3 (analogous to X1)
Y3 = None #Alternatively: series for Y3 (analogous to Y1)
X4 = None #Alternatively: series for X4 (analogous to X1)
Y4 = None #Alternatively: series for Y4 (analogous to Y1)
X5 = None #Alternatively: series for X5 (analogous to X1)
Y5 = None #Alternatively: series for Y5 (analogous to Y1)
X6 = None #Alternatively: series for X6 (analogous to X1)
Y6 = None #Alternatively: series for Y6 (analogous to Y1)
# Warning: if X2, X3, X4, X5, and X6 were timestamps, do not forget to use the method
# .astype('datetime64[D]'). e.g.: X2 = (DATASET['DATE']).astype('datetime64[D]')
# If all X axis are the same, you can also declare: X2 = X1, X3 = X1, X4 = X1, X5 = X1
# and X6 = X1.

X_AXIS_ROTATION = 70
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.
ADD_SPLINE_LINES = True #Alternatively: True or False
# If ADD_SPLINE_LINES = False, no lines connecting the successive values are shown.
ADD_SCATTER_DOTS = False #Alternatively: True or False
# If ADD_SCATTER_DOTS = False, the dots (scatter plot) are omitted, so only the lines
# correspondent to the series are shown.

# Notice that adding the dots and omitting the spline lines is equivalent to obtain a
# scatter plot. If you want to do so, consider using the scatter_plot_lin_reg function, 
# capable of calculating the linear regressions.

LAB1 = None #Alternatively: string inside quotes containing the label for series 1
LAB2 = None #Alternatively: string inside quotes containing the label for series 2
LAB3 = None #Alternatively: string inside quotes containing the label for series 3
LAB4 = None #Alternatively: string inside quotes containing the label for series 4
LAB5 = None #Alternatively: string inside quotes containing the label for series 5
LAB6 = None #Alternatively: string inside quotes containing the label for series 6
#e.g. LAB1 = "Y1_values"

HORIZONTAL_AXIS_TITLE = None #Alternatively: string inside quotes for horizontal title
VERTICAL_AXIS_TITLE = None #Alternatively: string inside quotes for vertical title
PLOT_TITLE = None #Alternatively: string inside quotes for graphic title
# e.g. HORIZONTAL_AXIS_TITLE = "X", VERTICAL_AXIS_TITLE = "Y", PLOT_TITLE = "YxX"

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'time_series_vis.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

time_series_vis (x1 = X1, y1 = Y1, x2 = X2, y2 = Y2, x3 = X3, y3 = Y3, x4 = X4, y4 = Y4, x5 = X5, y5 = Y5, x6 = X6, y6 = Y6, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, add_splines_lines = ADD_SPLINE_LINES, add_scatter_dots = ADD_SCATTER_DOTS, lab1 = LAB1, lab2 = LAB2, lab3 = LAB3, lab4 = LAB4, lab5 = LAB5, lab6 = LAB6, horizontal_axis_title = HORIZONTAL_AXIS_TITLE, vertical_axis_title = VERTICAL_AXIS_TITLE, plot_title = PLOT_TITLE, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Visualizing histograms**

#### Case 1: automatically calculate the ideal histogram bin size
- The ideal bin interval is calculated through Montgomery's method. Histogram is obtained from this calculated bin size.
    - Douglas C. Montgomery (2009). Introduction to Statistical Process Control, Sixth Edition, John Wiley & Sons.

In [ ]:
# REMEMBER: A histogram is the representation of a statistical distribution 
# of a given variable.

DATASET = dataset #Alternatively: object containing the dataset to be analyzed

ANALYZED_VARIABLE = DATASET['analyzed_variable']
#Alternatively: other column in quotes, substituting 'analyzed_variable'
# e.g., if the analyzed variable is in a column named 'column1':
# ANALYZED_VARIABLE = DATASET['column1']

SET_GRAPHIC_BAR_WIDTH = 2.0
# This parameter must be visually adjusted for each particular analyzed variable.
# Manually set this parameter until you see only a minimal separation between successive
# bars (i.e., you know that the bars are not overlapping, but they are not so distant that
# the statistic distribution profile is not clear).
# You can input any numeric value, and the order of magnitude will vary depending on the
# dispersion and on the width of the sample space.
# e.g. SET_GRAPHIC_BAR_WIDTH = 3; SET_GRAPHIC_BAR_WIDTH = 0.003

X_AXIS_ROTATION = 70 
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

Y_AXIS_ROTATION = 0
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.

NORMAL_CURVE_OVERLAY = True
#Alternatively: set NORMAL_CURVE_OVERLAY = True to show a normal curve overlaying the
# histogram; or set NORMAL_CURVE_OVERLAY = False to omit the normal curve (show only
# the histogram).

DATA_UNITS_LABEL = None
# Input a string inside quotes for setting a label for the X-axis, that will represent
# The type of data that the histogram is evaluating, i.e., what the statistic distribution
# shown by the histogram means.
# e.g. if DATA_UNITS_LABEL = "particle_diameter_in_nm", the axis X will be labelled with
# this string. Then, we can know that the diagram represents the distribution (counts of
# data for each defined bin) of particles diameters.

Y_TITLE = None
#Alternatively: string inside quotes for vertical title. e.g. Y_TITLE = "Analyzed_values".

HISTOGRAM_TITLE = None
#Alternatively: string inside quotes for graphic title. e.g. 
# HISTOGRAM_TITLE = "Analyzed_values_histogram".

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'histogram.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

#New dataframes saved as general_statistics and frequency_tab.
# Simply modify this object on the left of equality:
general_statistics, frequency_tab = histogram (y = ANALYZED_VARIABLE, bar_width = SET_GRAPHIC_BAR_WIDTH, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, normal_curve_overlay = NORMAL_CURVE_OVERLAY, data_units_label = DATA_UNITS_LABEL, y_title = Y_TITLE, histogram_title = HISTOGRAM_TITLE, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

# Suppose we registered the values corresponding to a given feature / property / attribute
# in column Y, and we want to know the Y statistic distribution. the maximum value observed
# for Y is named Ymax, whereas the minimum value observed is Ymin.
# Therefore, our sample space ranges from Ymin to Ymax. Now, we divide this sample space into
# equally-separated intervals, named bins. The width of each bin is the bin_size. The 1st bin
# corresponds to the interval Ymin to (Ymin + bin_size), where we can call (Ymin + bin_size)
# = Y1. Then, the 2nd interval ranges from Y1 to (Y1 + bin_size),..., until we get to the
# last interval, where we find Ymax.
# Now, we count how many values of Y belong to each bin. The graphic of count of values in
# each bin x the bin interval (or the value correspondent to the half of the bin) is the
# histogram. For the first bin this mid-value would be Ymin + bin_size/2, since this value is
# exactly in the middle of interval Ymin to (Ymin + bin_size).

# In other words we can imagine that each Y value was print on the surface of a ball, and
# each bin is a bucket labelled Ymin - (Ymin + bin_size), (Ymin + bin_size) - 
# (Ymin + 2bin_size), untill we cover the Ymax value. We put every ball inside the bucket,
# given that the ball value must be in the interval labelling the bucket. Finally, we count
# the balls per bucket, and plot count of balls x (the middle value of the interval 
# labelling the correspondent bucket). This graphic will be the histogram and will 
# represent the statistical distribution.

#### Case 2: set number of bins
- Use this one if the distance between data is too small, or if the histogram function did not return a valid histogram.
- Here, the histogram is obtained by manually defining the total of bins (i.e., into how much intervals the sample space should be divided).

In [ ]:
# REMEMBER: A histogram is the representation of a statistical distribution 
# of a given variable.

DATASET = dataset #Alternatively: object containing the dataset to be analyzed

ANALYZED_VARIABLE = DATASET['analyzed_variable']
#Alternatively: other column in quotes, substituting 'analyzed_variable'
# e.g., if the analyzed variable is in a column named 'column1':
# ANALYZED_VARIABLE = DATASET['column1']

TOTAL_OF_BINS = 50
# This parameter must be an integer number: it represents the total of bins of the 
# histogram, i.e., the number of divisions of the sample space (in how much intervals
# the sample space will be divided. Check comments after the histogram_alternative
# function call).
# Manually adjust this parameter to obtain more or less resolution of the statistical
# distribution: less bins tend to result into higher counting of values per bin, since
# a larger interval of values is grouped. After modifying the total of bins, do not forget
# to adjust the bar width in SET_GRAPHIC_BAR_WIDTH.
# Examples: TOTAL_OF_BINS = 50, to divide the sample space into 50 equally-separated 
# intervals; TOTAL_OF_BINS = 10 to divide it into 10 intervals; TOTAL_OF_BINS = 100 to
# divide it into 100 intervals.

SET_GRAPHIC_BAR_WIDTH = 2.0
# This parameter must be visually adjusted for each particular analyzed variable.
# Manually set this parameter until you see only a minimal separation between successive
# bars (i.e., you know that the bars are not overlapping, but they are not so distant that
# the statistic distribution profile is not clear).
# You can input any numeric value, and the order of magnitude will vary depending on the
# dispersion and on the width of the sample space.
# e.g. SET_GRAPHIC_BAR_WIDTH = 3; SET_GRAPHIC_BAR_WIDTH = 0.003

X_AXIS_ROTATION = 70 
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

Y_AXIS_ROTATION = 0
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).

GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.

DATA_UNITS_LABEL = None
# Input a string inside quotes for setting a label for the X-axis, that will represent
# The type of data that the histogram is evaluating, i.e., what the statistic distribution
# shown by the histogram means.
# e.g. if DATA_UNITS_LABEL = "particle_diameter_in_nm", the axis X will be labelled with
# this string. Then, we can know that the diagram represents the distribution (counts of
# data for each defined bin) of particles diameters.

Y_TITLE = None
#Alternatively: string inside quotes for vertical title. e.g. Y_TITLE = "Analyzed_values".

HISTOGRAM_TITLE = None
#Alternatively: string inside quotes for graphic title. e.g. 
# HISTOGRAM_TITLE = "Analyzed_values_histogram".

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'histogram_alternative.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

#New dataframes saved as general_statistics and frequency_tab.
# Simply modify this object on the left of equality:
general_statistics, frequency_tab = histogram_alternative (y = ANALYZED_VARIABLE, total_of_bins = TOTAL_OF_BINS, bar_width = SET_GRAPHIC_BAR_WIDTH, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, data_units_label = DATA_UNITS_LABEL, y_title = Y_TITLE, histogram_title = HISTOGRAM_TITLE, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

# Suppose we registered the values corresponding to a given feature / property / attribute
# in column Y, and we want to know the Y statistic distribution. the maximum value observed
# for Y is named Ymax, whereas the minimum value observed is Ymin.
# Therefore, our sample space ranges from Ymin to Ymax. Now, we divide this sample space into
# equally-separated intervals, named bins. The width of each bin is the bin_size. The 1st bin
# corresponds to the interval Ymin to (Ymin + bin_size), where we can call (Ymin + bin_size)
# = Y1. Then, the 2nd interval ranges from Y1 to (Y1 + bin_size),..., until we get to the
# last interval, where we find Ymax.
# Now, we count how many values of Y belong to each bin. The graphic of count of values in
# each bin x the bin interval (or the value correspondent to the half of the bin) is the
# histogram. For the first bin this mid-value would be Ymin + bin_size/2, since this value is
# exactly in the middle of interval Ymin to (Ymin + bin_size).

# In other words we can imagine that each Y value was print on the surface of a ball, and
# each bin is a bucket labelled Ymin - (Ymin + bin_size), (Ymin + bin_size) - 
# (Ymin + 2bin_size), untill we cover the Ymax value. We put every ball inside the bucket,
# given that the ball value must be in the interval labelling the bucket. Finally, we count
# the balls per bucket, and plot count of balls x (the middle value of the interval 
# labelling the correspondent bucket). This graphic will be the histogram and will 
# represent the statistical distribution.

### **Testing data normality and visualizing the probability plot**
- Check the probability that data is actually described by a normal distribution.

In [ ]:
# WARNING: The statistical tests require at least 20 samples

DATASET = dataset #Alternatively: object containing the dataset to be analyzed

Y = DATASET['Y'] 
#Alternatively: other column in quotes, substituting 'Y'
# e.g. Y = DATASET['Speed'] for a Y variable named 'Speed'

ALPHA = 0.10
# Confidence level = 1 - ALPHA. For ALPHA = 0.10, we get a 0.90 = 90% confidence
# Set ALPHA = 0.05 to get 0.95 = 95% confidence in the analysis.
# Notice that, when less trust is needed, we can increase ALPHA to get less restrictive
# results.

SHOW_PROBABILITY_PLOT = True
#Alternatively: set SHOW_PROBABILITY_PLOT = True to obtain the probability plot for the
# variable Y (normal distribution tested). 
# Set SHOW_PROBABILITY_PLOT = False to omit the probability plot.
X_AXIS_ROTATION = 0
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'scatter_plot_lin_reg.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

#New dataframe saved as data_normality_res
# Skewness kurtosis and general statistics dictionary returned as general_statistics_dict
# Simply modify these objects on the left of equality:
data_normality_res, general_statistics_dict = test_data_normality (y = Y, alpha = ALPHA, show_probability_plot = SHOW_PROBABILITY_PLOT, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Testing and visualizing probability plots for different statistical distributions**

In [ ]:
# WARNING: The statistical tests require at least 20 samples
# Attention: if you want to test a normal distribution, use the function 
# test_data_normality.Function test_data_normality tests normality through 3 methods 
# and compare them: D’Agostino and Pearson’s; Lilliefors; and Anderson-Darling tests.
# The calculus of the p-value from the Anderson-Darling statistic is available only 
# for some distributions. The function specific for the normality calculates these 
# probabilities of following the normal.
# Here, the function is destined to test a variety of distributions, and so only the 
# Anderson-Darling test is performed.

DATASET = dataset #Alternatively: object containing the dataset to be analyzed

Y = DATASET['Y'] 
#Alternatively: other column in quotes, substituting 'Y'
# e.g. Y = DATASET['Speed'] for a Y variable named 'Speed'

STATISTICAL_DISTRIBUTION = 'lognormal'
#STATISTICAL_DISTRIBUTION: string (inside quotes) containing the tested statistical 
# distribution. 
## Notice: if data Y follow a 'lognormal', log(Y) follow a normal
## Poisson is a special case from 'gamma' distribution.
## There are 91 accepted statistical distributions:
# 'alpha', 'anglit', 'arcsine', 'beta', 'beta_prime', 'bradford', 'burr', 'burr12', 'cauchy',
# 'skewed_cauchy', 'chi', 'chi-squared', 'cosine', 'double_gamma', 'double_weibull', 
# 'erlang', 'exponential', 'exponentiated_weibull', 'exponential_power',
# 'fatigue_life_birnbaum-saunders', 'fisk_log_logistic', 'folded_cauchy', 'folded_normal',
# 'F', 'gamma', 'generalized_logistic', 'generalized_pareto', 'generalized_exponential', 
# 'generalized_extreme_value', 'generalized_gamma', 'generalized_half-logistic', 
# 'generalized_hyperbolic', 'generalized_inverse_gaussian', 'generalized_normal', 
# 'gilbrat', 'gompertz_truncated_gumbel', 'gumbel', 'gumbel_left-skewed', 'half-cauchy', 
# 'half-normal', 'half-logistic', 'hyperbolic_secant', 'gauss_hypergeometric', 
# 'inverted_gamma', 'inverse_normal', 'inverted_weibull', 'johnson_SB', 'johnson_SU', 
# 'KSone', 'KStwo', 'KStwobign', 'laplace', 'asymmetric_laplace', 'left-skewed_levy', 
# 'levy', 'logistic', 'log_laplace', 'log_gamma', 'lognormal', 'log-uniform', 'maxwell', 
# 'mielke_Beta-Kappa', 'nakagami', 'noncentral_chi-squared', 'noncentral_F', 
# 'noncentral_t', 'normal', 'normal_inverse_gaussian', 'pareto', 'lomax', 'power_lognormal',
# 'power_normal', 'power-function', 'R', 'rayleigh', 'rice', 'reciprocal_inverse_gaussian', 
# 'semicircular', 'studentized_range', 'student-t', 'trapezoidal', 'triangular', 
# 'truncated_exponential', 'truncated_normal', 'tukey-lambda', 'uniform', 'von_mises', 
# 'wald', 'weibull_maximum_extreme_value', 'weibull_minimum_extreme_value', 'wrapped_cauchy'

SHOW_PROBABILITY_PLOT = True
#Alternatively: set SHOW_PROBABILITY_PLOT = True to obtain the probability plot for the
# variable Y (normal distribution tested). 
# Set SHOW_PROBABILITY_PLOT = False to omit the probability plot.
X_AXIS_ROTATION = 0
#Rotation of X axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
Y_AXIS_ROTATION = 0
#Rotation of Y axis labels. Alternatively, insert any numeric value from 0 to 90 (degrees).
GRID = True #Alternatively: True or False
# If GRID = False, no grid lines are shown in the graphic.

EXPORT_PNG = False
# Set EXPORT_PNG = False if you do not want to export the obtained image;
# Set EXPORT_PNG = True to export the obtained image.

DIRECTORY_TO_SAVE = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file will be stored. e.g. DIRECTORY_TO_SAVE = "/" 
# or DIRECTORY_TO_SAVE = "/folder"
# If EXPORT_PNG = True and DIRECTORY_TO_SAVE = None, the file will be saved in the root
# path, DIRECTORY_TO_SAVE = "/"

FILE_NAME = None
# This parameter has effect only if EXPORT_PNG = True.
# (string, in quotes): input the name you want for the file without the 
# extension, which will be 'png'. e.g. FILE_NAME = "my_image" will save a file 'my_image.png' 
# If EXPORT_PNG = True and FILE_NAME = None, the file will be saved as:
# 'scatter_plot_lin_reg.png'
# WARNING: if there is already a file in the path DIRECTORY_TO_SAVE saved as FILE_NAME,
# the file will be overwritten.

PNG_RESOLUTION_DPI = 110
# This parameter has effect only if EXPORT_PNG = True.
# Alternatively, input an integer that will correspond to the resolution of the exported
# image in dpi. If PNG_RESOLUTION_DPI = None, it will be set as 110.

# General statistics dictionary returned as general_statistics_dict
# Simply modify this object on the left of equality:
general_statistics_dict = test_stat_distribution (y = Y, statistical_distribution = STATISTICAL_DISTRIBUTION, show_probability_plot = SHOW_PROBABILITY_PLOT, x_axis_rotation = X_AXIS_ROTATION, y_axis_rotation = Y_AXIS_ROTATION, grid = GRID, export_png = EXPORT_PNG, directory_to_save = DIRECTORY_TO_SAVE, file_name = FILE_NAME, png_resolution_dpi = PNG_RESOLUTION_DPI)

### **Filtering (selecting); or renaming columns of the dataframe**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

MODE = 'filter'
# MODE = 'filter' for filtering only the list of columns passed as cols_list;
# MODE = 'rename' for renaming the columns with the names passed as cols_list.

COLS_LIST = ['column1', 'column2', 'column3']
# COLS_LIST = list of strings containing the names (headers) of the columns to select
# (filter); or to be set as the new columns' names, according to the selected mode.
# For instance: COLS_LIST = ['col1', 'col2', 'col3'] will 
# select columns 'col1', 'col2', and 'col3' (or rename the columns with these names). 
# Declare the names inside quotes.
# Simply substitute the list by the list of columns that you want to select; or the
# list of the new names you want to give to the dataset columns.

# New dataframe saved as new_df. Simply modify this object on the left of equality:
new_df = col_filter_rename (df = DATASET, cols_list = COLS_LIST, mode = MODE)

### **Dropping specific columns or rows from the dataframe**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

WHAT_TO_DROP = 'columns'
# WHAT_TO_DROP = 'columns' for removing the columns specified by their names (headers)
# in COLS_LIST (a list of strings).
# WHAT_TO_DROP = 'rows' for removing the rows specified by their indices in
# ROW_INDEX_LIST (a list of integers). Remember that the indexing starts from zero, i.e.,
# the first row is row number zero.

COLS_LIST = None
# COLS_LIST = list of strings containing the names (headers) of the columns to be removed
# For instance: COLS_LIST = ['col1', 'col2', 'col3'] will 
# remove columns 'col1', 'col2', and 'col3' from the dataframe.
# If a single column will be dropped, you can declare it as a string (outside a list)
# e.g. COLS_LIST = 'col1'; or COLS_LIST = ['col1']

ROW_INDEX_LIST = None
# ROW_INDEX_LIST = a list of integers containing the indices of the rows that will be dropped.
# e.g. ROW_INDEX_LIST = [0, 1, 2] will drop the rows with indices 0 (1st row), 1 (2nd row), and
# 2 (third row). Again, if a single row will be dropped, you can declare it as an integer (outside
# a list).
# e.g. ROW_INDEX_LIST = 20 or ROW_INDEX_LIST = [20] to drop the row with index 20 (21st row).
    
RESET_INDEX_AFTER_DROP = True
# RESET_INDEX_AFTER_DROP = True. keep it True to restarting the indexing numeration after dropping.
# Alternatively, set RESET_INDEX_AFTER_DROP = False to keep the original numeration (the removed indices
# will be missing).

# New dataframe saved as cleaned_df. Simply modify this object on the left of equality:
cleaned_df = drop_columns (df = DATASET, what_to_drop = WHAT_TO_DROP, cols_list = COLS_LIST, row_index_list = ROW_INDEX_LIST, reset_index_after_drop = RESET_INDEX_AFTER_DROP)

### **Removing duplicate rows from the dataframe**

In [ ]:
DATASET = dataset #Alternatively: object containing the dataset to be analyzed

LIST_OF_COLUMNS_TO_ANALYZE = None
# if LIST_OF_COLUMNS_TO_ANALYZE = None, the whole dataset will be analyzed, i.e., rows
# will be removed only if they have same values for all columns from the dataset.
# Alternatively, pass a list of columns names (strings), if you want to remove rows with
# same values for that combination of columns. Pass it as a list, even if there is a single column
# being declared.
# e.g. LIST_OF_COLUMNS_TO_ANALYZE = ['column1'] will check only 'column1'. Entries with same value
# on 'column1' will be considered duplicates and will be removed.
# LIST_OF_COLUMNS_TO_ANALYZE = ['col1', 'col2',  'col3'] will analyze the combination of 3 columns:
# 'col1', 'col2', and 'col3'. Only rows with same value for these 3 columns will be considered
# duplicates and will be removed.

WHICH_ROW_TO_KEEP = 'first'
# WHICH_ROW_TO_KEEP = 'first' will keep the first detected row and remove all other duplicates. If
# None or an invalid string is input, this method will be selected.
# WHICH_ROW_TO_KEEP = 'last' will keep only the last detected duplicate row, and remove all the others.
    
RESET_INDEX_AFTER_DROP = True
# RESET_INDEX_AFTER_DROP = True. keep it True to restarting the indexing numeration after dropping.
# Alternatively, set RESET_INDEX_AFTER_DROP = False to keep the original numeration (the removed indices
# will be missing).

# New dataframe saved as cleaned_df. Simply modify this object on the left of equality:
cleaned_df = remove_duplicate_rows (df = DATASET, list_of_columns_to_analyze = LIST_OF_COLUMNS_TO_ANALYZE, which_row_to_keep = WHICH_ROW_TO_KEEP, reset_index_after_drop = RESET_INDEX_AFTER_DROP)

## **Exporting the dataframe as CSV file (to notebook's workspace)**

In [ ]:
## WARNING: all files exported from this function are .csv (comma separated values)

DATAFRAME_TO_BE_EXPORTED = dataset
# Alternatively: object containing the dataset to be exported.
# DATAFRAME_TO_BE_EXPORTED: dataframe object that is going to be exported from the
# function. Since it is an object (not a string), it should not be declared in quotes.
# example: DATAFRAME_TO_BE_EXPORTED = dataset will export the dataset object.
# ATTENTION: The dataframe object must be a Pandas dataframe.

FILE_DIRECTORY_PATH = "/"
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory 
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = "/" 
# or FILE_DIRECTORY_PATH = "/folder"
# If you want to export the file to AWS S3, this parameter will have no effect.
# In this case, you can set FILE_DIRECTORY_PATH = None

NEW_FILE_NAME = "dataset"
# NEW_FILE_NAME - (string, in quotes): input the name of the 
# file without the extension. e.g. set NEW_FILE_NAME = "my_file" 
# to export the CSV file 'my_file.csv' to notebook's workspace.

export_dataframe (dataframe_to_be_exported = DATAFRAME_TO_BE_EXPORTED, new_file_name = NEW_FILE_NAME, file_directory_path = FILE_DIRECTORY_PATH)

## **Downloading a file from Google Colab to the local machine; or uploading a file from the machine to Colab's instant memory**

#### Case 1: upload a file to Colab's workspace

In [ ]:
ACTION = 'upload'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to Google Colab's 
# instant memory

FILE_TO_DOWNLOAD_FROM_COLAB = None
# FILE_TO_DOWNLOAD_FROM_COLAB = None. This parameter is obbligatory when
# action = 'download'. 
# Declare as FILE_TO_DOWNLOAD_FROM_COLAB the file that you want to download, with
# the correspondent extension.
# It should not be declared in quotes.
# e.g. to download a dictionary named dict, FILE_TO_DOWNLOAD_FROM_COLAB = 'dict.pkl'
# To download a dataframe named df, declare FILE_TO_DOWNLOAD_FROM_COLAB = 'df.csv'
# To export a model named keras_model, declare FILE_TO_DOWNLOAD_FROM_COLAB = 'keras_model.h5'

# Dictionary storing the uploaded files returned as colab_files_dict.
# Simply modify this object on the left of the equality:
colab_files_dict = upload_to_or_download_file_from_colab (action = ACTION, file_to_download_from_colab = FILE_TO_DOWNLOAD_FROM_COLAB)

#### Case 2: download a file from Colab's workspace

In [ ]:
ACTION = 'download'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to Google Colab's 
# instant memory

FILE_TO_DOWNLOAD_FROM_COLAB = None
# FILE_TO_DOWNLOAD_FROM_COLAB = None. This parameter is obbligatory when
# action = 'download'. 
# Declare as FILE_TO_DOWNLOAD_FROM_COLAB the file that you want to download, with
# the correspondent extension.
# It should not be declared in quotes.
# e.g. to download a dictionary named dict, FILE_TO_DOWNLOAD_FROM_COLAB = 'dict.pkl'
# To download a dataframe named df, declare FILE_TO_DOWNLOAD_FROM_COLAB = 'df.csv'
# To export a model nameACTION = 'upload'
# ACTION = 'download' to download the file to the local machine
# ACTION = 'upload' to upload a file from local machine to Google Colab's 
# instant memory

upload_to_or_download_file_from_colab (action = ACTION, file_to_download_from_colab = FILE_TO_DOWNLOAD_FROM_COLAB)

## **Exporting a list of files from notebook's workspace to AWS Simple Storage Service (S3)**

In [ ]:
LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['s3_file1.txt', 's3_file2.txt']
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS: list containing all the files to export to S3.
# Declare it as a list even if only a single file will be exported.
# It must be a list of strings containing the file names followed by the extensions.
# Example, to a export a single file my_file.ext, where my_file is the name and ext is the
# extension:
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['my_file.ext']
# To export 3 files, file1.ext1, file2.ext2, and file3.ext3:
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['file1.ext1', 'file2.ext2', 'file3.ext3']
# Other examples:
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['Screen_Shot.png', 'dataset.csv']
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ["dictionary.pkl", "model.h5"]
# LIST_OF_FILE_NAMES_WITH_EXTENSIONS = ['doc.pdf', 'model.dill']

DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = ''
# DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT: directory from notebook's workspace
# from which the files will be exported to S3. Keep it None, or
# DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = "/"; or
# DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = '' (empty string) to export from
# the root (main) directory.
# Alternatively, set as a string containing only the directories and folders, not the file names.
# Examples: DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = 'folder1';
# DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT = 'folder1/folder2/'
    
# For this function, all exported files must be located in the same directory.

S3_BUCKET_NAME = 'my_bucket'
## This parameter is obbligatory to access an AWS S3 bucket. Substitute it for a string
# with the bucket's name. e.g. s3_bucket_name = "aws-bucket-1" access a bucket named as
# "aws-bucket-1"

S3_OBJECT_FOLDER_PREFIX = ""
# S3_OBJECT_FOLDER_PREFIX = None. Keep it None; or as an empty string 
# (S3_OBJECT_FOLDER_PREFIX = ''); or as the root "/" to import the 
# whole bucket content, instead of a single object from it.
# Alternatively, set it as a string containing the subfolder from the bucket to import:
# Suppose that your bucket (admin-created) has four objects with the following object 
# keys: Development/Projects1.xls; Finance/statement1.pdf; Private/taxdocument.pdf; and
# s3-dg.pdf. 
# The s3-dg.pdf key does not have a prefix, so its object appears directly 
# at the root level of the bucket. If you open the Development/ folder, you see 
# the Projects.xlsx object in it.
# In summary, if the path of the file is: 'bucket/my_path/.../file.csv'
# where 'bucket' is the bucket's name, prefix = 'my_path/.../', without the
# 'file.csv' (file name with extension) last part.

# So, declare the prefix as S3_OBJECT_FOLDER_PREFIX to import only files from
# a given folder (directory) of the bucket.
# DO NOT PUT A SLASH before (to the right of) the prefix;
# DO NOT ADD THE BUCKET'S NAME TO THE right of the prefix:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/"

# Alternatively, provide the full path of a given file if you want to import only it:
# S3_OBJECT_FOLDER_PREFIX = "bucket_directory1/.../bucket_directoryN/my_file.ext"
# where my_file is the file's name, and ext is its extension.


# Attention: after running this function for connecting with AWS Simple Storage System (S3), 
# your 'AWS Access key ID' and your 'Secret access key' will be requested.
# The 'Secret access key' will be hidden through dots, so it cannot be visualized or copied by
# other users. On the other hand, the same is not true for 'Access key ID', the bucket's name 
# and the prefix. All of these are sensitive information from the organization.
# Therefore, after importing the information, always remember of cleaning the output of this cell
# and of removing such information from the strings.
# Remember that these data may contain privilege for accessing protected information, 
# so it should not be used for non-authorized people.

# Also, remember of deleting the imported files from the workspace after finishing the analysis.
# The costs for storing the files in S3 is quite inferior than those for storing directly in the
# workspace. Also, files stored in S3 may be accessed for other users than those with access to
# the notebook's workspace.
export_files_to_s3 (list_of_file_names_with_extensions = LIST_OF_FILE_NAMES_WITH_EXTENSIONS, directory_of_notebook_workspace_storing_files_to_export = DIRECTORY_OF_NOTEBOOK_WORKSPACE_STORING_FILES_TO_EXPORT, s3_bucket_name = S3_BUCKET_NAME, s3_obj_prefix = S3_OBJECT_FOLDER_PREFIX)

****

# **Statiscal distribution skewness and kurtosis - Background**

### scipy stats.skew()
- scipy.stats.skew(array, axis=0, bias=True) function calculates the skewness of the data set.
    - skewness = 0 : normally distributed.
    - skewness > 0 : more weight in the left tail of the distribution.
    - skewness < 0 : more weight in the right tail of the distribution. 

Its basic formula:

`skewness = (3 *(Mean - Median))/(Standard deviation)`

Parameters :
- array : Input array or object having the elements.
- axis : Axis along which the skewness value is to be measured. By default axis = 0.
- bias : Bool; calculations are corrected for statistical bias, if set to False.
- Returns : Skewness value of the data set, along the axis.

https://www.geeksforgeeks.org/scipy-stats-skew-python/

Scipy uses a more general and complex formula, shown in its documentation:
https://docs.scipy.org/doc/scipy-1.8.0/html-scipyorg/reference/generated/scipy.stats.skew.html

### scipy stats.kurtosis()
- scipy.stats.kurtosis(array, axis=0, fisher=True, bias=True) function calculates the kurtosis (Fisher or Pearson) of a data set. 
- It is the the fourth central moment divided by the square of the variance. 
- It is a measure of the “tailedness” i.e. descriptor of shape of probability distribution of a real-valued random variable. 
- In simple terms, one can say it is a measure of how heavy tail is compared to a normal distribution.

Its formula:

`Kurtosis(X) = E[((X - mu)/sigma)^4]`

Parameters :
- array : Input array or object having the elements.
- axis : Axis along which the kurtosis value is to be measured. By default axis = 0.
- fisher : Bool; Fisher’s definition is used (normal 0.0) if True; else Pearson’s definition is used (normal 3.0) if set to False.
- bias : Bool; calculations are corrected for statistical bias, if set to False.
- Returns : Kurtosis value of the normal distribution for the data set.

https://www.geeksforgeeks.org/scipy-stats-kurtosis-function-python/

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosis.html